In [8]:
pip install python-Levenshtein

     |████████████████████████████████| 50 kB 381 kB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=183203 sha256=868e3cde22c9690e9dc85305cfde5b8d471607893121c13a68c7ba72040539f4
  Stored in directory: /home/ojas_d/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
Note: you may need to restart the kernel to use updated packages.


In [15]:
##Class for extracting the company names from case documents

from bs4 import BeautifulSoup
import os
import pandas as pd
import datetime as dt
from pathlib import Path
from pandas import *
from fuzzywuzzy import process

class extractMetadata():
    def __init__(self, fpath):
        with open(fpath, 'r') as f:
            self.soup = BeautifulSoup(f.read(), 'html.parser').find('div', attrs={'class':"judgments"})
        if(self.soup):
            self.case_text = self.soup.text
            
        else:
            self.case_text=''
        

    def null_check(self, soup_item, attr):
        if attr=='text':
            try:
                return soup_item.text
            except:
                return
        else:
            return soup_item.find(attr)


    def get_court(self):
        if(self.soup):
            source = self.soup.find('div', attrs={'class':'docsource_main'})
        else:
            source=None
        
        self.court = self.null_check(source, 'text')
        return self.court

    def get_pre(self):
        
        pre = self.soup.find('pre')
        self.pre = self.null_check(pre, 'text')
        return self.pre

    def get_title(self):
        if(self.soup):
            title = self.null_check(self.soup.find_all('div', attrs={'class': 'doc_title'})[-1], 'text')
        
        else:
            title=None
            
        if(title):
            self.title = title
            small_title, date = title.split(' on ')
            self.petitioner, self.respondent = small_title.split(' vs ')
            date = dt.datetime.strptime(date.strip(), '%d %B, %Y').date()
            self.date = dt.datetime.strftime(date, '%d-%m-%Y')
        else:
            self.title=None
            self.date=None
            self.petitioner=None
            self.respondent=None
        return self.title

    def get_author(self):
        # NOTE: the author could be mentioned in the pretag when listing the CORAM
        
        if(self.soup):
            author = self.null_check(self.soup.find('div', attrs={'class':'doc_author'}), 'text')
        else:
            author=None
            
        
        if author:
            self.author = author.split('Author:')[-1]
        else:
            self.author = None
        return self.author

    def get_bench(self):
        if(self.soup):
            bench = self.null_check(self.soup.find('div', attrs={'class':'doc_bench'}), 'text')
        else:
            bench=None
        if bench:
            self.bench = bench.split('Bench:')[-1]
        else:
            self.bench = None
        return self.bench

    def get_eq_citations(self):
        if(self.soup):
            eq_citations = self.null_check(self.soup.find('div', attrs={'class':'doc_citations'}), 'text')
        else:
            eq_citations=None
        
        if eq_citations:
            self.eq_citations = eq_citations.split('Equivalent citations:')[-1]
        else:
            self.eq_citations = None
        return self.eq_citations

    def get_jud_order(self):
        # make a guess about whether the document is a judgment or an order
        # 0 if ambiguous, 1 if judgment, -1 if order

        self.jud_order = 0
        if any([x in self.case_text for x in ['JUDGMENT', 'JUDGEMENT']]):
            self.jud_order += 1

        if 'ORDER' in self.case_text:
            self.jud_order+=1
        
        return self.jud_order

    def get_citations(self):
        # note: these citations are to both cases and statutes
        if(self.soup):
            self.citations = [x['href'] for x in self.soup.find_all('a')]
        else:
            self.citations=[]
        
        return self.citations
    
    def check_petitioner_firm(self):
        if(self.soup):
            title = self.null_check(self.soup.find_all('div', attrs={'class': 'doc_title'})[-1], 'text')
        
        else:
            title=None
        if(title):
            self.title=title
            small_title, date = title.split(' on ')
            self.petitioner, self.respondent = small_title.split(' vs ')
            petitioner_upper=self.petitioner.upper()
            query=petitioner_upper.split()
            firm_names=['&', '&DEVELOPERS', '&INFRASTRUCTURE', '&SERVICES', '(BANGALORE)PRIVATE', '(EXPORTS)', '(I)PRIVATE', '(INDIA)LIMITED', '(INDIA)PRIVATE', '(INDIA)PVT', '(OPC)', '(OPC)PRIVATE', '(P)', '(PVT)', ')PRIVATE', 'ABRASIVE', 'ABRASIVES', 'ACADEMIC', 'ACADEMICS', 'ACADEMY', 'ACADEMYPRIVATE', 'ACCENT', 'ACCESSORIES', 'ACCESSORIESPRIVATE', 'ACCOUNTING', 'ACIDS', 'ACOUSTICS', 'ACQUA', 'ADDITIVES', 'ADHESIVE', 'ADHESIVES', 'ADVERTISERS', 'ADVERTISING', 'ADVERTISINGPRIVATE', 'ADVISERS', 'ADVISOR', 'ADVISORS', 'ADVISORSPRIVATE', 'ADVISORY', 'ADVISORYPRIVATE', 'ADVISORYSERVICES', 'AEROSPACE', 'AGENCIES', 'AGENCIESPRIVATE', 'AGENCY', 'AGENCYPRIVATE', 'AGENTS', 'AGGREGATES', 'AGRI-TECH', 'AGRICARE', 'AGRICHEM', 'AGRICO', 'AGRICULTURAL', 'AGRICULTURE', 'AGRIM', 'AGRITECH', 'AGRITRADE', 'AGRO', 'AGRO-FARMS', 'AGRO-TECH', 'AGROCHEM', 'AGROFARM', 'AGROFARMS', 'AGROFOOD', 'AGROFOODS', 'AGROHA', 'AGROPRODUCTS', 'AGROS', 'AGROTECH', 'AGROVET', 'AHEAD', 'AIDS', 'AIRCON', 'AIRCONDITIONING', 'AIRLINES', 'AIRPORT', 'AIRTECH', 'AIRWAYS', 'ALCHEMY', 'ALCO', 'ALKA', 'ALKALIES', 'ALLIANCE', 'ALLIED', 'ALLIEDINDUSTRIES', 'ALLIEDSERVICES', 'ALLOY', 'ALLOYS', 'ALLUMINIUM', 'ALPHA', 'ALPS', 'ALUMINIUM', 'AMALGAMATED', 'AMBAY', 'AMUSEMENTS', 'ANALYSIS', 'ANALYTICAL', 'ANALYTICS', 'ANCHOR', 'ANCILLARIES', 'ANCILLARY', 'ANDCONSTRUCTION', 'ANDCONSTRUCTIONS', 'ANDCONSULTANCY', 'ANDCONSULTANTS', 'ANDDEVELOPERS', 'ANDDEVELOPMENT', 'ANDENGINEERING', 'ANDFINANCE', 'ANDINFRASTRUCTURE', 'ANDINVESTMENT', 'ANDINVESTMENTS', 'ANDMANAGEMENT', 'ANDMARKETING', 'ANDRESEARCH', 'ANDSERVICES', 'ANDSOLUTIONS', 'ANDTECHNOLOGY', 'ANIMATION', 'ANIMATIONS', 'ANNEX', 'APEX', 'APPAREL', 'APPARELS', 'APPLIANCE', 'APPLIANCES', 'APPLIANCESPRIVATE', 'APPLICATION', 'APPLICATIONS', 'APPLIED', 'APPS', 'APT', 'AQUACULTURE', 'AQUATECH', 'AQUATIC', 'ARCADE', 'ARCHITECT', 'ARCHITECTS', 'ARCHITECTURAL', 'ARCHITECTURE', 'ARENA', 'ARMOUR', 'ARMS', 'AROMA', 'AROMATIC', 'AROMATICS', 'ARTS', 'ASBESTOS', 'ASSESSMENT', 'ASSESSORS', 'ASSET', 'ASSETS', 'ASSIST', 'ASSISTANCE', 'ASSOCIATE', 'ASSOCIATED', 'ASSOCIATES', 'ASSOCIATESPRIVATE', 'ASSOCIATION', 'ASSOCIATIONLIMITED', 'AUDIO', 'AUTOCOMP', 'AUTOMATIC', 'AUTOMATION', 'AUTOMATIONS', 'AUTOMOBILE', 'AUTOMOBILES', 'AUTOMOTIVE', 'AUTOMOTIVES', 'AUTOPARTS', 'AUTOS', 'AUTOTECH', 'AUTOWHEELS', 'AVENUE', 'AVENUES', 'AVIATION', 'AYURVED', 'AYURVEDA', 'AYURVEDIC', 'AYURVEDICS', 'AYUSH', 'AYUSHI', 'BAGS', 'BAKERS', 'BAKERY', 'BANK', 'BANQUET', 'BANQUETS', 'BARS', 'BATH', 'BATTERIES', 'BATTERY', 'BAZAAR', 'BAZAR', 'BEANS', 'BEARINGS', 'BEAUTY', 'BELL', 'BELLS', 'BELTING', 'BETA', 'BEVERAGE', 'BEVERAGES', 'BEVERAGESPRIVATE', 'BIKES', 'BINARY', 'BIO', 'BIO-TECH', 'BIOCARE', 'BIOCHEM', 'BIOENERGY', 'BIOFUELS', 'BIOLOGICALS', 'BIOMED', 'BIOMEDICAL', 'BIOPHARMA', 'BIOSCIENCE', 'BIOSCIENCES', 'BIOTEC', 'BIOTECH', 'BIOTECHNOLOGIES', 'BIOTECHNOLOGY', 'BIOTEK', 'BIRDS', 'BISCUIT', 'BISCUITS', 'BITS', 'BIZ', 'BLEACHING', 'BLOCK', 'BLOCKS', 'BLOOM', 'BLOOMING', 'BLOSSOM', 'BLOSSOMS', 'BLOW', 'BOARD', 'BOARDS', 'BODY', 'BOILERS', 'BONANZA', 'BOND', 'BONE', 'BOOK', 'BOOKS', 'BOTTLERS', 'BOTTLING', 'BOUTIQUE', 'BOX', 'BOXES', 'BRAIN', 'BRAINS', 'BRAKES', 'BRAND', 'BRANDING', 'BRANDS', 'BRASS', 'BRAVO', 'BREEDERS', 'BREEDING', 'BREEZE', 'BREWERIES', 'BRICK', 'BRICKS', 'BROAD', 'BROADBAND', 'BROADCAST', 'BROADCASTING', 'BROKER', 'BROKERAGE', 'BROKERS', 'BROKERSPRIVATE', 'BROS', 'BROS.', 'BROTHER', 'BROTHERS', 'BRUSH', 'BUILD-TECH', 'BUILDCOM', 'BUILDCON', 'BUILDCONS', 'BUILDER', 'BUILDERS', 'BUILDERSPRIVATE', 'BUILDESTATE', 'BUILDESTATES', 'BUILDHOME', 'BUILDHOMES', 'BUILDINFRA', 'BUILDING', 'BUILDINGS', 'BUILDMART', 'BUILDMAT', 'BUILDPRO', 'BUILDPROP', 'BUILDSQUARE', 'BUILDTECH', 'BUILDWEL', 'BUILDWELL', 'BUILTECH', 'BULK', 'BULLION', 'BULLIONS', 'BUREAU', 'BUSINESS', 'BUYING', 'BYTE', 'BYTES', 'CABLE', 'CABLES', 'CABS', 'CAFE', 'CALIBRATION', 'CAM', 'CAMELLIA', 'CAMP', 'CANCER', 'CANNING', 'CANS', 'CANVAS', 'CAP', 'CAPACITORS', 'CAPFIN', 'CAPITAL', 'CAPITALS', 'CAPRI', 'CAPRICORN', 'CAPS', 'CAPSTONE', 'CAPSULES', 'CAR', 'CARAVAN', 'CARBIDE', 'CARBIDES', 'CARBON', 'CARBONS', 'CARDIAC', 'CARDS', 'CARE', 'CAREER', 'CAREERS', 'CAREPRIVATE', 'CAREWELL', 'CARGO', 'CARNATION', 'CARNIVAL', 'CARPET', 'CARPETS', 'CARRIER', 'CARRIERS', 'CARS', 'CART', 'CARTONS', 'CASA', 'CASCADE', 'CASH', 'CASHEW', 'CASSETTES', 'CAST', 'CASTING', 'CASTINGS', 'CASTLE', 'CASTOR', 'CATALYST', 'CATERERS', 'CATERING', 'CELL', 'CELLULAR', 'CELLULOSE', 'CEMENT', 'CEMENTS', 'CENTER', 'CENTERPRIVATE', 'CENTERS', 'CENTRAL', 'CENTRE', 'CENTREPRIVATE', 'CENTRES', 'CENTURION', 'CENTURY', 'CERA', 'CERAMIC', 'CERAMICS', 'CEREALS', 'CERTIFICATION', 'CERTIFICATIONS', 'CHAIN', 'CHAINS', 'CHAMBER', 'CHAMBERS', 'CHANGER', 'CHANGERS', 'CHANNEL', 'CHANNELS', 'CHARAN', 'CHARTERED', 'CHARTERING', 'CHEF', 'CHEM', 'CHEMI', 'CHEMICAL', 'CHEMICALS', 'CHEMICALSPRIVATE', 'CHEMIE', 'CHEMTECH', 'CHIP', 'CHIPS', 'CHIT', 'CHITFUND', 'CHITFUNDS', 'CHITS', 'CHITTIES', 'CHITTY', 'CHROME', 'CINE', 'CINEMA', 'CINEMAS', 'CIRCUIT', 'CIRCUITS', 'CLEANERS', 'CLEANING', 'CLEANTECH', 'CLEARING', 'CLINIC', 'CLINICAL', 'CLINICS', 'CLOTHING', 'CLOTHINGS', 'CLOUD', 'CLUB', 'CLUBS', 'CO.', 'CO.LTD.', 'CO.PRIVATE', 'CO.PVT', 'CO.PVT.LTD.', 'COACH', 'COACHING', 'COASTAL', 'COAT', 'COATED', 'COATERS', 'COATING', 'COATINGS', 'COATS', 'CODE', 'COFFEE', 'COGENT', 'COIN', 'COIR', 'COKE', 'COLDSTORAGE', 'COLLECTION', 'COLLECTIONS', 'COLLECTIVE', 'COLLEGE', 'COLLIERIES', 'COLLIERY', 'COLONISERS', 'COLONIZER', 'COLONIZERS', 'COLOR', 'COLORS', 'COLOUR', 'COLOURS', 'COMFORTS', 'COMM', 'COMMERCE', 'COMMERCIAL', 'COMMERCIALS', 'COMMODEAL', 'COMMODITIES', 'COMMODITY', 'COMMOSALE', 'COMMOSALES', 'COMMOTRADE', 'COMMUNICATION', 'COMMUNICATIONPRIVATE', 'COMMUNICATIONS', 'COMMUNICATIONSPRIVATE', 'COMMUNITY', 'COMNET', 'COMP', 'COMPACT', 'COMPANY', 'COMPANYLIMITED', 'COMPANYPRIVATE', 'COMPANYPVT', 'COMPANYPVT.LTD.', 'COMPASS', 'COMPETENT', 'COMPLETE', 'COMPLEX', 'COMPLIANCE', 'COMPNAY', 'COMPONENT', 'COMPONENTS', 'COMPONENTSPRIVATE', 'COMPOSITE', 'COMPOSITES', 'COMPREHENSIVE', 'COMPRESSORS', 'COMPTECH', 'COMPU', 'COMPUSOFT', 'COMPUTECH', 'COMPUTER', 'COMPUTERS', 'COMPUTING', 'COMPUTRONICS', 'COMTECH', 'COMTRADE', 'CONBUILD', 'CONCAST', 'CONCEPT', 'CONCEPTS', 'CONCERN', 'CONCLAVE', 'CONCORD', 'CONCORDE', 'CONCRETE', 'CONCRETES', 'CONDITIONING', 'CONDUCTORS', 'CONES', 'CONFECTIONERS', 'CONFECTIONERY', 'CONFIDENT', 'CONGLOMERATE', 'CONNECT', 'CONNECTION', 'CONNECTIONS', 'CONNECTORS', 'CONSERVATION', 'CONSOLIDATED', 'CONSORTIUM', 'CONSTECH', 'CONSTRO', 'CONSTRUCTION', 'CONSTRUCTIONCOMPANY', 'CONSTRUCTIONPRIVATE', 'CONSTRUCTIONS', 'CONSTRUCTIONSPRIVATE', 'CONSTRUCTORS', 'CONSTRUWELL', 'CONSULT', 'CONSULTANCIES', 'CONSULTANCY', 'CONSULTANCYPRIVATE', 'CONSULTANCYSERVICES', 'CONSULTANT', 'CONSULTANTPRIVATE', 'CONSULTANTS', 'CONSULTANTSPRIVATE', 'CONSULTING', 'CONSULTINGPRIVATE', 'CONSULTINGSERVICES', 'CONSUMABLES', 'CONSUMER', 'CONSUMERS', 'CONTACT', 'CONTAINER', 'CONTAINERS', 'CONTECH', 'CONTEMPORARY', 'CONTINENTAL', 'CONTRACT', 'CONTRACTING', 'CONTRACTOR', 'CONTRACTORS', 'CONTRACTORSPRIVATE', 'CONTRACTS', 'CONTROLS', 'CONVENTION', 'CONVERGENCE', 'CONVERSION', 'CONVERTERS', 'CONVERTORS', 'CONVEYOR', 'CONVEYORS', 'COOLING', 'COPPER', 'COPRIVATE', 'COPVT', 'CORAL', 'CORE', 'CORN', 'CORNER', 'CORNERSTONE', 'COROMANDEL', 'CORONATION', 'CORP', 'CORPN', 'CORPN.', 'CORPORATE', 'CORPORATION', 'CORPORATIONLIMITED', 'CORPORATIONPRIVATE', 'CORRUGATED', 'CORRUGATORS', 'COSMETIC', 'COSMETICS', 'COSMIC', 'COSMO', 'COSMOPOLITAN', 'COSMOS', 'COST', 'COTEX', 'COTSPIN', 'COTSYN', 'COTTAGE', 'COTTAGES', 'COTTON', 'COTTONS', 'COUNCIL', 'COUNTRY', 'COUNTRYWIDE', 'COUNTY', 'COURIER', 'COURIERS', 'COUTURE', 'COZY', 'CRAFT', 'CRAFTS', 'CREAMS', 'CREATION', 'CREATIONS', 'CREATIVE', 'CREATIVES', 'CREATORS', 'CREDIT', 'CREDITS', 'CROP', 'CROPS', 'CROSS', 'CROWN', 'CRUSHER', 'CRUSHERS', 'CRUSHING', 'CRYSTAL', 'CRYSTALS', 'CUISINE', 'CUISINES', 'CULINARY', 'CULTIVATION', 'CULTIVATORS', 'CULTURAL', 'CULTURE', 'CURE', 'CURRENCY', 'CUSTOMER', 'CUTTING', 'CYBER', 'CYBERNETICS', 'CYBERTECH', 'CYCLE', 'CYCLES', 'CYLINDERS', 'DAILY', 'DAIRIES', 'DAIRY', 'DATA', 'DATABASE', 'DATACOM', 'DATAMATICS', 'DATASOFT', 'DEALCOM', 'DEALCOMM', 'DEALER', 'DEALERS', 'DEALMARK', 'DEALS', 'DEALTRADE', 'DEBT', 'DECO', 'DECOR', 'DECORATIVE', 'DECORATORS', 'DECORS', 'DEFENCE', 'DEGREE', 'DELIVERY', 'DELUX', 'DELUXE', 'DEPARTMENTAL', 'DEPOSIT', 'DEPOSITS', 'DEPOT', 'DERIVATIVES', 'DESIGN', 'DESIGNER', 'DESIGNERS', 'DESIGNING', 'DESIGNS', 'DETECTIVE', 'DETECTIVES', 'DETERGENT', 'DETERGENTS', 'DEVBUILD', 'DEVCON', 'DEVELOPER', 'DEVELOPERPRIVATE', 'DEVELOPERS', 'DEVELOPERSLIMITED', 'DEVELOPERSPRIVATE', 'DEVELOPMENT', 'DEVELOPMENTCOMPANY', 'DEVELOPMENTCORPORATION', 'DEVELOPMENTLIMITED', 'DEVELOPMENTPRIVATE', 'DEVELOPMENTS', 'DIAGNOSTIC', 'DIAGNOSTICS', 'DIAM', 'DIAMOND', 'DIAMONDS', 'DIARY', 'DIES', 'DIESEL', 'DIESELS', 'DIGI', 'DIGITAL', 'DIGITECH', 'DIGNITY', 'DIMENSION', 'DIMENSIONS', 'DISCOUNT', 'DISPLAY', 'DISTILLERIES', 'DISTILLERS', 'DISTILLERY', 'DISTRIBUTING', 'DISTRIBUTION', 'DISTRIBUTIONPRIVATE', 'DISTRIBUTIONS', 'DISTRIBUTOR', 'DISTRIBUTORS', 'DISTRIBUTORSPRIVATE', 'DOCTORS', 'DOMAIN', 'DOMESTIC', 'DOORS', 'DOT', 'DOTCOM', 'DOTS', 'DP', 'DRESSES', 'DRILLING', 'DRINKS', 'DRUG', 'DRUGS', 'DRUMS', 'DWELLERS', 'DWELLING', 'DWELLINGS', 'DYE', 'DYE-CHEM', 'DYECHEM', 'DYEING', 'DYERS', 'DYES', 'DYESTUFF', 'DYESTUFFS', 'E-COM', 'E-COMMERCE', 'E-LEARNING', 'E-SERVICES', 'E-SOLUTIONS', 'EARTHMOVERS', 'EATABLES', 'ECO', 'ECOM', 'ECOMMERCE', 'ECONOMIC', 'ECOTECH', 'EDIBLE', 'EDIBLES', 'EDUCARE', 'EDUCATION', 'EDUCATIONAL', 'EDUCATIONPRIVATE', 'EDUCATIONS', 'EDUCOM', 'EDUCON', 'EDUSOFT', 'EDUSOLUTIONS', 'EDUTAINMENT', 'EDUTECH', 'EDUVENTURES', 'EFFLUENT', 'ELECTRIC', 'ELECTRICAL', 'ELECTRICALS', 'ELECTRICALSPRIVATE', 'ELECTRICITY', 'ELECTRICS', 'ELECTRO', 'ELECTRODES', 'ELECTROMECH', 'ELECTRONIC', 'ELECTRONICS', 'ELECTRONICSPRIVATE', 'ELECTROTECH', 'ELEMENTS', 'ELEVATOR', 'ELEVATORS', 'EMBROIDERIES', 'EMBROIDERY', 'EMERGENCY', 'EMERGING', 'EMPLOYEES', 'EMPLOYMENT', 'EMPORIUM', 'EMPOWER', 'ENCLAVE', 'ENCLAVES', 'ENERGIES', 'ENERGY', 'ENERGYPRIVATE', 'ENERTECH', 'ENFIELD', 'ENGG', 'ENGG.', 'ENGICON', 'ENGICONS', 'ENGINE', 'ENGINEER', 'ENGINEERING', 'ENGINEERINGCOMPANY', 'ENGINEERINGPRIVATE', 'ENGINEERINGS', 'ENGINEERINGSERVICES', 'ENGINEERS', 'ENGINEERSPRIVATE', 'ENGINES', 'ENGITECH', 'ENTERPRISE', 'ENTERPRISES', 'ENTERPRISESPRIVATE', 'ENTERTAINERS', 'ENTERTAINMENT', 'ENTERTAINMENTPRIVATE', 'ENTERTAINMENTS', 'ENTREPRENEUR', 'ENTREPRENEURS', 'ENVIROTECH', 'ENVISION', 'EQUIPMENT', 'EQUIPMENTS', 'EQUIPMENTSPRIVATE', 'EQUITIES', 'EQUITY', 'ESCALATORS', 'ESCORTS', 'ESERVICES', 'ESOLUTIONS', 'ESQUIRE', 'ESSENTIALS', 'ESTATE', 'ESTATEPRIVATE', 'ESTATES', 'ESTATESPRIVATE', 'EX-SERVICEMEN', 'EXECUTIVE', 'EXHIBITION', 'EXHIBITIONS', 'EXHIBITORS', 'EXPEDITIONS', 'EXPERIENCE', 'EXPERT', 'EXPERTS', 'EXPLORATION', 'EXPLOSIVES', 'EXPO', 'EXPORT', 'EXPORTERS', 'EXPORTS', 'EXPORTSPRIVATE', 'EXPOSITIONS', 'EXPOTRADE', 'EXTRACTION', 'EXTRACTIONS', 'EXTRACTS', 'EXTRUSION', 'EXTRUSIONS', 'FABRIC', 'FABRICATION', 'FABRICATIONS', 'FABRICATORS', 'FABRICS', 'FABS', 'FABTECH', 'FABTEX', 'FACADE', 'FACILITATORS', 'FACILITIES', 'FACILITY', 'FACILITYMANAGEMENT', 'FACTOR', 'FACTORS', 'FACTORY', 'FACTORYPRIVATE', 'FARMER', 'FARMERS', 'FARMERSPRODUCER', 'FARMING', 'FARMS', 'FARMSPRIVATE', 'FASHION', 'FASHIONS', 'FASTENERS', 'FASTNERS', 'FASTSPEED', 'FATS', 'FEED', 'FEEDS', 'FERROUS', 'FERTILISERS', 'FERTILITY', 'FERTILIZER', 'FERTILIZERS', 'FERTILIZERSPRIVATE', 'FIBER', 'FIBERS', 'FIBRE', 'FIBRES', 'FIELD', 'FIELDS', 'FILAMENT', 'FILAMENTS', 'FILM', 'FILMS', 'FILTER', 'FILTERS', 'FILTRATION', 'FIN-LEASE', 'FINANCE', 'FINANCECOMPANY', 'FINANCELIMITED', 'FINANCEPRIVATE', 'FINANCERS', 'FINANCES', 'FINANCIAL', 'FINANCIALCONSULTANTS', 'FINANCIALS', 'FINANCIALSERVICES', 'FINANCIERS', 'FINANCING', 'FINCAP', 'FINCO', 'FINCOM', 'FINCON', 'FINCORP', 'FINE', 'FINECHEM', 'FINESSE', 'FININVEST', 'FINISHERS', 'FINISHING', 'FINLEASE', 'FINMART', 'FINSEC', 'FINSERV', 'FINSERVE', 'FINSTOCK', 'FINTECH', 'FINTRADE', 'FINVEST', 'FISH', 'FISHERIES', 'FISHING', 'FIT', 'FITNESS', 'FITTINGS', 'FIXTURES', 'FLAT', 'FLATS', 'FLAVOURS', 'FLEET', 'FLEX', 'FLEXI', 'FLEXIBLE', 'FLEXIPACK', 'FLIGHT', 'FLOOR', 'FLOORING', 'FLOORINGS', 'FLOORS', 'FLORA', 'FLORAL', 'FLORENCE', 'FLORICULTURE', 'FLORITECH', 'FLOUR', 'FLOW', 'FLOWER', 'FLOWERS', 'FLUID', 'FLYING', 'FOAM', 'FOAMS', 'FOILS', 'FOOD', 'FOODS', 'FOODSPRIVATE', 'FOODTECH', 'FOOT', 'FOOTBALL', 'FOOTCARE', 'FOOTWEAR', 'FOOTWEARS', 'FORESTRY', 'FOREX', 'FORGE', 'FORGING', 'FORGINGS', 'FORMING', 'FORMS', 'FORMULATION', 'FORMULATIONS', 'FORTUNE', 'FORUM', 'FORWARDERS', 'FOSTER', 'FOTO', 'FOUNDATION', 'FOUNDATIONS', 'FOUNDERS', 'FOUNDRIES', 'FOUNDRY', 'FRAGRANCE', 'FRAGRANCES', 'FRAME', 'FRAMES', 'FRANCHISE', 'FREIGHT', 'FROZEN', 'FRUIT', 'FRUITS', 'FUEL', 'FUELS', 'FUND', 'FUNDLIMITED', 'FUNDPRIVATE', 'FUNDS', 'FUNDSPRIVATE', 'FUNIDEA', 'FURNACE', 'FURNACES', 'FURNISHERS', 'FURNISHING', 'FURNISHINGS', 'FURNITURE', 'FURNITURES', 'FUTURES', 'GADGETS', 'GAIN', 'GALLERY', 'GAME', 'GAMES', 'GAMING', 'GARDENS', 'GARMENT', 'GARMENTS', 'GAS', 'GASES', 'GASKETS', 'GATEWAY', 'GAUGES', 'GEAR', 'GEARS', 'GEMS', 'GENERATORS', 'GENETIC', 'GENETICS', 'GENUINE', 'GEOTECH', 'GIFTS', 'GINNING', 'GLASS', 'GLASSES', 'GLAZE', 'GLAZING', 'GLOVES', 'GOODS', 'GRAINS', 'GRANITE', 'GRANITES', 'GRAPES', 'GRAPHICS', 'GRAPHITE', 'GREENTECH', 'GREY', 'GROUP', 'HABITATS', 'HANDICRAFT', 'HANDICRAFTS', 'HANDLING', 'HANDLOOM', 'HANDLOOMS', 'HARBOUR', 'HARDWARE', 'HARDWARES', 'HEALTHCARE', 'HEALTHTECH', 'HELPLINE', 'HERBALS', 'HERBS', 'HERITAGE', 'HI-TECH', 'HITECH', 'HOLDING', 'HOLDINGS', 'HOLDINGSPRIVATE', 'HOLIDAYS', 'HOSPITAL', 'HOSPITALITIES', 'HOSPITALITY', 'HOSPITALITYPRIVATE', 'HOSPITALPRIVATE', 'HOSPITALS', 'HOSPITALSPRIVATE', 'HOTELS', 'HOTELSPRIVATE', 'HOUSING', 'HYBRID', 'HYDEL', 'HYDRAULIC', 'HYDRAULICS', 'HYDRO', 'HYDROCARBONS', 'HYGIENE', 'HYGIENIC', 'I-TECH', 'I.T.', 'IMAGINE', 'IMAGING', 'IMPORT', 'IMPORTERS', 'IMPORTS', 'INDUSTRIAL', 'INDUSTRIALS', 'INDUSTRIES', 'INDUSTRIESLIMITED', 'INDUSTRIESPRIVATE', 'INDUSTRIESPVT', 'INDUSTRY', 'INFIN', 'INFINITE', 'INFINITI', 'INFINITY', 'INFO', 'INFO-TECH', 'INFOCARE', 'INFOCOM', 'INFOCOMM', 'INFOMATICS', 'INFOMEDIA', 'INFORMATIC', 'INFORMATICS', 'INFORMATION', 'INFORMATIONTECHNOLOGY', 'INFOSERVE', 'INFOSERVICES', 'INFOSOFT', 'INFOSOL', 'INFOSOLUTION', 'INFOSOLUTIONS', 'INFOSYS', 'INFOSYSTEM', 'INFOSYSTEMS', 'INFOTAINMENT', 'INFOTEC', 'INFOTECH', 'INFOTEK', 'INFOTEL', 'INFOVISION', 'INFOWARE', 'INFOWAY', 'INFOWAYS', 'INFOWEB', 'INFRA', 'INFRA-TECH', 'INFRAA', 'INFRABUILD', 'INFRABUILDCON', 'INFRABUILDERS', 'INFRACARE', 'INFRACITY', 'INFRACOM', 'INFRACON', 'INFRACONS', 'INFRACONSTRUCTION', 'INFRADEV', 'INFRADEVELOPER', 'INFRADEVELOPERS', 'INFRAENGINEERS', 'INFRAESTATE', 'INFRAESTATES', 'INFRAHEIGHT', 'INFRAPROJECT', 'INFRAPROJECTS', 'INFRAREALTY', 'INFRASERVICES', 'INFRASOLUTION', 'INFRASOLUTIONS', 'INFRASPACE', 'INFRASTRUCTURAL', 'INFRASTRUCTURE', 'INFRASTRUCTUREDEVELOPMENT', 'INFRASTRUCTURELIMITED', 'INFRASTRUCTUREPRIVATE', 'INFRASTRUCTURES', 'INFRASTRUCTURESPRIVATE', 'INFRATEC', 'INFRATECH', 'INFRATEL', 'INFRATRADE', 'INFRAVENTURE', 'INFRAVENTURES', 'INFRAVISION', 'INFRAZONE', 'INGENIOUS', 'INGOTS', 'INGREDIENTS', 'INITIATIVE', 'INITIATIVES', 'INK', 'INKS', 'INLAND', 'INN', 'INNOTECH', 'INNOVA', 'INNOVATION', 'INNOVATIONS', 'INNOVATIVE', 'INNOVATORS', 'INNOVISION', 'INORGANICS', 'INSECTICIDES', 'INSPECTION', 'INSTITUTE', 'INSTITUTEPRIVATE', 'INSTITUTES', 'INSTITUTION', 'INSTITUTIONS', 'INSTRUMENT', 'INSTRUMENTATION', 'INSTRUMENTS', 'INSULATION', 'INSULATIONS', 'INSULATORS', 'INSURANCE', 'INTECH', 'INTEGRA', 'INTEGRAL', 'INTEGRATED', 'INTEGRATION', 'INTEGRATORS', 'INTEGRITY', 'INTEL', 'INTELLECT', 'INTELLECTUAL', 'INTELLIGENCE', 'INTELLIGENT', 'INTER', 'INTERACTIVE', 'INTERCONTINENTAL', 'INTERFACE', 'INTERIORS', 'INTERLINK', 'INTERMEDIATES', 'INTERNATIONAL', 'INTERNATIONALPRIVATE', 'INTERNET', 'INTERTRADE', 'INTIMATE', 'INTIME', 'INTRA', 'INTRADE', 'INVENTIVE', 'INVEST', 'INVESTIGATION', 'INVESTMENT', 'INVESTMENTCOMPANY', 'INVESTMENTLIMITED', 'INVESTMENTPRIVATE', 'INVESTMENTS', 'INVESTMENTSLIMITED', 'INVESTMENTSPRIVATE', 'INVESTOR', 'INVESTORS', 'INVESTRADE', 'IRRIGATION', 'JEWEL', 'JEWELLERS', 'JEWELLERSPRIVATE', 'JEWELLERY', 'JEWELRY', 'JEWELS', 'KITCHEN', 'KITCHENS', 'KNIT', 'KNITS', 'KNITTERS', 'KNITTING', 'KNITWEAR', 'KNITWEARS', 'LAB', 'LABEL', 'LABELS', 'LABORATORIES', 'LABORATORIESPRIVATE', 'LABORATORY', 'LABOUR', 'LABS', 'LAMINATES', 'LAMINATIONS', 'LAMINATORS', 'LAMP', 'LAMPS', 'LANDDEVELOPERS', 'LATEX', 'LAUNDRY', 'LAW', 'LEAD', 'LEADING', 'LEARNING', 'LEASE', 'LEASING', 'LEASINGLIMITED', 'LEASINGPRIVATE', 'LEASINGS', 'LEATHER', 'LEATHERS', 'LEISURES', 'LIFESTYLE', 'LIFESTYLES', 'LIFT', 'LIFTS', 'LIGHT', 'LIGHTING', 'LIGHTINGS', 'LIGHTS', 'LIMITED', 'LIMITED(OPC)', 'LIMITED,', 'LIMITED.', 'LIMTED', 'LIMTIED', 'LINERS', 'LINES', 'LINK', 'LINKERS', 'LINKS', 'LIQUOR', 'LIQUORS', 'LOAN', 'LOANS', 'LOCKERS', 'LOCKS', 'LODGE', 'LOGIC', 'LOGICAL', 'LOGICS', 'LOGISTIC', 'LOGISTICS', 'LOGISTICSPRIVATE', 'LTD', 'LTD,', 'LTD.', 'LTD.,', 'LUBE', 'LUBES', 'LUBRICANTS', 'LUGGAGE', 'LUMBERS', 'MACHINE', 'MACHINERIES', 'MACHINERY', 'MACHINES', 'MACHINING', 'MACRO', 'MAGNET', 'MAGNETIC', 'MAGNETICS', 'MAINTENANCE', 'MAKERS', 'MALLS', 'MANAGED', 'MANAGEMENT', 'MANAGEMENTCONSULTANTS', 'MANAGEMENTPRIVATE', 'MANAGEMENTS', 'MANAGEMENTSERVICES', 'MANAGERS', 'MANAGMENT', 'MANUFACTURERS', 'MANUFACTURERSPRIVATE', 'MANUFACTURES', 'MANUFACTURING', 'MANUFACTURINGCOMPANY', 'MANUFACTURINGPRIVATE', 'MARBLE', 'MARBLES', 'MARKET', 'MARKETERS', 'MARKETING', 'MARKETINGPRIVATE', 'MARKETINGS', 'MARKETINGSERVICES', 'MARKETPLACE', 'MARKETS', 'MARKETTING', 'MATCH', 'MATCHES', 'MATE', 'MATERIAL', 'MATERIALS', 'MATRIMONIAL', 'MECH', 'MECHANICAL', 'MECHANICALS', 'MECHATRONICS', 'MED', 'MEDIA', 'MEDIATECH', 'MEDICAL', 'MEDICALS', 'MEDICAMENTS', 'MEDICINE', 'MEDICINES', 'MEDICO', 'MEDICOS', 'MEDITECH', 'MEDTECH', 'MEENA', 'MEENAKSHI', 'MEERA', 'MEERUT', 'MEET', 'MEGA', 'MEGACITY', 'MEGAMART', 'MEGH', 'MEGHA', 'MEGHALAYA', 'MEGHANA', 'MEGHDOOT', 'MEGHNA', 'MEHAK', 'MEHAR', 'MEHER', 'MEHRA', 'MEHTA', 'MEHUL', 'MELANGE', 'MELODY', 'MEMORIAL', 'MEMORY', 'MEN', 'MENON', 'MENTOR', 'MENTORS', 'MEP', 'MERA', 'MERCANTILE', 'MERCANTILES', 'MERCHANDISE', 'MERCHANDISERS', 'MERCHANDISING', 'MERCHANT', 'MERCHANTILE', 'MERCHANTS', 'MERCURY', 'MERIDIAN', 'MERIT', 'MERLIN', 'MERRY', 'MERU', 'MESH', 'MET', 'META', 'METACAST', 'METAL', 'METALIKS', 'METALLIC', 'METALLICS', 'METALLOYS', 'METALLURGICAL', 'METALLURGICALS', 'METALS', 'METERS', 'METRO', 'METROPOLIS', 'METROPOLITAN', 'MEWAR', 'MFG', 'MFG.', 'MG', 'MICA', 'MICRO', 'MICROFINANCE', 'MICRON', 'MICRONS', 'MICROSYSTEMS', 'MICROTECH', 'MIDAS', 'MIDDLE', 'MIDLAND', 'MIDWAY', 'MIGRATION', 'MILE', 'MILES', 'MILESTONE', 'MILK', 'MILL', 'MILLENIUM', 'MILLENNIUM', 'MILLERS', 'MILLING', 'MILLS', 'MILLSLIMITED', 'MILLSPRIVATE', 'MINAR', 'MINCHEM', 'MIND', 'MINDA', 'MINDS', 'MINE', 'MINECHEM', 'MINERAL', 'MINERALS', 'MINERALSPRIVATE', 'MINERVA', 'MINES', 'MINI', 'MINING', 'MINT', 'MIR', 'MIRA', 'MIRACLE', 'MIRAGE', 'MIRAJ', 'MIRROR', 'MISSION', 'MIST', 'MOBILE', 'MOBILES', 'MOBILITY', 'MOBITECH', 'MOD', 'MODE', 'MODEL', 'MODULAR', 'MOMENTUM', 'MONARCH', 'MONEY', 'MONITORING', 'MONSOON', 'MOON', 'MOONLIGHT', 'MOONLINK', 'MOONSHINE', 'MOONSTAR', 'MOSAIC', 'MOTEL', 'MOTELS', 'MOTION', 'MOTOR', 'MOTORS', 'MOULD', 'MOULDERS', 'MOULDING', 'MOULDINGS', 'MOULDS', 'MOVERS', 'MOVIE', 'MOVIES', 'MOVING', 'MULTI', 'MULTI-TRADE', 'MULTICON', 'MULTILEVEL', 'MULTILINK', 'MULTIMEDIA', 'MULTIPLE', 'MULTIPLEX', 'MULTIPROJECTS', 'MULTIPURPOSE', 'MULTISERVICES', 'MULTISOLUTIONS', 'MULTISPECIALITY', 'MULTITECH', 'MULTITRADE', 'MULTITRADING', 'MULTIVENTURES', 'NANOTECH', 'NATURALS', 'NEOTECH', 'NETCOM', 'NETLINK', 'NETMART', 'NETS', 'NETWORK', 'NETWORKING', 'NETWORKPRIVATE', 'NETWORKS', 'NETWORTH', 'NEURO', 'NURSING', 'NUTRACEUTICALS', 'NUTRI', 'NUTRIENTS', 'NUTRITION', 'NUTRITIONS', 'OIL', 'OILFIELD', 'OILS', 'OLIVE', 'ONCOLOGY', 'OPERATORS', 'OPTION', 'OPTIONS', 'ORCHID', 'ORCHIDS', 'ORGANISATION', 'ORGANISERS', 'ORGANIZATION', 'ORIENTAL', 'ORNATE', 'ORTHO', 'OUTSOURCE', 'OUTSOURCING', 'OVERSEAS', 'OWNERS', 'OXYGEN', 'OYSTER', 'OZONE', 'P.LTD.', 'PACK', 'PACKAGERS', 'PACKAGES', 'PACKAGING', 'PACKAGINGS', 'PACKERS', 'PACKING', 'PACKS', 'PAGES', 'PAINT', 'PAINTS', 'PALACE', 'PALMS', 'PANEL', 'PANELS', 'PAPER', 'PAPERS', 'PARADISE', 'PARAM', 'PARKPRIVATE', 'PARTNERS', 'PAYMENT', 'PAYMENTS', 'PEARL', 'PEARLS', 'PELICAN', 'PENS', 'PEOPLE', 'PEOPLES', 'PERFUMES', 'PERIPHERALS', 'PERSONNEL', 'PEST', 'PESTICIDES', 'PET', 'PETALS', 'PETRO', 'PETROCHEM', 'PETROCHEMICALS', 'PETROLEUM', 'PETROLEUMS', 'PHARMA', 'PHARMACEUTICAL', 'PHARMACEUTICALS', 'PHARMACEUTICALSPRIVATE', 'PHARMACHEM', 'PHARMACY', 'PHARMATECH', 'PHOENIX', 'PHOSPHATES', 'PHOTO', 'PHOTOGRAPHY', 'PHOTON', 'PICTURE', 'PICTURES', 'PIGMENTS', 'PILOT', 'PIPE', 'PIPES', 'PIPING', 'PLACEMENT', 'PLACEMENTS', 'PLANNER', 'PLANNERS', 'PLANNING', 'PLANTATION', 'PLANTATIONS', 'PLANTERS', 'PLANTS', 'PLASMA', 'PLASTECH', 'PLASTIC', 'PLASTICS', 'PLASTO', 'PLASTOCHEM', 'PLATFORM', 'PLATINUM', 'PLAY', 'PLAZA', 'PLAZZA', 'PLEASANT', 'PLEASURE', 'PLEXUS', 'PLYWOOD', 'PLYWOODS', 'PNEUMATICS', 'POINT', 'POLAR', 'POLARIS', 'POLE', 'POLES', 'POLESTAR', 'POLLUTION', 'POLY', 'POLYCHEM', 'POLYCLINIC', 'POLYCOATS', 'POLYESTER', 'POLYESTERS', 'POLYFAB', 'POLYFILMS', 'POLYGON', 'POLYMER', 'POLYMERS', 'POLYPACK', 'POLYPACKS', 'POLYPLAST', 'POLYSACKS', 'POLYTECH', 'POLYTEX', 'PONDICHERRY', 'PORT', 'PORTFOLIOS', 'PORTS', 'PORWAL', 'POST', 'POTENTIAL', 'POULTRIES', 'POULTRY', 'POWDER', 'POWDERS', 'POWERGEN', 'POWERS', 'POWERTECH', 'POWERTRONICS', 'PPRIVATE', 'PRECISIONS', 'PREMIUM', 'PRESS', 'PRESSINGS', 'PRESSURE', 'PRESTIGE', 'PRIAVTE', 'PRIMARY', 'PRIME', 'PRINTING', 'PRINTPACK', 'PRINTS', 'PRISM', 'PRIVATE', 'PRIVATELIMITED', 'PRIVATELIMITED.', 'PRIVATELTD', 'PROACTIVE', 'PROBUILD', 'PROCESS', 'PROCESSED', 'PROCESSES', 'PROCESSING', 'PROCESSINGPRIVATE', 'PROCESSOR', 'PROCESSORS', 'PROCON', 'PRODIGY', 'PRODUCE', 'PRODUCER', 'PRODUCERCOMPANY', 'PRODUCERS', 'PRODUCT', 'PRODUCTION', 'PRODUCTIONS', 'PRODUCTS', 'PRODUCTSLIMITED', 'PRODUCTSPRIVATE', 'PROFESSIONAL', 'PROFESSIONALS', 'PROFICIENT', 'PROFILE', 'PROFILES', 'PROFIT', 'PROGRESS', 'PROGRESSIVE', 'PROJECT', 'PROJECTS', 'PROJECTSLIMITED', 'PROJECTSPRIVATE', 'PROLIFIC', 'PROMINENT', 'PROMOTER', 'PROMOTERS', 'PROMOTERSPRIVATE', 'PROMOTION', 'PROMOTIONS', 'PROMOTORS', 'PROMPT', 'PROPBUILD', 'PROPERTIES', 'PROPERTIESPRIVATE', 'PROPERTY', 'PROPMART', 'PROPTECH', 'PROSPERITY', 'PROTECH', 'PROTECT', 'PROTECTIVE', 'PROTEINS', 'PROVIDENT', 'PROVIDER', 'PROVIDERS', 'PUBLICATION', 'PUBLICATIONS', 'PUBLICATIONSPRIVATE', 'PUBLISHERS', 'PUBLISHING', 'PULSES', 'PUMP', 'PUMPS', 'PVT.', 'PVT.LTD', 'PVT.LTD.', 'PVTLTD', 'PVTLTD.', 'QUALITY', 'QUARRIES', 'QUARRY', 'QUARTZ', 'QUEEN', 'RACING', 'RADIATORS', 'RAYON', 'RAYONS', 'REAL', 'REALATORS', 'REALBUILD', 'REALCON', 'REALESTATE', 'REALESTATES', 'REALHOME', 'REALINFRA', 'REALITIES', 'REALITY', 'REALMART', 'REALTECH', 'REALTERS', 'REALTIES', 'REALTIME', 'REALTOR', 'REALTORS', 'REALTORSPRIVATE', 'REALTY', 'RECKON', 'RECON', 'RECONSTRUCTION', 'RECORDING', 'RECORDS', 'RECREATION', 'RECREATIONS', 'RECRUITERS', 'RECRUITMENT', 'RECYCLERS', 'RECYCLING', 'REFINERIES', 'REFINERY', 'REFRACTORIES', 'REFRACTORY', 'REFRIGERATION', 'REMEDIES', 'RENAISSANCE', 'RENEWABLE', 'RENEWABLES', 'REROLLING', 'RESEARCH', 'RESEARCHCENTRE', 'RESEARCHFOUNDATION', 'RESEARCHINSTITUTE', 'RESEARCHPRIVATE', 'RESIDENCES', 'RESIDENCY', 'RESIDENTIAL', 'RESINS', 'RESONANCE', 'RESORT', 'RESORTS', 'RESORTSPRIVATE', 'RESOURCE', 'RESOURCES', 'RESOURCESPRIVATE', 'RESPONSE', 'RESTAURANT', 'RESTAURANTS', 'RETAIL', 'RETAILERS', 'RETAILING', 'RETAILS', 'RETREADING', 'RETREADS', 'RETREAT', 'RETREATS', 'ROADLINES', 'ROADS', 'ROADWAYS', 'ROBOTICS', 'RUBBER', 'RUBBERS', 'RUIA', 'SAPHIRE', 'SAPPHIRE', 'SAR', 'SAREE', 'SAREES', 'SATELLITE', 'SAVING', 'SAVINGS', 'SAW', 'SCAFFOLDING', 'SCAN', 'SCANS', 'SCHEME', 'SCHOOL', 'SCHOOLS', 'SCIENCE', 'SCIENCES', 'SCIENCESPRIVATE', 'SCIENTIFIC', 'SEAFOODS', 'SEAL', 'SEALS', 'SECURITIES', 'SECURITIESPRIVATE', 'SECURITY', 'SECURITYSERVICES', 'SEED', 'SEEDS', 'SENSORS', 'SERICULTURE', 'SERVICE', 'SERVICEPRIVATE', 'SERVICES', 'SERVICES(INDIA)', 'SERVICESINDIA', 'SERVICESLIMITED', 'SERVICESPRIVATE', 'SERVICESPVT', 'SHADES', 'SHARES', 'SHEETS', 'SHELTER', 'SHELTERS', 'SHIPBREAKING', 'SHIPYARD', 'SHOE', 'SHOES', 'SHOP', 'SHOPPE', 'SHOPPEE', 'SHOPPERS', 'SHOPPING', 'SIGNAGE', 'SIGNS', 'SILICA', 'SILICATE', 'SILICATES', 'SILICON', 'SILK', 'SILKS', 'SILVER', 'SILVERLINE', 'SIZING', 'SKILLS', 'SMITH', 'SNACKS', 'SOAP', 'SOAPS', 'SOFT-TECH', 'SOFTECH', 'SOFTSOLUTIONS', 'SOFTTECH', 'SOFTWARE', 'SOFTWARES', 'SOLUTION', 'SOLUTIONPRIVATE', 'SOLUTIONS', 'SOLUTIONS(INDIA)', 'SOLUTIONSPRIVATE', 'SOLVENT', 'SOLVENTS', 'SONAR', 'SOURCES', 'SOURCING', 'SPARE', 'SPARES', 'SPARKLE', 'SPICE', 'SPICES', 'SPINNERS', 'SPINNING', 'SPIRIT', 'SPIRITS', 'SPONGE', 'SPORTING', 'SPORTS', 'STAINLESS', 'STANDARD', 'STATIONERS', 'STATIONERY', 'STEEL', 'STEELS', 'STOCK', 'STOCKS', 'STONE', 'STONES', 'STORAGE', 'STORAGEPRIVATE', 'STORAGES', 'STORE', 'STORES', 'STRATEGIES', 'STRATEGY', 'STRUCTURALS', 'STRUCTURE', 'STRUCTURES', 'STUDIO', 'STUDIOS', 'STYLE', 'STYLES', 'STYLISH', 'SUGAR', 'SUGARCANE', 'SUGARS', 'SUITES', 'SUITINGS', 'SUPPLIER', 'SUPPLIERS', 'SUPPLIERSPRIVATE', 'SUPPLIES', 'SUPPLY', 'SURFACE', 'SURFACTANTS', 'SURGE', 'SURGICAL', 'SURGICALS', 'SURVEILLANCE', 'SURVEYORS', 'SURVEYS', 'SUSTAINABLE', 'SWEET', 'SWEETS', 'SYNAPSE', 'SYNERGIES', 'SYNERGY', 'SYNTHETIC', 'SYNTHETICS', 'SYSTECH', 'SYSTEM', 'SYSTEMATIC', 'SYSTEMPRIVATE', 'SYSTEMS', 'SYSTEMSPRIVATE', 'TALKIES', 'TANK', 'TANKS', 'TANNERIES', 'TANNERS', 'TANNERY', 'TANNING', 'TAPE', 'TAPES', 'TAXI', 'TAYAL', 'TEAK', 'TEC', 'TECH', 'TECHLABS', 'TECHNET', 'TECHNICAL', 'TECHNICS', 'TECHNIK', 'TECHNIQUES', 'TECHNO', 'TECHNOBUILD', 'TECHNOCAST', 'TECHNOCOM', 'TECHNOCRAFT', 'TECHNOCRAT', 'TECHNOCRATE', 'TECHNOCRATES', 'TECHNOCRATS', 'TECHNOFAB', 'TECHNOLABS', 'TECHNOLOGICAL', 'TECHNOLOGIES', 'TECHNOLOGIESPRIVATE', 'TECHNOLOGY', 'TECHNOLOGYPRIVATE', 'TECHNOPLAST', 'TECHNOSOFT', 'TECHNOSOLUTIONS', 'TECHNOSYS', 'TECHNOVISION', 'TECHSOFT', 'TECHSOLUTIONS', 'TELEMATICS', 'TELEMEDIA', 'TELENET', 'TELESERVICES', 'TELESOFT', 'TELESYSTEMS', 'TELETECH', 'TELEVENTURES', 'TELEVISION', 'TERMINALS', 'TEXFAB', 'TEXO', 'TEXTILE', 'TEXTILES', 'TEXTRADE', 'TEXTURISERS', 'TEXTURISING', 'THERAPEUTICS', 'THERAPY', 'THERMAL', 'THERMO', 'THINK', 'THREADS', 'TILE', 'TILES', 'TIMBER', 'TIMBERS', 'TISSUE', 'TISSUES', 'TOBACCO', 'TOLL', 'TOLLWAY', 'TOLLWAYS', 'TOOL', 'TOOLING', 'TOOLINGS', 'TOOLS', 'TOPAZ', 'TOPLINE', 'TOPSTAR', 'TORRENT', 'TOURISM', 'TOURIST', 'TOURS', 'TOWERS', 'TOWNSHIPS', 'TOYS', 'TRADE', 'TRADE-LINK', 'TRADECOM', 'TRADECOMM', 'TRADECON', 'TRADEFIN', 'TRADELINK', 'TRADELINKS', 'TRADEMART', 'TRADEPLACE', 'TRADER', 'TRADERS', 'TRADERSPRIVATE', 'TRADES', 'TRADEVIN', 'TRADEVISION', 'TRADEWELL', 'TRADEWING', 'TRADEWINGS', 'TRADEX', 'TRADEXIM', 'TRADING', 'TRADINGCOMPANY', 'TRADINGPRIVATE', 'TRADINGS', 'TRAFFIC', 'TRAINERS', 'TRAINING', 'TRAININGPRIVATE', 'TRANSCRIPTION', 'TRANSFORMER', 'TRANSFORMERS', 'TRANSIT', 'TRANSLOGISTICS', 'TRANSMISSION', 'TRANSMISSIONS', 'TRANSPORT', 'TRANSPORTATION', 'TRANSPORTERS', 'TRANSPORTS', 'TRANSTECH', 'TRANSWAYS', 'TRANSWORLD', 'TRAUMA', 'TRAVELLERS', 'TRAVELS', 'TRAVELSPRIVATE', 'TREADS', 'TRENDS', 'TRENDZ', 'TREXIM', 'TRIO', 'TUBE', 'TUBES', 'TYRE', 'TYRES', 'UMBRELLA', 'UNION', 'UNITECH', 'UTENSILS', 'UTILITIES', 'VACATIONS', 'VALUERS', 'VALVES', 'VAULT', 'VAULTS', 'VEHICLES', 'VENEER', 'VENEERS', 'VENTURA', 'VENTURESPRIVATE', 'VIDEO', 'WAREHOUSE', 'WAREHOUSING', 'WARES', 'WATCH', 'WATCHES', 'WATERPROOFING', 'WAVES', 'WEAR', 'WEARS', 'WEAVE', 'WEAVERS', 'WEAVES', 'WEAVING', 'WEB', 'WEBSOFT', 'WEBSOLUTIONS', 'WEBTECH', 'WEIGHING', 'WELD', 'WELDING', 'WELFARE', 'WHEAT', 'WHEEL', 'WHEELS', 'WHOLESALE', 'WINDOW', 'WINDOWS', 'WINE', 'WINERY', 'WINES', 'WING', 'WINGS', 'WIRES', 'WOOL', 'WOOLLEN', 'WOOLLENS', 'WOOLTEX', 'WORKSHOP', 'WORKSPRIVATE', 'X', 'YARD', 'YARN', 'YARNS', 'ZINC', 'ZIP', 'ZIPPERS', 'PVT']
            if any(ext in query for ext in firm_names):
                self.petfirm=1
            
            
            else:
                self.petfirm=0
            
        
        else:
            self.petfirm=0
            
            
       
        
    def check_respondent_firm(self):
        if(self.soup):
            title = self.null_check(self.soup.find_all('div', attrs={'class': 'doc_title'})[-1], 'text')
        
        else:
            title=None
        if(title):
            self.title=title
            small_title, date = title.split(' on ')
            self.petitioner, self.respondent = small_title.split(' vs ')
            respondent_upper=self.respondent.upper()
            query=respondent_upper.split()
            firm_names=['&', '&DEVELOPERS', '&INFRASTRUCTURE', '&SERVICES', '(BANGALORE)PRIVATE', '(EXPORTS)', '(I)PRIVATE', '(INDIA)LIMITED', '(INDIA)PRIVATE', '(INDIA)PVT', '(OPC)', '(OPC)PRIVATE', '(P)', '(PVT)', ')PRIVATE', 'ABRASIVE', 'ABRASIVES', 'ACADEMIC', 'ACADEMICS', 'ACADEMY', 'ACADEMYPRIVATE', 'ACCENT', 'ACCESSORIES', 'ACCESSORIESPRIVATE', 'ACCOUNTING', 'ACIDS', 'ACOUSTICS', 'ACQUA', 'ADDITIVES', 'ADHESIVE', 'ADHESIVES', 'ADVERTISERS', 'ADVERTISING', 'ADVERTISINGPRIVATE', 'ADVISERS', 'ADVISOR', 'ADVISORS', 'ADVISORSPRIVATE', 'ADVISORY', 'ADVISORYPRIVATE', 'ADVISORYSERVICES', 'AEROSPACE', 'AGENCIES', 'AGENCIESPRIVATE', 'AGENCY', 'AGENCYPRIVATE', 'AGENTS', 'AGGREGATES', 'AGRI-TECH', 'AGRICARE', 'AGRICHEM', 'AGRICO', 'AGRICULTURAL', 'AGRICULTURE', 'AGRIM', 'AGRITECH', 'AGRITRADE', 'AGRO', 'AGRO-FARMS', 'AGRO-TECH', 'AGROCHEM', 'AGROFARM', 'AGROFARMS', 'AGROFOOD', 'AGROFOODS', 'AGROHA', 'AGROPRODUCTS', 'AGROS', 'AGROTECH', 'AGROVET', 'AHEAD', 'AIDS', 'AIRCON', 'AIRCONDITIONING', 'AIRLINES', 'AIRPORT', 'AIRTECH', 'AIRWAYS', 'ALCHEMY', 'ALCO', 'ALKA', 'ALKALIES', 'ALLIANCE', 'ALLIED', 'ALLIEDINDUSTRIES', 'ALLIEDSERVICES', 'ALLOY', 'ALLOYS', 'ALLUMINIUM', 'ALPHA', 'ALPS', 'ALUMINIUM', 'AMALGAMATED', 'AMBAY', 'AMUSEMENTS', 'ANALYSIS', 'ANALYTICAL', 'ANALYTICS', 'ANCHOR', 'ANCILLARIES', 'ANCILLARY', 'ANDCONSTRUCTION', 'ANDCONSTRUCTIONS', 'ANDCONSULTANCY', 'ANDCONSULTANTS', 'ANDDEVELOPERS', 'ANDDEVELOPMENT', 'ANDENGINEERING', 'ANDFINANCE', 'ANDINFRASTRUCTURE', 'ANDINVESTMENT', 'ANDINVESTMENTS', 'ANDMANAGEMENT', 'ANDMARKETING', 'ANDRESEARCH', 'ANDSERVICES', 'ANDSOLUTIONS', 'ANDTECHNOLOGY', 'ANIMATION', 'ANIMATIONS', 'ANNEX', 'APEX', 'APPAREL', 'APPARELS', 'APPLIANCE', 'APPLIANCES', 'APPLIANCESPRIVATE', 'APPLICATION', 'APPLICATIONS', 'APPLIED', 'APPS', 'APT', 'AQUACULTURE', 'AQUATECH', 'AQUATIC', 'ARCADE', 'ARCHITECT', 'ARCHITECTS', 'ARCHITECTURAL', 'ARCHITECTURE', 'ARENA', 'ARMOUR', 'ARMS', 'AROMA', 'AROMATIC', 'AROMATICS', 'ARTS', 'ASBESTOS', 'ASSESSMENT', 'ASSESSORS', 'ASSET', 'ASSETS', 'ASSIST', 'ASSISTANCE', 'ASSOCIATE', 'ASSOCIATED', 'ASSOCIATES', 'ASSOCIATESPRIVATE', 'ASSOCIATION', 'ASSOCIATIONLIMITED', 'AUDIO', 'AUTOCOMP', 'AUTOMATIC', 'AUTOMATION', 'AUTOMATIONS', 'AUTOMOBILE', 'AUTOMOBILES', 'AUTOMOTIVE', 'AUTOMOTIVES', 'AUTOPARTS', 'AUTOS', 'AUTOTECH', 'AUTOWHEELS', 'AVENUE', 'AVENUES', 'AVIATION', 'AYURVED', 'AYURVEDA', 'AYURVEDIC', 'AYURVEDICS', 'AYUSH', 'AYUSHI', 'BAGS', 'BAKERS', 'BAKERY', 'BANK', 'BANQUET', 'BANQUETS', 'BARS', 'BATH', 'BATTERIES', 'BATTERY', 'BAZAAR', 'BAZAR', 'BEANS', 'BEARINGS', 'BEAUTY', 'BELL', 'BELLS', 'BELTING', 'BETA', 'BEVERAGE', 'BEVERAGES', 'BEVERAGESPRIVATE', 'BIKES', 'BINARY', 'BIO', 'BIO-TECH', 'BIOCARE', 'BIOCHEM', 'BIOENERGY', 'BIOFUELS', 'BIOLOGICALS', 'BIOMED', 'BIOMEDICAL', 'BIOPHARMA', 'BIOSCIENCE', 'BIOSCIENCES', 'BIOTEC', 'BIOTECH', 'BIOTECHNOLOGIES', 'BIOTECHNOLOGY', 'BIOTEK', 'BIRDS', 'BISCUIT', 'BISCUITS', 'BITS', 'BIZ', 'BLEACHING', 'BLOCK', 'BLOCKS', 'BLOOM', 'BLOOMING', 'BLOSSOM', 'BLOSSOMS', 'BLOW', 'BOARD', 'BOARDS', 'BODY', 'BOILERS', 'BONANZA', 'BOND', 'BONE', 'BOOK', 'BOOKS', 'BOTTLERS', 'BOTTLING', 'BOUTIQUE', 'BOX', 'BOXES', 'BRAIN', 'BRAINS', 'BRAKES', 'BRAND', 'BRANDING', 'BRANDS', 'BRASS', 'BRAVO', 'BREEDERS', 'BREEDING', 'BREEZE', 'BREWERIES', 'BRICK', 'BRICKS', 'BROAD', 'BROADBAND', 'BROADCAST', 'BROADCASTING', 'BROKER', 'BROKERAGE', 'BROKERS', 'BROKERSPRIVATE', 'BROS', 'BROS.', 'BROTHER', 'BROTHERS', 'BRUSH', 'BUILD-TECH', 'BUILDCOM', 'BUILDCON', 'BUILDCONS', 'BUILDER', 'BUILDERS', 'BUILDERSPRIVATE', 'BUILDESTATE', 'BUILDESTATES', 'BUILDHOME', 'BUILDHOMES', 'BUILDINFRA', 'BUILDING', 'BUILDINGS', 'BUILDMART', 'BUILDMAT', 'BUILDPRO', 'BUILDPROP', 'BUILDSQUARE', 'BUILDTECH', 'BUILDWEL', 'BUILDWELL', 'BUILTECH', 'BULK', 'BULLION', 'BULLIONS', 'BUREAU', 'BUSINESS', 'BUYING', 'BYTE', 'BYTES', 'CABLE', 'CABLES', 'CABS', 'CAFE', 'CALIBRATION', 'CAM', 'CAMELLIA', 'CAMP', 'CANCER', 'CANNING', 'CANS', 'CANVAS', 'CAP', 'CAPACITORS', 'CAPFIN', 'CAPITAL', 'CAPITALS', 'CAPRI', 'CAPRICORN', 'CAPS', 'CAPSTONE', 'CAPSULES', 'CAR', 'CARAVAN', 'CARBIDE', 'CARBIDES', 'CARBON', 'CARBONS', 'CARDIAC', 'CARDS', 'CARE', 'CAREER', 'CAREERS', 'CAREPRIVATE', 'CAREWELL', 'CARGO', 'CARNATION', 'CARNIVAL', 'CARPET', 'CARPETS', 'CARRIER', 'CARRIERS', 'CARS', 'CART', 'CARTONS', 'CASA', 'CASCADE', 'CASH', 'CASHEW', 'CASSETTES', 'CAST', 'CASTING', 'CASTINGS', 'CASTLE', 'CASTOR', 'CATALYST', 'CATERERS', 'CATERING', 'CELL', 'CELLULAR', 'CELLULOSE', 'CEMENT', 'CEMENTS', 'CENTER', 'CENTERPRIVATE', 'CENTERS', 'CENTRAL', 'CENTRE', 'CENTREPRIVATE', 'CENTRES', 'CENTURION', 'CENTURY', 'CERA', 'CERAMIC', 'CERAMICS', 'CEREALS', 'CERTIFICATION', 'CERTIFICATIONS', 'CHAIN', 'CHAINS', 'CHAMBER', 'CHAMBERS', 'CHANGER', 'CHANGERS', 'CHANNEL', 'CHANNELS', 'CHARAN', 'CHARTERED', 'CHARTERING', 'CHEF', 'CHEM', 'CHEMI', 'CHEMICAL', 'CHEMICALS', 'CHEMICALSPRIVATE', 'CHEMIE', 'CHEMTECH', 'CHIP', 'CHIPS', 'CHIT', 'CHITFUND', 'CHITFUNDS', 'CHITS', 'CHITTIES', 'CHITTY', 'CHROME', 'CINE', 'CINEMA', 'CINEMAS', 'CIRCUIT', 'CIRCUITS', 'CLEANERS', 'CLEANING', 'CLEANTECH', 'CLEARING', 'CLINIC', 'CLINICAL', 'CLINICS', 'CLOTHING', 'CLOTHINGS', 'CLOUD', 'CLUB', 'CLUBS', 'CO.', 'CO.LTD.', 'CO.PRIVATE', 'CO.PVT', 'CO.PVT.LTD.', 'COACH', 'COACHING', 'COASTAL', 'COAT', 'COATED', 'COATERS', 'COATING', 'COATINGS', 'COATS', 'CODE', 'COFFEE', 'COGENT', 'COIN', 'COIR', 'COKE', 'COLDSTORAGE', 'COLLECTION', 'COLLECTIONS', 'COLLECTIVE', 'COLLEGE', 'COLLIERIES', 'COLLIERY', 'COLONISERS', 'COLONIZER', 'COLONIZERS', 'COLOR', 'COLORS', 'COLOUR', 'COLOURS', 'COMFORTS', 'COMM', 'COMMERCE', 'COMMERCIAL', 'COMMERCIALS', 'COMMODEAL', 'COMMODITIES', 'COMMODITY', 'COMMOSALE', 'COMMOSALES', 'COMMOTRADE', 'COMMUNICATION', 'COMMUNICATIONPRIVATE', 'COMMUNICATIONS', 'COMMUNICATIONSPRIVATE', 'COMMUNITY', 'COMNET', 'COMP', 'COMPACT', 'COMPANY', 'COMPANYLIMITED', 'COMPANYPRIVATE', 'COMPANYPVT', 'COMPANYPVT.LTD.', 'COMPASS', 'COMPETENT', 'COMPLETE', 'COMPLEX', 'COMPLIANCE', 'COMPNAY', 'COMPONENT', 'COMPONENTS', 'COMPONENTSPRIVATE', 'COMPOSITE', 'COMPOSITES', 'COMPREHENSIVE', 'COMPRESSORS', 'COMPTECH', 'COMPU', 'COMPUSOFT', 'COMPUTECH', 'COMPUTER', 'COMPUTERS', 'COMPUTING', 'COMPUTRONICS', 'COMTECH', 'COMTRADE', 'CONBUILD', 'CONCAST', 'CONCEPT', 'CONCEPTS', 'CONCERN', 'CONCLAVE', 'CONCORD', 'CONCORDE', 'CONCRETE', 'CONCRETES', 'CONDITIONING', 'CONDUCTORS', 'CONES', 'CONFECTIONERS', 'CONFECTIONERY', 'CONFIDENT', 'CONGLOMERATE', 'CONNECT', 'CONNECTION', 'CONNECTIONS', 'CONNECTORS', 'CONSERVATION', 'CONSOLIDATED', 'CONSORTIUM', 'CONSTECH', 'CONSTRO', 'CONSTRUCTION', 'CONSTRUCTIONCOMPANY', 'CONSTRUCTIONPRIVATE', 'CONSTRUCTIONS', 'CONSTRUCTIONSPRIVATE', 'CONSTRUCTORS', 'CONSTRUWELL', 'CONSULT', 'CONSULTANCIES', 'CONSULTANCY', 'CONSULTANCYPRIVATE', 'CONSULTANCYSERVICES', 'CONSULTANT', 'CONSULTANTPRIVATE', 'CONSULTANTS', 'CONSULTANTSPRIVATE', 'CONSULTING', 'CONSULTINGPRIVATE', 'CONSULTINGSERVICES', 'CONSUMABLES', 'CONSUMER', 'CONSUMERS', 'CONTACT', 'CONTAINER', 'CONTAINERS', 'CONTECH', 'CONTEMPORARY', 'CONTINENTAL', 'CONTRACT', 'CONTRACTING', 'CONTRACTOR', 'CONTRACTORS', 'CONTRACTORSPRIVATE', 'CONTRACTS', 'CONTROLS', 'CONVENTION', 'CONVERGENCE', 'CONVERSION', 'CONVERTERS', 'CONVERTORS', 'CONVEYOR', 'CONVEYORS', 'COOLING', 'COPPER', 'COPRIVATE', 'COPVT', 'CORAL', 'CORE', 'CORN', 'CORNER', 'CORNERSTONE', 'COROMANDEL', 'CORONATION', 'CORP', 'CORPN', 'CORPN.', 'CORPORATE', 'CORPORATION', 'CORPORATIONLIMITED', 'CORPORATIONPRIVATE', 'CORRUGATED', 'CORRUGATORS', 'COSMETIC', 'COSMETICS', 'COSMIC', 'COSMO', 'COSMOPOLITAN', 'COSMOS', 'COST', 'COTEX', 'COTSPIN', 'COTSYN', 'COTTAGE', 'COTTAGES', 'COTTON', 'COTTONS', 'COUNCIL', 'COUNTRY', 'COUNTRYWIDE', 'COUNTY', 'COURIER', 'COURIERS', 'COUTURE', 'COZY', 'CRAFT', 'CRAFTS', 'CREAMS', 'CREATION', 'CREATIONS', 'CREATIVE', 'CREATIVES', 'CREATORS', 'CREDIT', 'CREDITS', 'CROP', 'CROPS', 'CROSS', 'CROWN', 'CRUSHER', 'CRUSHERS', 'CRUSHING', 'CRYSTAL', 'CRYSTALS', 'CUISINE', 'CUISINES', 'CULINARY', 'CULTIVATION', 'CULTIVATORS', 'CULTURAL', 'CULTURE', 'CURE', 'CURRENCY', 'CUSTOMER', 'CUTTING', 'CYBER', 'CYBERNETICS', 'CYBERTECH', 'CYCLE', 'CYCLES', 'CYLINDERS', 'DAILY', 'DAIRIES', 'DAIRY', 'DATA', 'DATABASE', 'DATACOM', 'DATAMATICS', 'DATASOFT', 'DEALCOM', 'DEALCOMM', 'DEALER', 'DEALERS', 'DEALMARK', 'DEALS', 'DEALTRADE', 'DEBT', 'DECO', 'DECOR', 'DECORATIVE', 'DECORATORS', 'DECORS', 'DEFENCE', 'DEGREE', 'DELIVERY', 'DELUX', 'DELUXE', 'DEPARTMENTAL', 'DEPOSIT', 'DEPOSITS', 'DEPOT', 'DERIVATIVES', 'DESIGN', 'DESIGNER', 'DESIGNERS', 'DESIGNING', 'DESIGNS', 'DETECTIVE', 'DETECTIVES', 'DETERGENT', 'DETERGENTS', 'DEVBUILD', 'DEVCON', 'DEVELOPER', 'DEVELOPERPRIVATE', 'DEVELOPERS', 'DEVELOPERSLIMITED', 'DEVELOPERSPRIVATE', 'DEVELOPMENT', 'DEVELOPMENTCOMPANY', 'DEVELOPMENTCORPORATION', 'DEVELOPMENTLIMITED', 'DEVELOPMENTPRIVATE', 'DEVELOPMENTS', 'DIAGNOSTIC', 'DIAGNOSTICS', 'DIAM', 'DIAMOND', 'DIAMONDS', 'DIARY', 'DIES', 'DIESEL', 'DIESELS', 'DIGI', 'DIGITAL', 'DIGITECH', 'DIGNITY', 'DIMENSION', 'DIMENSIONS', 'DISCOUNT', 'DISPLAY', 'DISTILLERIES', 'DISTILLERS', 'DISTILLERY', 'DISTRIBUTING', 'DISTRIBUTION', 'DISTRIBUTIONPRIVATE', 'DISTRIBUTIONS', 'DISTRIBUTOR', 'DISTRIBUTORS', 'DISTRIBUTORSPRIVATE', 'DOCTORS', 'DOMAIN', 'DOMESTIC', 'DOORS', 'DOT', 'DOTCOM', 'DOTS', 'DP', 'DRESSES', 'DRILLING', 'DRINKS', 'DRUG', 'DRUGS', 'DRUMS', 'DWELLERS', 'DWELLING', 'DWELLINGS', 'DYE', 'DYE-CHEM', 'DYECHEM', 'DYEING', 'DYERS', 'DYES', 'DYESTUFF', 'DYESTUFFS', 'E-COM', 'E-COMMERCE', 'E-LEARNING', 'E-SERVICES', 'E-SOLUTIONS', 'EARTHMOVERS', 'EATABLES', 'ECO', 'ECOM', 'ECOMMERCE', 'ECONOMIC', 'ECOTECH', 'EDIBLE', 'EDIBLES', 'EDUCARE', 'EDUCATION', 'EDUCATIONAL', 'EDUCATIONPRIVATE', 'EDUCATIONS', 'EDUCOM', 'EDUCON', 'EDUSOFT', 'EDUSOLUTIONS', 'EDUTAINMENT', 'EDUTECH', 'EDUVENTURES', 'EFFLUENT', 'ELECTRIC', 'ELECTRICAL', 'ELECTRICALS', 'ELECTRICALSPRIVATE', 'ELECTRICITY', 'ELECTRICS', 'ELECTRO', 'ELECTRODES', 'ELECTROMECH', 'ELECTRONIC', 'ELECTRONICS', 'ELECTRONICSPRIVATE', 'ELECTROTECH', 'ELEMENTS', 'ELEVATOR', 'ELEVATORS', 'EMBROIDERIES', 'EMBROIDERY', 'EMERGENCY', 'EMERGING', 'EMPLOYEES', 'EMPLOYMENT', 'EMPORIUM', 'EMPOWER', 'ENCLAVE', 'ENCLAVES', 'ENERGIES', 'ENERGY', 'ENERGYPRIVATE', 'ENERTECH', 'ENFIELD', 'ENGG', 'ENGG.', 'ENGICON', 'ENGICONS', 'ENGINE', 'ENGINEER', 'ENGINEERING', 'ENGINEERINGCOMPANY', 'ENGINEERINGPRIVATE', 'ENGINEERINGS', 'ENGINEERINGSERVICES', 'ENGINEERS', 'ENGINEERSPRIVATE', 'ENGINES', 'ENGITECH', 'ENTERPRISE', 'ENTERPRISES', 'ENTERPRISESPRIVATE', 'ENTERTAINERS', 'ENTERTAINMENT', 'ENTERTAINMENTPRIVATE', 'ENTERTAINMENTS', 'ENTREPRENEUR', 'ENTREPRENEURS', 'ENVIROTECH', 'ENVISION', 'EQUIPMENT', 'EQUIPMENTS', 'EQUIPMENTSPRIVATE', 'EQUITIES', 'EQUITY', 'ESCALATORS', 'ESCORTS', 'ESERVICES', 'ESOLUTIONS', 'ESQUIRE', 'ESSENTIALS', 'ESTATE', 'ESTATEPRIVATE', 'ESTATES', 'ESTATESPRIVATE', 'EX-SERVICEMEN', 'EXECUTIVE', 'EXHIBITION', 'EXHIBITIONS', 'EXHIBITORS', 'EXPEDITIONS', 'EXPERIENCE', 'EXPERT', 'EXPERTS', 'EXPLORATION', 'EXPLOSIVES', 'EXPO', 'EXPORT', 'EXPORTERS', 'EXPORTS', 'EXPORTSPRIVATE', 'EXPOSITIONS', 'EXPOTRADE', 'EXTRACTION', 'EXTRACTIONS', 'EXTRACTS', 'EXTRUSION', 'EXTRUSIONS', 'FABRIC', 'FABRICATION', 'FABRICATIONS', 'FABRICATORS', 'FABRICS', 'FABS', 'FABTECH', 'FABTEX', 'FACADE', 'FACILITATORS', 'FACILITIES', 'FACILITY', 'FACILITYMANAGEMENT', 'FACTOR', 'FACTORS', 'FACTORY', 'FACTORYPRIVATE', 'FARMER', 'FARMERS', 'FARMERSPRODUCER', 'FARMING', 'FARMS', 'FARMSPRIVATE', 'FASHION', 'FASHIONS', 'FASTENERS', 'FASTNERS', 'FASTSPEED', 'FATS', 'FEED', 'FEEDS', 'FERROUS', 'FERTILISERS', 'FERTILITY', 'FERTILIZER', 'FERTILIZERS', 'FERTILIZERSPRIVATE', 'FIBER', 'FIBERS', 'FIBRE', 'FIBRES', 'FIELD', 'FIELDS', 'FILAMENT', 'FILAMENTS', 'FILM', 'FILMS', 'FILTER', 'FILTERS', 'FILTRATION', 'FIN-LEASE', 'FINANCE', 'FINANCECOMPANY', 'FINANCELIMITED', 'FINANCEPRIVATE', 'FINANCERS', 'FINANCES', 'FINANCIAL', 'FINANCIALCONSULTANTS', 'FINANCIALS', 'FINANCIALSERVICES', 'FINANCIERS', 'FINANCING', 'FINCAP', 'FINCO', 'FINCOM', 'FINCON', 'FINCORP', 'FINE', 'FINECHEM', 'FINESSE', 'FININVEST', 'FINISHERS', 'FINISHING', 'FINLEASE', 'FINMART', 'FINSEC', 'FINSERV', 'FINSERVE', 'FINSTOCK', 'FINTECH', 'FINTRADE', 'FINVEST', 'FISH', 'FISHERIES', 'FISHING', 'FIT', 'FITNESS', 'FITTINGS', 'FIXTURES', 'FLAT', 'FLATS', 'FLAVOURS', 'FLEET', 'FLEX', 'FLEXI', 'FLEXIBLE', 'FLEXIPACK', 'FLIGHT', 'FLOOR', 'FLOORING', 'FLOORINGS', 'FLOORS', 'FLORA', 'FLORAL', 'FLORENCE', 'FLORICULTURE', 'FLORITECH', 'FLOUR', 'FLOW', 'FLOWER', 'FLOWERS', 'FLUID', 'FLYING', 'FOAM', 'FOAMS', 'FOILS', 'FOOD', 'FOODS', 'FOODSPRIVATE', 'FOODTECH', 'FOOT', 'FOOTBALL', 'FOOTCARE', 'FOOTWEAR', 'FOOTWEARS', 'FORESTRY', 'FOREX', 'FORGE', 'FORGING', 'FORGINGS', 'FORMING', 'FORMS', 'FORMULATION', 'FORMULATIONS', 'FORTUNE', 'FORUM', 'FORWARDERS', 'FOSTER', 'FOTO', 'FOUNDATION', 'FOUNDATIONS', 'FOUNDERS', 'FOUNDRIES', 'FOUNDRY', 'FRAGRANCE', 'FRAGRANCES', 'FRAME', 'FRAMES', 'FRANCHISE', 'FREIGHT', 'FROZEN', 'FRUIT', 'FRUITS', 'FUEL', 'FUELS', 'FUND', 'FUNDLIMITED', 'FUNDPRIVATE', 'FUNDS', 'FUNDSPRIVATE', 'FUNIDEA', 'FURNACE', 'FURNACES', 'FURNISHERS', 'FURNISHING', 'FURNISHINGS', 'FURNITURE', 'FURNITURES', 'FUTURES', 'GADGETS', 'GAIN', 'GALLERY', 'GAME', 'GAMES', 'GAMING', 'GARDENS', 'GARMENT', 'GARMENTS', 'GAS', 'GASES', 'GASKETS', 'GATEWAY', 'GAUGES', 'GEAR', 'GEARS', 'GEMS', 'GENERATORS', 'GENETIC', 'GENETICS', 'GENUINE', 'GEOTECH', 'GIFTS', 'GINNING', 'GLASS', 'GLASSES', 'GLAZE', 'GLAZING', 'GLOVES', 'GOODS', 'GRAINS', 'GRANITE', 'GRANITES', 'GRAPES', 'GRAPHICS', 'GRAPHITE', 'GREENTECH', 'GREY', 'GROUP', 'HABITATS', 'HANDICRAFT', 'HANDICRAFTS', 'HANDLING', 'HANDLOOM', 'HANDLOOMS', 'HARBOUR', 'HARDWARE', 'HARDWARES', 'HEALTHCARE', 'HEALTHTECH', 'HELPLINE', 'HERBALS', 'HERBS', 'HERITAGE', 'HI-TECH', 'HITECH', 'HOLDING', 'HOLDINGS', 'HOLDINGSPRIVATE', 'HOLIDAYS', 'HOSPITAL', 'HOSPITALITIES', 'HOSPITALITY', 'HOSPITALITYPRIVATE', 'HOSPITALPRIVATE', 'HOSPITALS', 'HOSPITALSPRIVATE', 'HOTELS', 'HOTELSPRIVATE', 'HOUSING', 'HYBRID', 'HYDEL', 'HYDRAULIC', 'HYDRAULICS', 'HYDRO', 'HYDROCARBONS', 'HYGIENE', 'HYGIENIC', 'I-TECH', 'I.T.', 'IMAGINE', 'IMAGING', 'IMPORT', 'IMPORTERS', 'IMPORTS', 'INDUSTRIAL', 'INDUSTRIALS', 'INDUSTRIES', 'INDUSTRIESLIMITED', 'INDUSTRIESPRIVATE', 'INDUSTRIESPVT', 'INDUSTRY', 'INFIN', 'INFINITE', 'INFINITI', 'INFINITY', 'INFO', 'INFO-TECH', 'INFOCARE', 'INFOCOM', 'INFOCOMM', 'INFOMATICS', 'INFOMEDIA', 'INFORMATIC', 'INFORMATICS', 'INFORMATION', 'INFORMATIONTECHNOLOGY', 'INFOSERVE', 'INFOSERVICES', 'INFOSOFT', 'INFOSOL', 'INFOSOLUTION', 'INFOSOLUTIONS', 'INFOSYS', 'INFOSYSTEM', 'INFOSYSTEMS', 'INFOTAINMENT', 'INFOTEC', 'INFOTECH', 'INFOTEK', 'INFOTEL', 'INFOVISION', 'INFOWARE', 'INFOWAY', 'INFOWAYS', 'INFOWEB', 'INFRA', 'INFRA-TECH', 'INFRAA', 'INFRABUILD', 'INFRABUILDCON', 'INFRABUILDERS', 'INFRACARE', 'INFRACITY', 'INFRACOM', 'INFRACON', 'INFRACONS', 'INFRACONSTRUCTION', 'INFRADEV', 'INFRADEVELOPER', 'INFRADEVELOPERS', 'INFRAENGINEERS', 'INFRAESTATE', 'INFRAESTATES', 'INFRAHEIGHT', 'INFRAPROJECT', 'INFRAPROJECTS', 'INFRAREALTY', 'INFRASERVICES', 'INFRASOLUTION', 'INFRASOLUTIONS', 'INFRASPACE', 'INFRASTRUCTURAL', 'INFRASTRUCTURE', 'INFRASTRUCTUREDEVELOPMENT', 'INFRASTRUCTURELIMITED', 'INFRASTRUCTUREPRIVATE', 'INFRASTRUCTURES', 'INFRASTRUCTURESPRIVATE', 'INFRATEC', 'INFRATECH', 'INFRATEL', 'INFRATRADE', 'INFRAVENTURE', 'INFRAVENTURES', 'INFRAVISION', 'INFRAZONE', 'INGENIOUS', 'INGOTS', 'INGREDIENTS', 'INITIATIVE', 'INITIATIVES', 'INK', 'INKS', 'INLAND', 'INN', 'INNOTECH', 'INNOVA', 'INNOVATION', 'INNOVATIONS', 'INNOVATIVE', 'INNOVATORS', 'INNOVISION', 'INORGANICS', 'INSECTICIDES', 'INSPECTION', 'INSTITUTE', 'INSTITUTEPRIVATE', 'INSTITUTES', 'INSTITUTION', 'INSTITUTIONS', 'INSTRUMENT', 'INSTRUMENTATION', 'INSTRUMENTS', 'INSULATION', 'INSULATIONS', 'INSULATORS', 'INSURANCE', 'INTECH', 'INTEGRA', 'INTEGRAL', 'INTEGRATED', 'INTEGRATION', 'INTEGRATORS', 'INTEGRITY', 'INTEL', 'INTELLECT', 'INTELLECTUAL', 'INTELLIGENCE', 'INTELLIGENT', 'INTER', 'INTERACTIVE', 'INTERCONTINENTAL', 'INTERFACE', 'INTERIORS', 'INTERLINK', 'INTERMEDIATES', 'INTERNATIONAL', 'INTERNATIONALPRIVATE', 'INTERNET', 'INTERTRADE', 'INTIMATE', 'INTIME', 'INTRA', 'INTRADE', 'INVENTIVE', 'INVEST', 'INVESTIGATION', 'INVESTMENT', 'INVESTMENTCOMPANY', 'INVESTMENTLIMITED', 'INVESTMENTPRIVATE', 'INVESTMENTS', 'INVESTMENTSLIMITED', 'INVESTMENTSPRIVATE', 'INVESTOR', 'INVESTORS', 'INVESTRADE', 'IRRIGATION', 'JEWEL', 'JEWELLERS', 'JEWELLERSPRIVATE', 'JEWELLERY', 'JEWELRY', 'JEWELS', 'KITCHEN', 'KITCHENS', 'KNIT', 'KNITS', 'KNITTERS', 'KNITTING', 'KNITWEAR', 'KNITWEARS', 'LAB', 'LABEL', 'LABELS', 'LABORATORIES', 'LABORATORIESPRIVATE', 'LABORATORY', 'LABOUR', 'LABS', 'LAMINATES', 'LAMINATIONS', 'LAMINATORS', 'LAMP', 'LAMPS', 'LANDDEVELOPERS', 'LATEX', 'LAUNDRY', 'LAW', 'LEAD', 'LEADING', 'LEARNING', 'LEASE', 'LEASING', 'LEASINGLIMITED', 'LEASINGPRIVATE', 'LEASINGS', 'LEATHER', 'LEATHERS', 'LEISURES', 'LIFESTYLE', 'LIFESTYLES', 'LIFT', 'LIFTS', 'LIGHT', 'LIGHTING', 'LIGHTINGS', 'LIGHTS', 'LIMITED', 'LIMITED(OPC)', 'LIMITED,', 'LIMITED.', 'LIMTED', 'LIMTIED', 'LINERS', 'LINES', 'LINK', 'LINKERS', 'LINKS', 'LIQUOR', 'LIQUORS', 'LOAN', 'LOANS', 'LOCKERS', 'LOCKS', 'LODGE', 'LOGIC', 'LOGICAL', 'LOGICS', 'LOGISTIC', 'LOGISTICS', 'LOGISTICSPRIVATE', 'LTD', 'LTD,', 'LTD.', 'LTD.,', 'LUBE', 'LUBES', 'LUBRICANTS', 'LUGGAGE', 'LUMBERS', 'MACHINE', 'MACHINERIES', 'MACHINERY', 'MACHINES', 'MACHINING', 'MACRO', 'MAGNET', 'MAGNETIC', 'MAGNETICS', 'MAINTENANCE', 'MAKERS', 'MALLS', 'MANAGED', 'MANAGEMENT', 'MANAGEMENTCONSULTANTS', 'MANAGEMENTPRIVATE', 'MANAGEMENTS', 'MANAGEMENTSERVICES', 'MANAGERS', 'MANAGMENT', 'MANUFACTURERS', 'MANUFACTURERSPRIVATE', 'MANUFACTURES', 'MANUFACTURING', 'MANUFACTURINGCOMPANY', 'MANUFACTURINGPRIVATE', 'MARBLE', 'MARBLES', 'MARKET', 'MARKETERS', 'MARKETING', 'MARKETINGPRIVATE', 'MARKETINGS', 'MARKETINGSERVICES', 'MARKETPLACE', 'MARKETS', 'MARKETTING', 'MATCH', 'MATCHES', 'MATE', 'MATERIAL', 'MATERIALS', 'MATRIMONIAL', 'MECH', 'MECHANICAL', 'MECHANICALS', 'MECHATRONICS', 'MED', 'MEDIA', 'MEDIATECH', 'MEDICAL', 'MEDICALS', 'MEDICAMENTS', 'MEDICINE', 'MEDICINES', 'MEDICO', 'MEDICOS', 'MEDITECH', 'MEDTECH', 'MEENA', 'MEENAKSHI', 'MEERA', 'MEERUT', 'MEET', 'MEGA', 'MEGACITY', 'MEGAMART', 'MEGH', 'MEGHA', 'MEGHALAYA', 'MEGHANA', 'MEGHDOOT', 'MEGHNA', 'MEHAK', 'MEHAR', 'MEHER', 'MEHRA', 'MEHTA', 'MEHUL', 'MELANGE', 'MELODY', 'MEMORIAL', 'MEMORY', 'MEN', 'MENON', 'MENTOR', 'MENTORS', 'MEP', 'MERA', 'MERCANTILE', 'MERCANTILES', 'MERCHANDISE', 'MERCHANDISERS', 'MERCHANDISING', 'MERCHANT', 'MERCHANTILE', 'MERCHANTS', 'MERCURY', 'MERIDIAN', 'MERIT', 'MERLIN', 'MERRY', 'MERU', 'MESH', 'MET', 'META', 'METACAST', 'METAL', 'METALIKS', 'METALLIC', 'METALLICS', 'METALLOYS', 'METALLURGICAL', 'METALLURGICALS', 'METALS', 'METERS', 'METRO', 'METROPOLIS', 'METROPOLITAN', 'MEWAR', 'MFG', 'MFG.', 'MG', 'MICA', 'MICRO', 'MICROFINANCE', 'MICRON', 'MICRONS', 'MICROSYSTEMS', 'MICROTECH', 'MIDAS', 'MIDDLE', 'MIDLAND', 'MIDWAY', 'MIGRATION', 'MILE', 'MILES', 'MILESTONE', 'MILK', 'MILL', 'MILLENIUM', 'MILLENNIUM', 'MILLERS', 'MILLING', 'MILLS', 'MILLSLIMITED', 'MILLSPRIVATE', 'MINAR', 'MINCHEM', 'MIND', 'MINDA', 'MINDS', 'MINE', 'MINECHEM', 'MINERAL', 'MINERALS', 'MINERALSPRIVATE', 'MINERVA', 'MINES', 'MINI', 'MINING', 'MINT', 'MIR', 'MIRA', 'MIRACLE', 'MIRAGE', 'MIRAJ', 'MIRROR', 'MISSION', 'MIST', 'MOBILE', 'MOBILES', 'MOBILITY', 'MOBITECH', 'MOD', 'MODE', 'MODEL', 'MODULAR', 'MOMENTUM', 'MONARCH', 'MONEY', 'MONITORING', 'MONSOON', 'MOON', 'MOONLIGHT', 'MOONLINK', 'MOONSHINE', 'MOONSTAR', 'MOSAIC', 'MOTEL', 'MOTELS', 'MOTION', 'MOTOR', 'MOTORS', 'MOULD', 'MOULDERS', 'MOULDING', 'MOULDINGS', 'MOULDS', 'MOVERS', 'MOVIE', 'MOVIES', 'MOVING', 'MULTI', 'MULTI-TRADE', 'MULTICON', 'MULTILEVEL', 'MULTILINK', 'MULTIMEDIA', 'MULTIPLE', 'MULTIPLEX', 'MULTIPROJECTS', 'MULTIPURPOSE', 'MULTISERVICES', 'MULTISOLUTIONS', 'MULTISPECIALITY', 'MULTITECH', 'MULTITRADE', 'MULTITRADING', 'MULTIVENTURES', 'NANOTECH', 'NATURALS', 'NEOTECH', 'NETCOM', 'NETLINK', 'NETMART', 'NETS', 'NETWORK', 'NETWORKING', 'NETWORKPRIVATE', 'NETWORKS', 'NETWORTH', 'NEURO', 'NURSING', 'NUTRACEUTICALS', 'NUTRI', 'NUTRIENTS', 'NUTRITION', 'NUTRITIONS', 'OIL', 'OILFIELD', 'OILS', 'OLIVE', 'ONCOLOGY', 'OPERATORS', 'OPTION', 'OPTIONS', 'ORCHID', 'ORCHIDS', 'ORGANISATION', 'ORGANISERS', 'ORGANIZATION', 'ORIENTAL', 'ORNATE', 'ORTHO', 'OUTSOURCE', 'OUTSOURCING', 'OVERSEAS', 'OWNERS', 'OXYGEN', 'OYSTER', 'OZONE', 'P.LTD.', 'PACK', 'PACKAGERS', 'PACKAGES', 'PACKAGING', 'PACKAGINGS', 'PACKERS', 'PACKING', 'PACKS', 'PAGES', 'PAINT', 'PAINTS', 'PALACE', 'PALMS', 'PANEL', 'PANELS', 'PAPER', 'PAPERS', 'PARADISE', 'PARAM', 'PARKPRIVATE', 'PARTNERS', 'PAYMENT', 'PAYMENTS', 'PEARL', 'PEARLS', 'PELICAN', 'PENS', 'PEOPLE', 'PEOPLES', 'PERFUMES', 'PERIPHERALS', 'PERSONNEL', 'PEST', 'PESTICIDES', 'PET', 'PETALS', 'PETRO', 'PETROCHEM', 'PETROCHEMICALS', 'PETROLEUM', 'PETROLEUMS', 'PHARMA', 'PHARMACEUTICAL', 'PHARMACEUTICALS', 'PHARMACEUTICALSPRIVATE', 'PHARMACHEM', 'PHARMACY', 'PHARMATECH', 'PHOENIX', 'PHOSPHATES', 'PHOTO', 'PHOTOGRAPHY', 'PHOTON', 'PICTURE', 'PICTURES', 'PIGMENTS', 'PILOT', 'PIPE', 'PIPES', 'PIPING', 'PLACEMENT', 'PLACEMENTS', 'PLANNER', 'PLANNERS', 'PLANNING', 'PLANTATION', 'PLANTATIONS', 'PLANTERS', 'PLANTS', 'PLASMA', 'PLASTECH', 'PLASTIC', 'PLASTICS', 'PLASTO', 'PLASTOCHEM', 'PLATFORM', 'PLATINUM', 'PLAY', 'PLAZA', 'PLAZZA', 'PLEASANT', 'PLEASURE', 'PLEXUS', 'PLYWOOD', 'PLYWOODS', 'PNEUMATICS', 'POINT', 'POLAR', 'POLARIS', 'POLE', 'POLES', 'POLESTAR', 'POLLUTION', 'POLY', 'POLYCHEM', 'POLYCLINIC', 'POLYCOATS', 'POLYESTER', 'POLYESTERS', 'POLYFAB', 'POLYFILMS', 'POLYGON', 'POLYMER', 'POLYMERS', 'POLYPACK', 'POLYPACKS', 'POLYPLAST', 'POLYSACKS', 'POLYTECH', 'POLYTEX', 'PONDICHERRY', 'PORT', 'PORTFOLIOS', 'PORTS', 'PORWAL', 'POST', 'POTENTIAL', 'POULTRIES', 'POULTRY', 'POWDER', 'POWDERS', 'POWERGEN', 'POWERS', 'POWERTECH', 'POWERTRONICS', 'PPRIVATE', 'PRECISIONS', 'PREMIUM', 'PRESS', 'PRESSINGS', 'PRESSURE', 'PRESTIGE', 'PRIAVTE', 'PRIMARY', 'PRIME', 'PRINTING', 'PRINTPACK', 'PRINTS', 'PRISM', 'PRIVATE', 'PRIVATELIMITED', 'PRIVATELIMITED.', 'PRIVATELTD', 'PROACTIVE', 'PROBUILD', 'PROCESS', 'PROCESSED', 'PROCESSES', 'PROCESSING', 'PROCESSINGPRIVATE', 'PROCESSOR', 'PROCESSORS', 'PROCON', 'PRODIGY', 'PRODUCE', 'PRODUCER', 'PRODUCERCOMPANY', 'PRODUCERS', 'PRODUCT', 'PRODUCTION', 'PRODUCTIONS', 'PRODUCTS', 'PRODUCTSLIMITED', 'PRODUCTSPRIVATE', 'PROFESSIONAL', 'PROFESSIONALS', 'PROFICIENT', 'PROFILE', 'PROFILES', 'PROFIT', 'PROGRESS', 'PROGRESSIVE', 'PROJECT', 'PROJECTS', 'PROJECTSLIMITED', 'PROJECTSPRIVATE', 'PROLIFIC', 'PROMINENT', 'PROMOTER', 'PROMOTERS', 'PROMOTERSPRIVATE', 'PROMOTION', 'PROMOTIONS', 'PROMOTORS', 'PROMPT', 'PROPBUILD', 'PROPERTIES', 'PROPERTIESPRIVATE', 'PROPERTY', 'PROPMART', 'PROPTECH', 'PROSPERITY', 'PROTECH', 'PROTECT', 'PROTECTIVE', 'PROTEINS', 'PROVIDENT', 'PROVIDER', 'PROVIDERS', 'PUBLICATION', 'PUBLICATIONS', 'PUBLICATIONSPRIVATE', 'PUBLISHERS', 'PUBLISHING', 'PULSES', 'PUMP', 'PUMPS', 'PVT.', 'PVT.LTD', 'PVT.LTD.', 'PVTLTD', 'PVTLTD.', 'QUALITY', 'QUARRIES', 'QUARRY', 'QUARTZ', 'QUEEN', 'RACING', 'RADIATORS', 'RAYON', 'RAYONS', 'REAL', 'REALATORS', 'REALBUILD', 'REALCON', 'REALESTATE', 'REALESTATES', 'REALHOME', 'REALINFRA', 'REALITIES', 'REALITY', 'REALMART', 'REALTECH', 'REALTERS', 'REALTIES', 'REALTIME', 'REALTOR', 'REALTORS', 'REALTORSPRIVATE', 'REALTY', 'RECKON', 'RECON', 'RECONSTRUCTION', 'RECORDING', 'RECORDS', 'RECREATION', 'RECREATIONS', 'RECRUITERS', 'RECRUITMENT', 'RECYCLERS', 'RECYCLING', 'REFINERIES', 'REFINERY', 'REFRACTORIES', 'REFRACTORY', 'REFRIGERATION', 'REMEDIES', 'RENAISSANCE', 'RENEWABLE', 'RENEWABLES', 'REROLLING', 'RESEARCH', 'RESEARCHCENTRE', 'RESEARCHFOUNDATION', 'RESEARCHINSTITUTE', 'RESEARCHPRIVATE', 'RESIDENCES', 'RESIDENCY', 'RESIDENTIAL', 'RESINS', 'RESONANCE', 'RESORT', 'RESORTS', 'RESORTSPRIVATE', 'RESOURCE', 'RESOURCES', 'RESOURCESPRIVATE', 'RESPONSE', 'RESTAURANT', 'RESTAURANTS', 'RETAIL', 'RETAILERS', 'RETAILING', 'RETAILS', 'RETREADING', 'RETREADS', 'RETREAT', 'RETREATS', 'ROADLINES', 'ROADS', 'ROADWAYS', 'ROBOTICS', 'RUBBER', 'RUBBERS', 'RUIA', 'SAPHIRE', 'SAPPHIRE', 'SAR', 'SAREE', 'SAREES', 'SATELLITE', 'SAVING', 'SAVINGS', 'SAW', 'SCAFFOLDING', 'SCAN', 'SCANS', 'SCHEME', 'SCHOOL', 'SCHOOLS', 'SCIENCE', 'SCIENCES', 'SCIENCESPRIVATE', 'SCIENTIFIC', 'SEAFOODS', 'SEAL', 'SEALS', 'SECURITIES', 'SECURITIESPRIVATE', 'SECURITY', 'SECURITYSERVICES', 'SEED', 'SEEDS', 'SENSORS', 'SERICULTURE', 'SERVICE', 'SERVICEPRIVATE', 'SERVICES', 'SERVICES(INDIA)', 'SERVICESINDIA', 'SERVICESLIMITED', 'SERVICESPRIVATE', 'SERVICESPVT', 'SHADES', 'SHARES', 'SHEETS', 'SHELTER', 'SHELTERS', 'SHIPBREAKING', 'SHIPYARD', 'SHOE', 'SHOES', 'SHOP', 'SHOPPE', 'SHOPPEE', 'SHOPPERS', 'SHOPPING', 'SIGNAGE', 'SIGNS', 'SILICA', 'SILICATE', 'SILICATES', 'SILICON', 'SILK', 'SILKS', 'SILVER', 'SILVERLINE', 'SIZING', 'SKILLS', 'SMITH', 'SNACKS', 'SOAP', 'SOAPS', 'SOFT-TECH', 'SOFTECH', 'SOFTSOLUTIONS', 'SOFTTECH', 'SOFTWARE', 'SOFTWARES', 'SOLUTION', 'SOLUTIONPRIVATE', 'SOLUTIONS', 'SOLUTIONS(INDIA)', 'SOLUTIONSPRIVATE', 'SOLVENT', 'SOLVENTS', 'SONAR', 'SOURCES', 'SOURCING', 'SPARE', 'SPARES', 'SPARKLE', 'SPICE', 'SPICES', 'SPINNERS', 'SPINNING', 'SPIRIT', 'SPIRITS', 'SPONGE', 'SPORTING', 'SPORTS', 'STAINLESS', 'STANDARD', 'STATIONERS', 'STATIONERY', 'STEEL', 'STEELS', 'STOCK', 'STOCKS', 'STONE', 'STONES', 'STORAGE', 'STORAGEPRIVATE', 'STORAGES', 'STORE', 'STORES', 'STRATEGIES', 'STRATEGY', 'STRUCTURALS', 'STRUCTURE', 'STRUCTURES', 'STUDIO', 'STUDIOS', 'STYLE', 'STYLES', 'STYLISH', 'SUGAR', 'SUGARCANE', 'SUGARS', 'SUITES', 'SUITINGS', 'SUPPLIER', 'SUPPLIERS', 'SUPPLIERSPRIVATE', 'SUPPLIES', 'SUPPLY', 'SURFACE', 'SURFACTANTS', 'SURGE', 'SURGICAL', 'SURGICALS', 'SURVEILLANCE', 'SURVEYORS', 'SURVEYS', 'SUSTAINABLE', 'SWEET', 'SWEETS', 'SYNAPSE', 'SYNERGIES', 'SYNERGY', 'SYNTHETIC', 'SYNTHETICS', 'SYSTECH', 'SYSTEM', 'SYSTEMATIC', 'SYSTEMPRIVATE', 'SYSTEMS', 'SYSTEMSPRIVATE', 'TALKIES', 'TANK', 'TANKS', 'TANNERIES', 'TANNERS', 'TANNERY', 'TANNING', 'TAPE', 'TAPES', 'TAXI', 'TAYAL', 'TEAK', 'TEC', 'TECH', 'TECHLABS', 'TECHNET', 'TECHNICAL', 'TECHNICS', 'TECHNIK', 'TECHNIQUES', 'TECHNO', 'TECHNOBUILD', 'TECHNOCAST', 'TECHNOCOM', 'TECHNOCRAFT', 'TECHNOCRAT', 'TECHNOCRATE', 'TECHNOCRATES', 'TECHNOCRATS', 'TECHNOFAB', 'TECHNOLABS', 'TECHNOLOGICAL', 'TECHNOLOGIES', 'TECHNOLOGIESPRIVATE', 'TECHNOLOGY', 'TECHNOLOGYPRIVATE', 'TECHNOPLAST', 'TECHNOSOFT', 'TECHNOSOLUTIONS', 'TECHNOSYS', 'TECHNOVISION', 'TECHSOFT', 'TECHSOLUTIONS', 'TELEMATICS', 'TELEMEDIA', 'TELENET', 'TELESERVICES', 'TELESOFT', 'TELESYSTEMS', 'TELETECH', 'TELEVENTURES', 'TELEVISION', 'TERMINALS', 'TEXFAB', 'TEXO', 'TEXTILE', 'TEXTILES', 'TEXTRADE', 'TEXTURISERS', 'TEXTURISING', 'THERAPEUTICS', 'THERAPY', 'THERMAL', 'THERMO', 'THINK', 'THREADS', 'TILE', 'TILES', 'TIMBER', 'TIMBERS', 'TISSUE', 'TISSUES', 'TOBACCO', 'TOLL', 'TOLLWAY', 'TOLLWAYS', 'TOOL', 'TOOLING', 'TOOLINGS', 'TOOLS', 'TOPAZ', 'TOPLINE', 'TOPSTAR', 'TORRENT', 'TOURISM', 'TOURIST', 'TOURS', 'TOWERS', 'TOWNSHIPS', 'TOYS', 'TRADE', 'TRADE-LINK', 'TRADECOM', 'TRADECOMM', 'TRADECON', 'TRADEFIN', 'TRADELINK', 'TRADELINKS', 'TRADEMART', 'TRADEPLACE', 'TRADER', 'TRADERS', 'TRADERSPRIVATE', 'TRADES', 'TRADEVIN', 'TRADEVISION', 'TRADEWELL', 'TRADEWING', 'TRADEWINGS', 'TRADEX', 'TRADEXIM', 'TRADING', 'TRADINGCOMPANY', 'TRADINGPRIVATE', 'TRADINGS', 'TRAFFIC', 'TRAINERS', 'TRAINING', 'TRAININGPRIVATE', 'TRANSCRIPTION', 'TRANSFORMER', 'TRANSFORMERS', 'TRANSIT', 'TRANSLOGISTICS', 'TRANSMISSION', 'TRANSMISSIONS', 'TRANSPORT', 'TRANSPORTATION', 'TRANSPORTERS', 'TRANSPORTS', 'TRANSTECH', 'TRANSWAYS', 'TRANSWORLD', 'TRAUMA', 'TRAVELLERS', 'TRAVELS', 'TRAVELSPRIVATE', 'TREADS', 'TRENDS', 'TRENDZ', 'TREXIM', 'TRIO', 'TUBE', 'TUBES', 'TYRE', 'TYRES', 'UMBRELLA', 'UNION', 'UNITECH', 'UTENSILS', 'UTILITIES', 'VACATIONS', 'VALUERS', 'VALVES', 'VAULT', 'VAULTS', 'VEHICLES', 'VENEER', 'VENEERS', 'VENTURA', 'VENTURESPRIVATE', 'VIDEO', 'WAREHOUSE', 'WAREHOUSING', 'WARES', 'WATCH', 'WATCHES', 'WATERPROOFING', 'WAVES', 'WEAR', 'WEARS', 'WEAVE', 'WEAVERS', 'WEAVES', 'WEAVING', 'WEB', 'WEBSOFT', 'WEBSOLUTIONS', 'WEBTECH', 'WEIGHING', 'WELD', 'WELDING', 'WELFARE', 'WHEAT', 'WHEEL', 'WHEELS', 'WHOLESALE', 'WINDOW', 'WINDOWS', 'WINE', 'WINERY', 'WINES', 'WING', 'WINGS', 'WIRES', 'WOOL', 'WOOLLEN', 'WOOLLENS', 'WOOLTEX', 'WORKSHOP', 'WORKSPRIVATE', 'X', 'YARD', 'YARN', 'YARNS', 'ZINC', 'ZIP', 'ZIPPERS', 'PVT']
            if any(ext in query for ext in firm_names):
                self.resfirm=1
            
            
            else:
                self.resfirm=0
                
            
        else:
            self.resfirm=0
            
            
    def check_petitioner_gov(self):
        title = self.null_check(self.soup.find_all('div', attrs={'class': 'doc_title'})[-1], 'text')
        self.title=title
        small_title, date = title.split(' on ')
        self.petitioner, self.respondent = small_title.split(' vs ')
        petitioner_upper=self.petitioner.upper()
        #firm_names=['&', '&DEVELOPERS', '&INFRASTRUCTURE', '&SERVICES', '(BANGALORE)PRIVATE', '(EXPORTS)', '(I)PRIVATE', '(INDIA)LIMITED', '(INDIA)PRIVATE', '(INDIA)PVT', '(OPC)', '(OPC)PRIVATE', '(P)', '(PVT)', ')PRIVATE', 'ABRASIVE', 'ABRASIVES', 'ACADEMIC', 'ACADEMICS', 'ACADEMY', 'ACADEMYPRIVATE', 'ACCENT', 'ACCESSORIES', 'ACCESSORIESPRIVATE', 'ACCOUNTING', 'ACIDS', 'ACOUSTICS', 'ACQUA', 'ADDITIVES', 'ADHESIVE', 'ADHESIVES', 'ADVERTISERS', 'ADVERTISING', 'ADVERTISINGPRIVATE', 'ADVISERS', 'ADVISOR', 'ADVISORS', 'ADVISORSPRIVATE', 'ADVISORY', 'ADVISORYPRIVATE', 'ADVISORYSERVICES', 'AEROSPACE', 'AGENCIES', 'AGENCIESPRIVATE', 'AGENCY', 'AGENCYPRIVATE', 'AGENTS', 'AGGREGATES', 'AGRI-TECH', 'AGRICARE', 'AGRICHEM', 'AGRICO', 'AGRICULTURAL', 'AGRICULTURE', 'AGRIM', 'AGRITECH', 'AGRITRADE', 'AGRO', 'AGRO-FARMS', 'AGRO-TECH', 'AGROCHEM', 'AGROFARM', 'AGROFARMS', 'AGROFOOD', 'AGROFOODS', 'AGROHA', 'AGROPRODUCTS', 'AGROS', 'AGROTECH', 'AGROVET', 'AHEAD', 'AIDS', 'AIRCON', 'AIRCONDITIONING', 'AIRLINES', 'AIRPORT', 'AIRTECH', 'AIRWAYS', 'ALCHEMY', 'ALCO', 'ALKA', 'ALKALIES', 'ALLIANCE', 'ALLIED', 'ALLIEDINDUSTRIES', 'ALLIEDSERVICES', 'ALLOY', 'ALLOYS', 'ALLUMINIUM', 'ALPHA', 'ALPS', 'ALUMINIUM', 'AMALGAMATED', 'AMBAY', 'AMUSEMENTS', 'ANALYSIS', 'ANALYTICAL', 'ANALYTICS', 'ANCHOR', 'ANCILLARIES', 'ANCILLARY', 'ANDCONSTRUCTION', 'ANDCONSTRUCTIONS', 'ANDCONSULTANCY', 'ANDCONSULTANTS', 'ANDDEVELOPERS', 'ANDDEVELOPMENT', 'ANDENGINEERING', 'ANDFINANCE', 'ANDINFRASTRUCTURE', 'ANDINVESTMENT', 'ANDINVESTMENTS', 'ANDMANAGEMENT', 'ANDMARKETING', 'ANDRESEARCH', 'ANDSERVICES', 'ANDSOLUTIONS', 'ANDTECHNOLOGY', 'ANIMATION', 'ANIMATIONS', 'ANNEX', 'APEX', 'APPAREL', 'APPARELS', 'APPLIANCE', 'APPLIANCES', 'APPLIANCESPRIVATE', 'APPLICATION', 'APPLICATIONS', 'APPLIED', 'APPS', 'APT', 'AQUACULTURE', 'AQUATECH', 'AQUATIC', 'ARCADE', 'ARCHITECT', 'ARCHITECTS', 'ARCHITECTURAL', 'ARCHITECTURE', 'ARENA', 'ARMOUR', 'ARMS', 'AROMA', 'AROMATIC', 'AROMATICS', 'ARTS', 'ASBESTOS', 'ASSESSMENT', 'ASSESSORS', 'ASSET', 'ASSETS', 'ASSIST', 'ASSISTANCE', 'ASSOCIATE', 'ASSOCIATED', 'ASSOCIATES', 'ASSOCIATESPRIVATE', 'ASSOCIATION', 'ASSOCIATIONLIMITED', 'AUDIO', 'AUTOCOMP', 'AUTOMATIC', 'AUTOMATION', 'AUTOMATIONS', 'AUTOMOBILE', 'AUTOMOBILES', 'AUTOMOTIVE', 'AUTOMOTIVES', 'AUTOPARTS', 'AUTOS', 'AUTOTECH', 'AUTOWHEELS', 'AVENUE', 'AVENUES', 'AVIATION', 'AYURVED', 'AYURVEDA', 'AYURVEDIC', 'AYURVEDICS', 'AYUSH', 'AYUSHI', 'BAGS', 'BAKERS', 'BAKERY', 'BANK', 'BANQUET', 'BANQUETS', 'BARS', 'BATH', 'BATTERIES', 'BATTERY', 'BAZAAR', 'BAZAR', 'BEANS', 'BEARINGS', 'BEAUTY', 'BELL', 'BELLS', 'BELTING', 'BETA', 'BEVERAGE', 'BEVERAGES', 'BEVERAGESPRIVATE', 'BIKES', 'BINARY', 'BIO', 'BIO-TECH', 'BIOCARE', 'BIOCHEM', 'BIOENERGY', 'BIOFUELS', 'BIOLOGICALS', 'BIOMED', 'BIOMEDICAL', 'BIOPHARMA', 'BIOSCIENCE', 'BIOSCIENCES', 'BIOTEC', 'BIOTECH', 'BIOTECHNOLOGIES', 'BIOTECHNOLOGY', 'BIOTEK', 'BIRDS', 'BISCUIT', 'BISCUITS', 'BITS', 'BIZ', 'BLEACHING', 'BLOCK', 'BLOCKS', 'BLOOM', 'BLOOMING', 'BLOSSOM', 'BLOSSOMS', 'BLOW', 'BOARD', 'BOARDS', 'BODY', 'BOILERS', 'BONANZA', 'BOND', 'BONE', 'BOOK', 'BOOKS', 'BOTTLERS', 'BOTTLING', 'BOUTIQUE', 'BOX', 'BOXES', 'BRAIN', 'BRAINS', 'BRAKES', 'BRAND', 'BRANDING', 'BRANDS', 'BRASS', 'BRAVO', 'BREEDERS', 'BREEDING', 'BREEZE', 'BREWERIES', 'BRICK', 'BRICKS', 'BROAD', 'BROADBAND', 'BROADCAST', 'BROADCASTING', 'BROKER', 'BROKERAGE', 'BROKERS', 'BROKERSPRIVATE', 'BROS', 'BROS.', 'BROTHER', 'BROTHERS', 'BRUSH', 'BUILD-TECH', 'BUILDCOM', 'BUILDCON', 'BUILDCONS', 'BUILDER', 'BUILDERS', 'BUILDERSPRIVATE', 'BUILDESTATE', 'BUILDESTATES', 'BUILDHOME', 'BUILDHOMES', 'BUILDINFRA', 'BUILDING', 'BUILDINGS', 'BUILDMART', 'BUILDMAT', 'BUILDPRO', 'BUILDPROP', 'BUILDSQUARE', 'BUILDTECH', 'BUILDWEL', 'BUILDWELL', 'BUILTECH', 'BULK', 'BULLION', 'BULLIONS', 'BUREAU', 'BUSINESS', 'BUYING', 'BYTE', 'BYTES', 'CABLE', 'CABLES', 'CABS', 'CAFE', 'CALIBRATION', 'CAM', 'CAMELLIA', 'CAMP', 'CANCER', 'CANNING', 'CANS', 'CANVAS', 'CAP', 'CAPACITORS', 'CAPFIN', 'CAPITAL', 'CAPITALS', 'CAPRI', 'CAPRICORN', 'CAPS', 'CAPSTONE', 'CAPSULES', 'CAR', 'CARAVAN', 'CARBIDE', 'CARBIDES', 'CARBON', 'CARBONS', 'CARDIAC', 'CARDS', 'CARE', 'CAREER', 'CAREERS', 'CAREPRIVATE', 'CAREWELL', 'CARGO', 'CARNATION', 'CARNIVAL', 'CARPET', 'CARPETS', 'CARRIER', 'CARRIERS', 'CARS', 'CART', 'CARTONS', 'CASA', 'CASCADE', 'CASH', 'CASHEW', 'CASSETTES', 'CAST', 'CASTING', 'CASTINGS', 'CASTLE', 'CASTOR', 'CATALYST', 'CATERERS', 'CATERING', 'CELL', 'CELLULAR', 'CELLULOSE', 'CEMENT', 'CEMENTS', 'CENTER', 'CENTERPRIVATE', 'CENTERS', 'CENTRAL', 'CENTRE', 'CENTREPRIVATE', 'CENTRES', 'CENTURION', 'CENTURY', 'CERA', 'CERAMIC', 'CERAMICS', 'CEREALS', 'CERTIFICATION', 'CERTIFICATIONS', 'CHAIN', 'CHAINS', 'CHAMBER', 'CHAMBERS', 'CHANGER', 'CHANGERS', 'CHANNEL', 'CHANNELS', 'CHARAN', 'CHARTERED', 'CHARTERING', 'CHEF', 'CHEM', 'CHEMI', 'CHEMICAL', 'CHEMICALS', 'CHEMICALSPRIVATE', 'CHEMIE', 'CHEMTECH', 'CHIP', 'CHIPS', 'CHIT', 'CHITFUND', 'CHITFUNDS', 'CHITS', 'CHITTIES', 'CHITTY', 'CHROME', 'CINE', 'CINEMA', 'CINEMAS', 'CIRCUIT', 'CIRCUITS', 'CLEANERS', 'CLEANING', 'CLEANTECH', 'CLEARING', 'CLINIC', 'CLINICAL', 'CLINICS', 'CLOTHING', 'CLOTHINGS', 'CLOUD', 'CLUB', 'CLUBS', 'CO.', 'CO.LTD.', 'CO.PRIVATE', 'CO.PVT', 'CO.PVT.LTD.', 'COACH', 'COACHING', 'COASTAL', 'COAT', 'COATED', 'COATERS', 'COATING', 'COATINGS', 'COATS', 'CODE', 'COFFEE', 'COGENT', 'COIN', 'COIR', 'COKE', 'COLDSTORAGE', 'COLLECTION', 'COLLECTIONS', 'COLLECTIVE', 'COLLEGE', 'COLLIERIES', 'COLLIERY', 'COLONISERS', 'COLONIZER', 'COLONIZERS', 'COLOR', 'COLORS', 'COLOUR', 'COLOURS', 'COMFORTS', 'COMM', 'COMMERCE', 'COMMERCIAL', 'COMMERCIALS', 'COMMODEAL', 'COMMODITIES', 'COMMODITY', 'COMMOSALE', 'COMMOSALES', 'COMMOTRADE', 'COMMUNICATION', 'COMMUNICATIONPRIVATE', 'COMMUNICATIONS', 'COMMUNICATIONSPRIVATE', 'COMMUNITY', 'COMNET', 'COMP', 'COMPACT', 'COMPANY', 'COMPANYLIMITED', 'COMPANYPRIVATE', 'COMPANYPVT', 'COMPANYPVT.LTD.', 'COMPASS', 'COMPETENT', 'COMPLETE', 'COMPLEX', 'COMPLIANCE', 'COMPNAY', 'COMPONENT', 'COMPONENTS', 'COMPONENTSPRIVATE', 'COMPOSITE', 'COMPOSITES', 'COMPREHENSIVE', 'COMPRESSORS', 'COMPTECH', 'COMPU', 'COMPUSOFT', 'COMPUTECH', 'COMPUTER', 'COMPUTERS', 'COMPUTING', 'COMPUTRONICS', 'COMTECH', 'COMTRADE', 'CONBUILD', 'CONCAST', 'CONCEPT', 'CONCEPTS', 'CONCERN', 'CONCLAVE', 'CONCORD', 'CONCORDE', 'CONCRETE', 'CONCRETES', 'CONDITIONING', 'CONDUCTORS', 'CONES', 'CONFECTIONERS', 'CONFECTIONERY', 'CONFIDENT', 'CONGLOMERATE', 'CONNECT', 'CONNECTION', 'CONNECTIONS', 'CONNECTORS', 'CONSERVATION', 'CONSOLIDATED', 'CONSORTIUM', 'CONSTECH', 'CONSTRO', 'CONSTRUCTION', 'CONSTRUCTIONCOMPANY', 'CONSTRUCTIONPRIVATE', 'CONSTRUCTIONS', 'CONSTRUCTIONSPRIVATE', 'CONSTRUCTORS', 'CONSTRUWELL', 'CONSULT', 'CONSULTANCIES', 'CONSULTANCY', 'CONSULTANCYPRIVATE', 'CONSULTANCYSERVICES', 'CONSULTANT', 'CONSULTANTPRIVATE', 'CONSULTANTS', 'CONSULTANTSPRIVATE', 'CONSULTING', 'CONSULTINGPRIVATE', 'CONSULTINGSERVICES', 'CONSUMABLES', 'CONSUMER', 'CONSUMERS', 'CONTACT', 'CONTAINER', 'CONTAINERS', 'CONTECH', 'CONTEMPORARY', 'CONTINENTAL', 'CONTRACT', 'CONTRACTING', 'CONTRACTOR', 'CONTRACTORS', 'CONTRACTORSPRIVATE', 'CONTRACTS', 'CONTROLS', 'CONVENTION', 'CONVERGENCE', 'CONVERSION', 'CONVERTERS', 'CONVERTORS', 'CONVEYOR', 'CONVEYORS', 'COOLING', 'COPPER', 'COPRIVATE', 'COPVT', 'CORAL', 'CORE', 'CORN', 'CORNER', 'CORNERSTONE', 'COROMANDEL', 'CORONATION', 'CORP', 'CORPN', 'CORPN.', 'CORPORATE', 'CORPORATION', 'CORPORATIONLIMITED', 'CORPORATIONPRIVATE', 'CORRUGATED', 'CORRUGATORS', 'COSMETIC', 'COSMETICS', 'COSMIC', 'COSMO', 'COSMOPOLITAN', 'COSMOS', 'COST', 'COTEX', 'COTSPIN', 'COTSYN', 'COTTAGE', 'COTTAGES', 'COTTON', 'COTTONS', 'COUNCIL', 'COUNTRY', 'COUNTRYWIDE', 'COUNTY', 'COURIER', 'COURIERS', 'COUTURE', 'COZY', 'CRAFT', 'CRAFTS', 'CREAMS', 'CREATION', 'CREATIONS', 'CREATIVE', 'CREATIVES', 'CREATORS', 'CREDIT', 'CREDITS', 'CROP', 'CROPS', 'CROSS', 'CROWN', 'CRUSHER', 'CRUSHERS', 'CRUSHING', 'CRYSTAL', 'CRYSTALS', 'CUISINE', 'CUISINES', 'CULINARY', 'CULTIVATION', 'CULTIVATORS', 'CULTURAL', 'CULTURE', 'CURE', 'CURRENCY', 'CUSTOMER', 'CUTTING', 'CYBER', 'CYBERNETICS', 'CYBERTECH', 'CYCLE', 'CYCLES', 'CYLINDERS', 'DAILY', 'DAIRIES', 'DAIRY', 'DATA', 'DATABASE', 'DATACOM', 'DATAMATICS', 'DATASOFT', 'DEALCOM', 'DEALCOMM', 'DEALER', 'DEALERS', 'DEALMARK', 'DEALS', 'DEALTRADE', 'DEBT', 'DECO', 'DECOR', 'DECORATIVE', 'DECORATORS', 'DECORS', 'DEFENCE', 'DEGREE', 'DELIVERY', 'DELUX', 'DELUXE', 'DEPARTMENTAL', 'DEPOSIT', 'DEPOSITS', 'DEPOT', 'DERIVATIVES', 'DESIGN', 'DESIGNER', 'DESIGNERS', 'DESIGNING', 'DESIGNS', 'DETECTIVE', 'DETECTIVES', 'DETERGENT', 'DETERGENTS', 'DEVBUILD', 'DEVCON', 'DEVELOPER', 'DEVELOPERPRIVATE', 'DEVELOPERS', 'DEVELOPERSLIMITED', 'DEVELOPERSPRIVATE', 'DEVELOPMENT', 'DEVELOPMENTCOMPANY', 'DEVELOPMENTCORPORATION', 'DEVELOPMENTLIMITED', 'DEVELOPMENTPRIVATE', 'DEVELOPMENTS', 'DIAGNOSTIC', 'DIAGNOSTICS', 'DIAM', 'DIAMOND', 'DIAMONDS', 'DIARY', 'DIES', 'DIESEL', 'DIESELS', 'DIGI', 'DIGITAL', 'DIGITECH', 'DIGNITY', 'DIMENSION', 'DIMENSIONS', 'DISCOUNT', 'DISPLAY', 'DISTILLERIES', 'DISTILLERS', 'DISTILLERY', 'DISTRIBUTING', 'DISTRIBUTION', 'DISTRIBUTIONPRIVATE', 'DISTRIBUTIONS', 'DISTRIBUTOR', 'DISTRIBUTORS', 'DISTRIBUTORSPRIVATE', 'DOCTORS', 'DOMAIN', 'DOMESTIC', 'DOORS', 'DOT', 'DOTCOM', 'DOTS', 'DP', 'DRESSES', 'DRILLING', 'DRINKS', 'DRUG', 'DRUGS', 'DRUMS', 'DWELLERS', 'DWELLING', 'DWELLINGS', 'DYE', 'DYE-CHEM', 'DYECHEM', 'DYEING', 'DYERS', 'DYES', 'DYESTUFF', 'DYESTUFFS', 'E-COM', 'E-COMMERCE', 'E-LEARNING', 'E-SERVICES', 'E-SOLUTIONS', 'EARTHMOVERS', 'EATABLES', 'ECO', 'ECOM', 'ECOMMERCE', 'ECONOMIC', 'ECOTECH', 'EDIBLE', 'EDIBLES', 'EDUCARE', 'EDUCATION', 'EDUCATIONAL', 'EDUCATIONPRIVATE', 'EDUCATIONS', 'EDUCOM', 'EDUCON', 'EDUSOFT', 'EDUSOLUTIONS', 'EDUTAINMENT', 'EDUTECH', 'EDUVENTURES', 'EFFLUENT', 'ELECTRIC', 'ELECTRICAL', 'ELECTRICALS', 'ELECTRICALSPRIVATE', 'ELECTRICITY', 'ELECTRICS', 'ELECTRO', 'ELECTRODES', 'ELECTROMECH', 'ELECTRONIC', 'ELECTRONICS', 'ELECTRONICSPRIVATE', 'ELECTROTECH', 'ELEMENTS', 'ELEVATOR', 'ELEVATORS', 'EMBROIDERIES', 'EMBROIDERY', 'EMERGENCY', 'EMERGING', 'EMPLOYEES', 'EMPLOYMENT', 'EMPORIUM', 'EMPOWER', 'ENCLAVE', 'ENCLAVES', 'ENERGIES', 'ENERGY', 'ENERGYPRIVATE', 'ENERTECH', 'ENFIELD', 'ENGG', 'ENGG.', 'ENGICON', 'ENGICONS', 'ENGINE', 'ENGINEER', 'ENGINEERING', 'ENGINEERINGCOMPANY', 'ENGINEERINGPRIVATE', 'ENGINEERINGS', 'ENGINEERINGSERVICES', 'ENGINEERS', 'ENGINEERSPRIVATE', 'ENGINES', 'ENGITECH', 'ENTERPRISE', 'ENTERPRISES', 'ENTERPRISESPRIVATE', 'ENTERTAINERS', 'ENTERTAINMENT', 'ENTERTAINMENTPRIVATE', 'ENTERTAINMENTS', 'ENTREPRENEUR', 'ENTREPRENEURS', 'ENVIROTECH', 'ENVISION', 'EQUIPMENT', 'EQUIPMENTS', 'EQUIPMENTSPRIVATE', 'EQUITIES', 'EQUITY', 'ESCALATORS', 'ESCORTS', 'ESERVICES', 'ESOLUTIONS', 'ESQUIRE', 'ESSENTIALS', 'ESTATE', 'ESTATEPRIVATE', 'ESTATES', 'ESTATESPRIVATE', 'EX-SERVICEMEN', 'EXECUTIVE', 'EXHIBITION', 'EXHIBITIONS', 'EXHIBITORS', 'EXPEDITIONS', 'EXPERIENCE', 'EXPERT', 'EXPERTS', 'EXPLORATION', 'EXPLOSIVES', 'EXPO', 'EXPORT', 'EXPORTERS', 'EXPORTS', 'EXPORTSPRIVATE', 'EXPOSITIONS', 'EXPOTRADE', 'EXTRACTION', 'EXTRACTIONS', 'EXTRACTS', 'EXTRUSION', 'EXTRUSIONS', 'FABRIC', 'FABRICATION', 'FABRICATIONS', 'FABRICATORS', 'FABRICS', 'FABS', 'FABTECH', 'FABTEX', 'FACADE', 'FACILITATORS', 'FACILITIES', 'FACILITY', 'FACILITYMANAGEMENT', 'FACTOR', 'FACTORS', 'FACTORY', 'FACTORYPRIVATE', 'FARMER', 'FARMERS', 'FARMERSPRODUCER', 'FARMING', 'FARMS', 'FARMSPRIVATE', 'FASHION', 'FASHIONS', 'FASTENERS', 'FASTNERS', 'FASTSPEED', 'FATS', 'FEED', 'FEEDS', 'FERROUS', 'FERTILISERS', 'FERTILITY', 'FERTILIZER', 'FERTILIZERS', 'FERTILIZERSPRIVATE', 'FIBER', 'FIBERS', 'FIBRE', 'FIBRES', 'FIELD', 'FIELDS', 'FILAMENT', 'FILAMENTS', 'FILM', 'FILMS', 'FILTER', 'FILTERS', 'FILTRATION', 'FIN-LEASE', 'FINANCE', 'FINANCECOMPANY', 'FINANCELIMITED', 'FINANCEPRIVATE', 'FINANCERS', 'FINANCES', 'FINANCIAL', 'FINANCIALCONSULTANTS', 'FINANCIALS', 'FINANCIALSERVICES', 'FINANCIERS', 'FINANCING', 'FINCAP', 'FINCO', 'FINCOM', 'FINCON', 'FINCORP', 'FINE', 'FINECHEM', 'FINESSE', 'FININVEST', 'FINISHERS', 'FINISHING', 'FINLEASE', 'FINMART', 'FINSEC', 'FINSERV', 'FINSERVE', 'FINSTOCK', 'FINTECH', 'FINTRADE', 'FINVEST', 'FISH', 'FISHERIES', 'FISHING', 'FIT', 'FITNESS', 'FITTINGS', 'FIXTURES', 'FLAT', 'FLATS', 'FLAVOURS', 'FLEET', 'FLEX', 'FLEXI', 'FLEXIBLE', 'FLEXIPACK', 'FLIGHT', 'FLOOR', 'FLOORING', 'FLOORINGS', 'FLOORS', 'FLORA', 'FLORAL', 'FLORENCE', 'FLORICULTURE', 'FLORITECH', 'FLOUR', 'FLOW', 'FLOWER', 'FLOWERS', 'FLUID', 'FLYING', 'FOAM', 'FOAMS', 'FOILS', 'FOOD', 'FOODS', 'FOODSPRIVATE', 'FOODTECH', 'FOOT', 'FOOTBALL', 'FOOTCARE', 'FOOTWEAR', 'FOOTWEARS', 'FORESTRY', 'FOREX', 'FORGE', 'FORGING', 'FORGINGS', 'FORMING', 'FORMS', 'FORMULATION', 'FORMULATIONS', 'FORTUNE', 'FORUM', 'FORWARDERS', 'FOSTER', 'FOTO', 'FOUNDATION', 'FOUNDATIONS', 'FOUNDERS', 'FOUNDRIES', 'FOUNDRY', 'FRAGRANCE', 'FRAGRANCES', 'FRAME', 'FRAMES', 'FRANCHISE', 'FREIGHT', 'FROZEN', 'FRUIT', 'FRUITS', 'FUEL', 'FUELS', 'FUND', 'FUNDLIMITED', 'FUNDPRIVATE', 'FUNDS', 'FUNDSPRIVATE', 'FUNIDEA', 'FURNACE', 'FURNACES', 'FURNISHERS', 'FURNISHING', 'FURNISHINGS', 'FURNITURE', 'FURNITURES', 'FUTURES', 'GADGETS', 'GAIN', 'GALLERY', 'GAME', 'GAMES', 'GAMING', 'GARDENS', 'GARMENT', 'GARMENTS', 'GAS', 'GASES', 'GASKETS', 'GATEWAY', 'GAUGES', 'GEAR', 'GEARS', 'GEMS', 'GENERATORS', 'GENETIC', 'GENETICS', 'GENUINE', 'GEOTECH', 'GIFTS', 'GINNING', 'GLASS', 'GLASSES', 'GLAZE', 'GLAZING', 'GLOVES', 'GOODS', 'GRAINS', 'GRANITE', 'GRANITES', 'GRAPES', 'GRAPHICS', 'GRAPHITE', 'GREENTECH', 'GREY', 'GROUP', 'HABITATS', 'HANDICRAFT', 'HANDICRAFTS', 'HANDLING', 'HANDLOOM', 'HANDLOOMS', 'HARBOUR', 'HARDWARE', 'HARDWARES', 'HEALTHCARE', 'HEALTHTECH', 'HELPLINE', 'HERBALS', 'HERBS', 'HERITAGE', 'HI-TECH', 'HITECH', 'HOLDING', 'HOLDINGS', 'HOLDINGSPRIVATE', 'HOLIDAYS', 'HOSPITAL', 'HOSPITALITIES', 'HOSPITALITY', 'HOSPITALITYPRIVATE', 'HOSPITALPRIVATE', 'HOSPITALS', 'HOSPITALSPRIVATE', 'HOTELS', 'HOTELSPRIVATE', 'HOUSING', 'HYBRID', 'HYDEL', 'HYDRAULIC', 'HYDRAULICS', 'HYDRO', 'HYDROCARBONS', 'HYGIENE', 'HYGIENIC', 'I-TECH', 'I.T.', 'IMAGINE', 'IMAGING', 'IMPORT', 'IMPORTERS', 'IMPORTS', 'INDUSTRIAL', 'INDUSTRIALS', 'INDUSTRIES', 'INDUSTRIESLIMITED', 'INDUSTRIESPRIVATE', 'INDUSTRIESPVT', 'INDUSTRY', 'INFIN', 'INFINITE', 'INFINITI', 'INFINITY', 'INFO', 'INFO-TECH', 'INFOCARE', 'INFOCOM', 'INFOCOMM', 'INFOMATICS', 'INFOMEDIA', 'INFORMATIC', 'INFORMATICS', 'INFORMATION', 'INFORMATIONTECHNOLOGY', 'INFOSERVE', 'INFOSERVICES', 'INFOSOFT', 'INFOSOL', 'INFOSOLUTION', 'INFOSOLUTIONS', 'INFOSYS', 'INFOSYSTEM', 'INFOSYSTEMS', 'INFOTAINMENT', 'INFOTEC', 'INFOTECH', 'INFOTEK', 'INFOTEL', 'INFOVISION', 'INFOWARE', 'INFOWAY', 'INFOWAYS', 'INFOWEB', 'INFRA', 'INFRA-TECH', 'INFRAA', 'INFRABUILD', 'INFRABUILDCON', 'INFRABUILDERS', 'INFRACARE', 'INFRACITY', 'INFRACOM', 'INFRACON', 'INFRACONS', 'INFRACONSTRUCTION', 'INFRADEV', 'INFRADEVELOPER', 'INFRADEVELOPERS', 'INFRAENGINEERS', 'INFRAESTATE', 'INFRAESTATES', 'INFRAHEIGHT', 'INFRAPROJECT', 'INFRAPROJECTS', 'INFRAREALTY', 'INFRASERVICES', 'INFRASOLUTION', 'INFRASOLUTIONS', 'INFRASPACE', 'INFRASTRUCTURAL', 'INFRASTRUCTURE', 'INFRASTRUCTUREDEVELOPMENT', 'INFRASTRUCTURELIMITED', 'INFRASTRUCTUREPRIVATE', 'INFRASTRUCTURES', 'INFRASTRUCTURESPRIVATE', 'INFRATEC', 'INFRATECH', 'INFRATEL', 'INFRATRADE', 'INFRAVENTURE', 'INFRAVENTURES', 'INFRAVISION', 'INFRAZONE', 'INGENIOUS', 'INGOTS', 'INGREDIENTS', 'INITIATIVE', 'INITIATIVES', 'INK', 'INKS', 'INLAND', 'INN', 'INNOTECH', 'INNOVA', 'INNOVATION', 'INNOVATIONS', 'INNOVATIVE', 'INNOVATORS', 'INNOVISION', 'INORGANICS', 'INSECTICIDES', 'INSPECTION', 'INSTITUTE', 'INSTITUTEPRIVATE', 'INSTITUTES', 'INSTITUTION', 'INSTITUTIONS', 'INSTRUMENT', 'INSTRUMENTATION', 'INSTRUMENTS', 'INSULATION', 'INSULATIONS', 'INSULATORS', 'INSURANCE', 'INTECH', 'INTEGRA', 'INTEGRAL', 'INTEGRATED', 'INTEGRATION', 'INTEGRATORS', 'INTEGRITY', 'INTEL', 'INTELLECT', 'INTELLECTUAL', 'INTELLIGENCE', 'INTELLIGENT', 'INTER', 'INTERACTIVE', 'INTERCONTINENTAL', 'INTERFACE', 'INTERIORS', 'INTERLINK', 'INTERMEDIATES', 'INTERNATIONAL', 'INTERNATIONALPRIVATE', 'INTERNET', 'INTERTRADE', 'INTIMATE', 'INTIME', 'INTRA', 'INTRADE', 'INVENTIVE', 'INVEST', 'INVESTIGATION', 'INVESTMENT', 'INVESTMENTCOMPANY', 'INVESTMENTLIMITED', 'INVESTMENTPRIVATE', 'INVESTMENTS', 'INVESTMENTSLIMITED', 'INVESTMENTSPRIVATE', 'INVESTOR', 'INVESTORS', 'INVESTRADE', 'IRRIGATION', 'JEWEL', 'JEWELLERS', 'JEWELLERSPRIVATE', 'JEWELLERY', 'JEWELRY', 'JEWELS', 'KITCHEN', 'KITCHENS', 'KNIT', 'KNITS', 'KNITTERS', 'KNITTING', 'KNITWEAR', 'KNITWEARS', 'LAB', 'LABEL', 'LABELS', 'LABORATORIES', 'LABORATORIESPRIVATE', 'LABORATORY', 'LABOUR', 'LABS', 'LAMINATES', 'LAMINATIONS', 'LAMINATORS', 'LAMP', 'LAMPS', 'LANDDEVELOPERS', 'LATEX', 'LAUNDRY', 'LAW', 'LEAD', 'LEADING', 'LEARNING', 'LEASE', 'LEASING', 'LEASINGLIMITED', 'LEASINGPRIVATE', 'LEASINGS', 'LEATHER', 'LEATHERS', 'LEISURES', 'LIFESTYLE', 'LIFESTYLES', 'LIFT', 'LIFTS', 'LIGHT', 'LIGHTING', 'LIGHTINGS', 'LIGHTS', 'LIMITED', 'LIMITED(OPC)', 'LIMITED,', 'LIMITED.', 'LIMTED', 'LIMTIED', 'LINERS', 'LINES', 'LINK', 'LINKERS', 'LINKS', 'LIQUOR', 'LIQUORS', 'LOAN', 'LOANS', 'LOCKERS', 'LOCKS', 'LODGE', 'LOGIC', 'LOGICAL', 'LOGICS', 'LOGISTIC', 'LOGISTICS', 'LOGISTICSPRIVATE', 'LTD', 'LTD,', 'LTD.', 'LTD.,', 'LUBE', 'LUBES', 'LUBRICANTS', 'LUGGAGE', 'LUMBERS', 'MACHINE', 'MACHINERIES', 'MACHINERY', 'MACHINES', 'MACHINING', 'MACRO', 'MAGNET', 'MAGNETIC', 'MAGNETICS', 'MAINTENANCE', 'MAKERS', 'MALLS', 'MANAGED', 'MANAGEMENT', 'MANAGEMENTCONSULTANTS', 'MANAGEMENTPRIVATE', 'MANAGEMENTS', 'MANAGEMENTSERVICES', 'MANAGERS', 'MANAGMENT', 'MANUFACTURERS', 'MANUFACTURERSPRIVATE', 'MANUFACTURES', 'MANUFACTURING', 'MANUFACTURINGCOMPANY', 'MANUFACTURINGPRIVATE', 'MARBLE', 'MARBLES', 'MARKET', 'MARKETERS', 'MARKETING', 'MARKETINGPRIVATE', 'MARKETINGS', 'MARKETINGSERVICES', 'MARKETPLACE', 'MARKETS', 'MARKETTING', 'MATCH', 'MATCHES', 'MATE', 'MATERIAL', 'MATERIALS', 'MATRIMONIAL', 'MECH', 'MECHANICAL', 'MECHANICALS', 'MECHATRONICS', 'MED', 'MEDIA', 'MEDIATECH', 'MEDICAL', 'MEDICALS', 'MEDICAMENTS', 'MEDICINE', 'MEDICINES', 'MEDICO', 'MEDICOS', 'MEDITECH', 'MEDTECH', 'MEENA', 'MEENAKSHI', 'MEERA', 'MEERUT', 'MEET', 'MEGA', 'MEGACITY', 'MEGAMART', 'MEGH', 'MEGHA', 'MEGHALAYA', 'MEGHANA', 'MEGHDOOT', 'MEGHNA', 'MEHAK', 'MEHAR', 'MEHER', 'MEHRA', 'MEHTA', 'MEHUL', 'MELANGE', 'MELODY', 'MEMORIAL', 'MEMORY', 'MEN', 'MENON', 'MENTOR', 'MENTORS', 'MEP', 'MERA', 'MERCANTILE', 'MERCANTILES', 'MERCHANDISE', 'MERCHANDISERS', 'MERCHANDISING', 'MERCHANT', 'MERCHANTILE', 'MERCHANTS', 'MERCURY', 'MERIDIAN', 'MERIT', 'MERLIN', 'MERRY', 'MERU', 'MESH', 'MET', 'META', 'METACAST', 'METAL', 'METALIKS', 'METALLIC', 'METALLICS', 'METALLOYS', 'METALLURGICAL', 'METALLURGICALS', 'METALS', 'METERS', 'METRO', 'METROPOLIS', 'METROPOLITAN', 'MEWAR', 'MFG', 'MFG.', 'MG', 'MICA', 'MICRO', 'MICROFINANCE', 'MICRON', 'MICRONS', 'MICROSYSTEMS', 'MICROTECH', 'MIDAS', 'MIDDLE', 'MIDLAND', 'MIDWAY', 'MIGRATION', 'MILE', 'MILES', 'MILESTONE', 'MILK', 'MILL', 'MILLENIUM', 'MILLENNIUM', 'MILLERS', 'MILLING', 'MILLS', 'MILLSLIMITED', 'MILLSPRIVATE', 'MINAR', 'MINCHEM', 'MIND', 'MINDA', 'MINDS', 'MINE', 'MINECHEM', 'MINERAL', 'MINERALS', 'MINERALSPRIVATE', 'MINERVA', 'MINES', 'MINI', 'MINING', 'MINT', 'MIR', 'MIRA', 'MIRACLE', 'MIRAGE', 'MIRAJ', 'MIRROR', 'MISSION', 'MIST', 'MOBILE', 'MOBILES', 'MOBILITY', 'MOBITECH', 'MOD', 'MODE', 'MODEL', 'MODULAR', 'MOMENTUM', 'MONARCH', 'MONEY', 'MONITORING', 'MONSOON', 'MOON', 'MOONLIGHT', 'MOONLINK', 'MOONSHINE', 'MOONSTAR', 'MOSAIC', 'MOTEL', 'MOTELS', 'MOTION', 'MOTOR', 'MOTORS', 'MOULD', 'MOULDERS', 'MOULDING', 'MOULDINGS', 'MOULDS', 'MOVERS', 'MOVIE', 'MOVIES', 'MOVING', 'MULTI', 'MULTI-TRADE', 'MULTICON', 'MULTILEVEL', 'MULTILINK', 'MULTIMEDIA', 'MULTIPLE', 'MULTIPLEX', 'MULTIPROJECTS', 'MULTIPURPOSE', 'MULTISERVICES', 'MULTISOLUTIONS', 'MULTISPECIALITY', 'MULTITECH', 'MULTITRADE', 'MULTITRADING', 'MULTIVENTURES', 'NANOTECH', 'NATURALS', 'NEOTECH', 'NETCOM', 'NETLINK', 'NETMART', 'NETS', 'NETWORK', 'NETWORKING', 'NETWORKPRIVATE', 'NETWORKS', 'NETWORTH', 'NEURO', 'NURSING', 'NUTRACEUTICALS', 'NUTRI', 'NUTRIENTS', 'NUTRITION', 'NUTRITIONS', 'OIL', 'OILFIELD', 'OILS', 'OLIVE', 'ONCOLOGY', 'OPERATORS', 'OPTION', 'OPTIONS', 'ORCHID', 'ORCHIDS', 'ORGANISATION', 'ORGANISERS', 'ORGANIZATION', 'ORIENTAL', 'ORNATE', 'ORTHO', 'OUTSOURCE', 'OUTSOURCING', 'OVERSEAS', 'OWNERS', 'OXYGEN', 'OYSTER', 'OZONE', 'P.LTD.', 'PACK', 'PACKAGERS', 'PACKAGES', 'PACKAGING', 'PACKAGINGS', 'PACKERS', 'PACKING', 'PACKS', 'PAGES', 'PAINT', 'PAINTS', 'PALACE', 'PALMS', 'PANEL', 'PANELS', 'PAPER', 'PAPERS', 'PARADISE', 'PARAM', 'PARKPRIVATE', 'PARTNERS', 'PAYMENT', 'PAYMENTS', 'PEARL', 'PEARLS', 'PELICAN', 'PENS', 'PEOPLE', 'PEOPLES', 'PERFUMES', 'PERIPHERALS', 'PERSONNEL', 'PEST', 'PESTICIDES', 'PET', 'PETALS', 'PETRO', 'PETROCHEM', 'PETROCHEMICALS', 'PETROLEUM', 'PETROLEUMS', 'PHARMA', 'PHARMACEUTICAL', 'PHARMACEUTICALS', 'PHARMACEUTICALSPRIVATE', 'PHARMACHEM', 'PHARMACY', 'PHARMATECH', 'PHOENIX', 'PHOSPHATES', 'PHOTO', 'PHOTOGRAPHY', 'PHOTON', 'PICTURE', 'PICTURES', 'PIGMENTS', 'PILOT', 'PIPE', 'PIPES', 'PIPING', 'PLACEMENT', 'PLACEMENTS', 'PLANNER', 'PLANNERS', 'PLANNING', 'PLANTATION', 'PLANTATIONS', 'PLANTERS', 'PLANTS', 'PLASMA', 'PLASTECH', 'PLASTIC', 'PLASTICS', 'PLASTO', 'PLASTOCHEM', 'PLATFORM', 'PLATINUM', 'PLAY', 'PLAZA', 'PLAZZA', 'PLEASANT', 'PLEASURE', 'PLEXUS', 'PLYWOOD', 'PLYWOODS', 'PNEUMATICS', 'POINT', 'POLAR', 'POLARIS', 'POLE', 'POLES', 'POLESTAR', 'POLLUTION', 'POLY', 'POLYCHEM', 'POLYCLINIC', 'POLYCOATS', 'POLYESTER', 'POLYESTERS', 'POLYFAB', 'POLYFILMS', 'POLYGON', 'POLYMER', 'POLYMERS', 'POLYPACK', 'POLYPACKS', 'POLYPLAST', 'POLYSACKS', 'POLYTECH', 'POLYTEX', 'PONDICHERRY', 'PORT', 'PORTFOLIOS', 'PORTS', 'PORWAL', 'POST', 'POTENTIAL', 'POULTRIES', 'POULTRY', 'POWDER', 'POWDERS', 'POWERGEN', 'POWERS', 'POWERTECH', 'POWERTRONICS', 'PPRIVATE', 'PRECISIONS', 'PREMIUM', 'PRESS', 'PRESSINGS', 'PRESSURE', 'PRESTIGE', 'PRIAVTE', 'PRIMARY', 'PRIME', 'PRINTING', 'PRINTPACK', 'PRINTS', 'PRISM', 'PRIVATE', 'PRIVATELIMITED', 'PRIVATELIMITED.', 'PRIVATELTD', 'PROACTIVE', 'PROBUILD', 'PROCESS', 'PROCESSED', 'PROCESSES', 'PROCESSING', 'PROCESSINGPRIVATE', 'PROCESSOR', 'PROCESSORS', 'PROCON', 'PRODIGY', 'PRODUCE', 'PRODUCER', 'PRODUCERCOMPANY', 'PRODUCERS', 'PRODUCT', 'PRODUCTION', 'PRODUCTIONS', 'PRODUCTS', 'PRODUCTSLIMITED', 'PRODUCTSPRIVATE', 'PROFESSIONAL', 'PROFESSIONALS', 'PROFICIENT', 'PROFILE', 'PROFILES', 'PROFIT', 'PROGRESS', 'PROGRESSIVE', 'PROJECT', 'PROJECTS', 'PROJECTSLIMITED', 'PROJECTSPRIVATE', 'PROLIFIC', 'PROMINENT', 'PROMOTER', 'PROMOTERS', 'PROMOTERSPRIVATE', 'PROMOTION', 'PROMOTIONS', 'PROMOTORS', 'PROMPT', 'PROPBUILD', 'PROPERTIES', 'PROPERTIESPRIVATE', 'PROPERTY', 'PROPMART', 'PROPTECH', 'PROSPERITY', 'PROTECH', 'PROTECT', 'PROTECTIVE', 'PROTEINS', 'PROVIDENT', 'PROVIDER', 'PROVIDERS', 'PUBLICATION', 'PUBLICATIONS', 'PUBLICATIONSPRIVATE', 'PUBLISHERS', 'PUBLISHING', 'PULSES', 'PUMP', 'PUMPS', 'PVT.', 'PVT.LTD', 'PVT.LTD.', 'PVTLTD', 'PVTLTD.', 'QUALITY', 'QUARRIES', 'QUARRY', 'QUARTZ', 'QUEEN', 'RACING', 'RADIATORS', 'RAYON', 'RAYONS', 'REAL', 'REALATORS', 'REALBUILD', 'REALCON', 'REALESTATE', 'REALESTATES', 'REALHOME', 'REALINFRA', 'REALITIES', 'REALITY', 'REALMART', 'REALTECH', 'REALTERS', 'REALTIES', 'REALTIME', 'REALTOR', 'REALTORS', 'REALTORSPRIVATE', 'REALTY', 'RECKON', 'RECON', 'RECONSTRUCTION', 'RECORDING', 'RECORDS', 'RECREATION', 'RECREATIONS', 'RECRUITERS', 'RECRUITMENT', 'RECYCLERS', 'RECYCLING', 'REFINERIES', 'REFINERY', 'REFRACTORIES', 'REFRACTORY', 'REFRIGERATION', 'REMEDIES', 'RENAISSANCE', 'RENEWABLE', 'RENEWABLES', 'REROLLING', 'RESEARCH', 'RESEARCHCENTRE', 'RESEARCHFOUNDATION', 'RESEARCHINSTITUTE', 'RESEARCHPRIVATE', 'RESIDENCES', 'RESIDENCY', 'RESIDENTIAL', 'RESINS', 'RESONANCE', 'RESORT', 'RESORTS', 'RESORTSPRIVATE', 'RESOURCE', 'RESOURCES', 'RESOURCESPRIVATE', 'RESPONSE', 'RESTAURANT', 'RESTAURANTS', 'RETAIL', 'RETAILERS', 'RETAILING', 'RETAILS', 'RETREADING', 'RETREADS', 'RETREAT', 'RETREATS', 'ROADLINES', 'ROADS', 'ROADWAYS', 'ROBOTICS', 'RUBBER', 'RUBBERS', 'RUIA', 'SAPHIRE', 'SAPPHIRE', 'SAR', 'SAREE', 'SAREES', 'SATELLITE', 'SAVING', 'SAVINGS', 'SAW', 'SCAFFOLDING', 'SCAN', 'SCANS', 'SCHEME', 'SCHOOL', 'SCHOOLS', 'SCIENCE', 'SCIENCES', 'SCIENCESPRIVATE', 'SCIENTIFIC', 'SEAFOODS', 'SEAL', 'SEALS', 'SECURITIES', 'SECURITIESPRIVATE', 'SECURITY', 'SECURITYSERVICES', 'SEED', 'SEEDS', 'SENSORS', 'SERICULTURE', 'SERVICE', 'SERVICEPRIVATE', 'SERVICES', 'SERVICES(INDIA)', 'SERVICESINDIA', 'SERVICESLIMITED', 'SERVICESPRIVATE', 'SERVICESPVT', 'SHADES', 'SHARES', 'SHEETS', 'SHELTER', 'SHELTERS', 'SHIPBREAKING', 'SHIPYARD', 'SHOE', 'SHOES', 'SHOP', 'SHOPPE', 'SHOPPEE', 'SHOPPERS', 'SHOPPING', 'SIGNAGE', 'SIGNS', 'SILICA', 'SILICATE', 'SILICATES', 'SILICON', 'SILK', 'SILKS', 'SILVER', 'SILVERLINE', 'SIZING', 'SKILLS', 'SMITH', 'SNACKS', 'SOAP', 'SOAPS', 'SOFT-TECH', 'SOFTECH', 'SOFTSOLUTIONS', 'SOFTTECH', 'SOFTWARE', 'SOFTWARES', 'SOLUTION', 'SOLUTIONPRIVATE', 'SOLUTIONS', 'SOLUTIONS(INDIA)', 'SOLUTIONSPRIVATE', 'SOLVENT', 'SOLVENTS', 'SONAR', 'SOURCES', 'SOURCING', 'SPARE', 'SPARES', 'SPARKLE', 'SPICE', 'SPICES', 'SPINNERS', 'SPINNING', 'SPIRIT', 'SPIRITS', 'SPONGE', 'SPORTING', 'SPORTS', 'STAINLESS', 'STANDARD', 'STATIONERS', 'STATIONERY', 'STEEL', 'STEELS', 'STOCK', 'STOCKS', 'STONE', 'STONES', 'STORAGE', 'STORAGEPRIVATE', 'STORAGES', 'STORE', 'STORES', 'STRATEGIES', 'STRATEGY', 'STRUCTURALS', 'STRUCTURE', 'STRUCTURES', 'STUDIO', 'STUDIOS', 'STYLE', 'STYLES', 'STYLISH', 'SUGAR', 'SUGARCANE', 'SUGARS', 'SUITES', 'SUITINGS', 'SUPPLIER', 'SUPPLIERS', 'SUPPLIERSPRIVATE', 'SUPPLIES', 'SUPPLY', 'SURFACE', 'SURFACTANTS', 'SURGE', 'SURGICAL', 'SURGICALS', 'SURVEILLANCE', 'SURVEYORS', 'SURVEYS', 'SUSTAINABLE', 'SWEET', 'SWEETS', 'SYNAPSE', 'SYNERGIES', 'SYNERGY', 'SYNTHETIC', 'SYNTHETICS', 'SYSTECH', 'SYSTEM', 'SYSTEMATIC', 'SYSTEMPRIVATE', 'SYSTEMS', 'SYSTEMSPRIVATE', 'TALKIES', 'TANK', 'TANKS', 'TANNERIES', 'TANNERS', 'TANNERY', 'TANNING', 'TAPE', 'TAPES', 'TAXI', 'TAYAL', 'TEAK', 'TEC', 'TECH', 'TECHLABS', 'TECHNET', 'TECHNICAL', 'TECHNICS', 'TECHNIK', 'TECHNIQUES', 'TECHNO', 'TECHNOBUILD', 'TECHNOCAST', 'TECHNOCOM', 'TECHNOCRAFT', 'TECHNOCRAT', 'TECHNOCRATE', 'TECHNOCRATES', 'TECHNOCRATS', 'TECHNOFAB', 'TECHNOLABS', 'TECHNOLOGICAL', 'TECHNOLOGIES', 'TECHNOLOGIESPRIVATE', 'TECHNOLOGY', 'TECHNOLOGYPRIVATE', 'TECHNOPLAST', 'TECHNOSOFT', 'TECHNOSOLUTIONS', 'TECHNOSYS', 'TECHNOVISION', 'TECHSOFT', 'TECHSOLUTIONS', 'TELEMATICS', 'TELEMEDIA', 'TELENET', 'TELESERVICES', 'TELESOFT', 'TELESYSTEMS', 'TELETECH', 'TELEVENTURES', 'TELEVISION', 'TERMINALS', 'TEXFAB', 'TEXO', 'TEXTILE', 'TEXTILES', 'TEXTRADE', 'TEXTURISERS', 'TEXTURISING', 'THERAPEUTICS', 'THERAPY', 'THERMAL', 'THERMO', 'THINK', 'THREADS', 'TILE', 'TILES', 'TIMBER', 'TIMBERS', 'TISSUE', 'TISSUES', 'TOBACCO', 'TOLL', 'TOLLWAY', 'TOLLWAYS', 'TOOL', 'TOOLING', 'TOOLINGS', 'TOOLS', 'TOPAZ', 'TOPLINE', 'TOPSTAR', 'TORRENT', 'TOURISM', 'TOURIST', 'TOURS', 'TOWERS', 'TOWNSHIPS', 'TOYS', 'TRADE', 'TRADE-LINK', 'TRADECOM', 'TRADECOMM', 'TRADECON', 'TRADEFIN', 'TRADELINK', 'TRADELINKS', 'TRADEMART', 'TRADEPLACE', 'TRADER', 'TRADERS', 'TRADERSPRIVATE', 'TRADES', 'TRADEVIN', 'TRADEVISION', 'TRADEWELL', 'TRADEWING', 'TRADEWINGS', 'TRADEX', 'TRADEXIM', 'TRADING', 'TRADINGCOMPANY', 'TRADINGPRIVATE', 'TRADINGS', 'TRAFFIC', 'TRAINERS', 'TRAINING', 'TRAININGPRIVATE', 'TRANSCRIPTION', 'TRANSFORMER', 'TRANSFORMERS', 'TRANSIT', 'TRANSLOGISTICS', 'TRANSMISSION', 'TRANSMISSIONS', 'TRANSPORT', 'TRANSPORTATION', 'TRANSPORTERS', 'TRANSPORTS', 'TRANSTECH', 'TRANSWAYS', 'TRANSWORLD', 'TRAUMA', 'TRAVELLERS', 'TRAVELS', 'TRAVELSPRIVATE', 'TREADS', 'TRENDS', 'TRENDZ', 'TREXIM', 'TRIO', 'TUBE', 'TUBES', 'TYRE', 'TYRES', 'UMBRELLA', 'UNION', 'UNITECH', 'UTENSILS', 'UTILITIES', 'VACATIONS', 'VALUERS', 'VALVES', 'VAULT', 'VAULTS', 'VEHICLES', 'VENEER', 'VENEERS', 'VENTURA', 'VENTURESPRIVATE', 'VIDEO', 'WAREHOUSE', 'WAREHOUSING', 'WARES', 'WATCH', 'WATCHES', 'WATERPROOFING', 'WAVES', 'WEAR', 'WEARS', 'WEAVE', 'WEAVERS', 'WEAVES', 'WEAVING', 'WEB', 'WEBSOFT', 'WEBSOLUTIONS', 'WEBTECH', 'WEIGHING', 'WELD', 'WELDING', 'WELFARE', 'WHEAT', 'WHEEL', 'WHEELS', 'WHOLESALE', 'WINDOW', 'WINDOWS', 'WINE', 'WINERY', 'WINES', 'WING', 'WINGS', 'WIRES', 'WOOL', 'WOOLLEN', 'WOOLLENS', 'WOOLTEX', 'WORKSHOP', 'WORKSPRIVATE', 'X', 'YARD', 'YARN', 'YARNS', 'ZINC', 'ZIP', 'ZIPPERS', 'PVT']
        govt_names=['STATE','GOVERNMENT']
        if any(ext in petitioner_upper for ext in govt_names):
            self.petgov=1
            
        else:
            
            self.petgov=0
            
            
    def check_respondent_gov(self):
        title = self.null_check(self.soup.find_all('div', attrs={'class': 'doc_title'})[-1], 'text')
        self.title=title
        small_title, date = title.split(' on ')
        self.petitioner, self.respondent = small_title.split(' vs ')
        respondent_upper=self.respondent.upper()
        #firm_names=['&', '&DEVELOPERS', '&INFRASTRUCTURE', '&SERVICES', '(BANGALORE)PRIVATE', '(EXPORTS)', '(I)PRIVATE', '(INDIA)LIMITED', '(INDIA)PRIVATE', '(INDIA)PVT', '(OPC)', '(OPC)PRIVATE', '(P)', '(PVT)', ')PRIVATE', 'ABRASIVE', 'ABRASIVES', 'ACADEMIC', 'ACADEMICS', 'ACADEMY', 'ACADEMYPRIVATE', 'ACCENT', 'ACCESSORIES', 'ACCESSORIESPRIVATE', 'ACCOUNTING', 'ACIDS', 'ACOUSTICS', 'ACQUA', 'ADDITIVES', 'ADHESIVE', 'ADHESIVES', 'ADVERTISERS', 'ADVERTISING', 'ADVERTISINGPRIVATE', 'ADVISERS', 'ADVISOR', 'ADVISORS', 'ADVISORSPRIVATE', 'ADVISORY', 'ADVISORYPRIVATE', 'ADVISORYSERVICES', 'AEROSPACE', 'AGENCIES', 'AGENCIESPRIVATE', 'AGENCY', 'AGENCYPRIVATE', 'AGENTS', 'AGGREGATES', 'AGRI-TECH', 'AGRICARE', 'AGRICHEM', 'AGRICO', 'AGRICULTURAL', 'AGRICULTURE', 'AGRIM', 'AGRITECH', 'AGRITRADE', 'AGRO', 'AGRO-FARMS', 'AGRO-TECH', 'AGROCHEM', 'AGROFARM', 'AGROFARMS', 'AGROFOOD', 'AGROFOODS', 'AGROHA', 'AGROPRODUCTS', 'AGROS', 'AGROTECH', 'AGROVET', 'AHEAD', 'AIDS', 'AIRCON', 'AIRCONDITIONING', 'AIRLINES', 'AIRPORT', 'AIRTECH', 'AIRWAYS', 'ALCHEMY', 'ALCO', 'ALKA', 'ALKALIES', 'ALLIANCE', 'ALLIED', 'ALLIEDINDUSTRIES', 'ALLIEDSERVICES', 'ALLOY', 'ALLOYS', 'ALLUMINIUM', 'ALPHA', 'ALPS', 'ALUMINIUM', 'AMALGAMATED', 'AMBAY', 'AMUSEMENTS', 'ANALYSIS', 'ANALYTICAL', 'ANALYTICS', 'ANCHOR', 'ANCILLARIES', 'ANCILLARY', 'ANDCONSTRUCTION', 'ANDCONSTRUCTIONS', 'ANDCONSULTANCY', 'ANDCONSULTANTS', 'ANDDEVELOPERS', 'ANDDEVELOPMENT', 'ANDENGINEERING', 'ANDFINANCE', 'ANDINFRASTRUCTURE', 'ANDINVESTMENT', 'ANDINVESTMENTS', 'ANDMANAGEMENT', 'ANDMARKETING', 'ANDRESEARCH', 'ANDSERVICES', 'ANDSOLUTIONS', 'ANDTECHNOLOGY', 'ANIMATION', 'ANIMATIONS', 'ANNEX', 'APEX', 'APPAREL', 'APPARELS', 'APPLIANCE', 'APPLIANCES', 'APPLIANCESPRIVATE', 'APPLICATION', 'APPLICATIONS', 'APPLIED', 'APPS', 'APT', 'AQUACULTURE', 'AQUATECH', 'AQUATIC', 'ARCADE', 'ARCHITECT', 'ARCHITECTS', 'ARCHITECTURAL', 'ARCHITECTURE', 'ARENA', 'ARMOUR', 'ARMS', 'AROMA', 'AROMATIC', 'AROMATICS', 'ARTS', 'ASBESTOS', 'ASSESSMENT', 'ASSESSORS', 'ASSET', 'ASSETS', 'ASSIST', 'ASSISTANCE', 'ASSOCIATE', 'ASSOCIATED', 'ASSOCIATES', 'ASSOCIATESPRIVATE', 'ASSOCIATION', 'ASSOCIATIONLIMITED', 'AUDIO', 'AUTOCOMP', 'AUTOMATIC', 'AUTOMATION', 'AUTOMATIONS', 'AUTOMOBILE', 'AUTOMOBILES', 'AUTOMOTIVE', 'AUTOMOTIVES', 'AUTOPARTS', 'AUTOS', 'AUTOTECH', 'AUTOWHEELS', 'AVENUE', 'AVENUES', 'AVIATION', 'AYURVED', 'AYURVEDA', 'AYURVEDIC', 'AYURVEDICS', 'AYUSH', 'AYUSHI', 'BAGS', 'BAKERS', 'BAKERY', 'BANK', 'BANQUET', 'BANQUETS', 'BARS', 'BATH', 'BATTERIES', 'BATTERY', 'BAZAAR', 'BAZAR', 'BEANS', 'BEARINGS', 'BEAUTY', 'BELL', 'BELLS', 'BELTING', 'BETA', 'BEVERAGE', 'BEVERAGES', 'BEVERAGESPRIVATE', 'BIKES', 'BINARY', 'BIO', 'BIO-TECH', 'BIOCARE', 'BIOCHEM', 'BIOENERGY', 'BIOFUELS', 'BIOLOGICALS', 'BIOMED', 'BIOMEDICAL', 'BIOPHARMA', 'BIOSCIENCE', 'BIOSCIENCES', 'BIOTEC', 'BIOTECH', 'BIOTECHNOLOGIES', 'BIOTECHNOLOGY', 'BIOTEK', 'BIRDS', 'BISCUIT', 'BISCUITS', 'BITS', 'BIZ', 'BLEACHING', 'BLOCK', 'BLOCKS', 'BLOOM', 'BLOOMING', 'BLOSSOM', 'BLOSSOMS', 'BLOW', 'BOARD', 'BOARDS', 'BODY', 'BOILERS', 'BONANZA', 'BOND', 'BONE', 'BOOK', 'BOOKS', 'BOTTLERS', 'BOTTLING', 'BOUTIQUE', 'BOX', 'BOXES', 'BRAIN', 'BRAINS', 'BRAKES', 'BRAND', 'BRANDING', 'BRANDS', 'BRASS', 'BRAVO', 'BREEDERS', 'BREEDING', 'BREEZE', 'BREWERIES', 'BRICK', 'BRICKS', 'BROAD', 'BROADBAND', 'BROADCAST', 'BROADCASTING', 'BROKER', 'BROKERAGE', 'BROKERS', 'BROKERSPRIVATE', 'BROS', 'BROS.', 'BROTHER', 'BROTHERS', 'BRUSH', 'BUILD-TECH', 'BUILDCOM', 'BUILDCON', 'BUILDCONS', 'BUILDER', 'BUILDERS', 'BUILDERSPRIVATE', 'BUILDESTATE', 'BUILDESTATES', 'BUILDHOME', 'BUILDHOMES', 'BUILDINFRA', 'BUILDING', 'BUILDINGS', 'BUILDMART', 'BUILDMAT', 'BUILDPRO', 'BUILDPROP', 'BUILDSQUARE', 'BUILDTECH', 'BUILDWEL', 'BUILDWELL', 'BUILTECH', 'BULK', 'BULLION', 'BULLIONS', 'BUREAU', 'BUSINESS', 'BUYING', 'BYTE', 'BYTES', 'CABLE', 'CABLES', 'CABS', 'CAFE', 'CALIBRATION', 'CAM', 'CAMELLIA', 'CAMP', 'CANCER', 'CANNING', 'CANS', 'CANVAS', 'CAP', 'CAPACITORS', 'CAPFIN', 'CAPITAL', 'CAPITALS', 'CAPRI', 'CAPRICORN', 'CAPS', 'CAPSTONE', 'CAPSULES', 'CAR', 'CARAVAN', 'CARBIDE', 'CARBIDES', 'CARBON', 'CARBONS', 'CARDIAC', 'CARDS', 'CARE', 'CAREER', 'CAREERS', 'CAREPRIVATE', 'CAREWELL', 'CARGO', 'CARNATION', 'CARNIVAL', 'CARPET', 'CARPETS', 'CARRIER', 'CARRIERS', 'CARS', 'CART', 'CARTONS', 'CASA', 'CASCADE', 'CASH', 'CASHEW', 'CASSETTES', 'CAST', 'CASTING', 'CASTINGS', 'CASTLE', 'CASTOR', 'CATALYST', 'CATERERS', 'CATERING', 'CELL', 'CELLULAR', 'CELLULOSE', 'CEMENT', 'CEMENTS', 'CENTER', 'CENTERPRIVATE', 'CENTERS', 'CENTRAL', 'CENTRE', 'CENTREPRIVATE', 'CENTRES', 'CENTURION', 'CENTURY', 'CERA', 'CERAMIC', 'CERAMICS', 'CEREALS', 'CERTIFICATION', 'CERTIFICATIONS', 'CHAIN', 'CHAINS', 'CHAMBER', 'CHAMBERS', 'CHANGER', 'CHANGERS', 'CHANNEL', 'CHANNELS', 'CHARAN', 'CHARTERED', 'CHARTERING', 'CHEF', 'CHEM', 'CHEMI', 'CHEMICAL', 'CHEMICALS', 'CHEMICALSPRIVATE', 'CHEMIE', 'CHEMTECH', 'CHIP', 'CHIPS', 'CHIT', 'CHITFUND', 'CHITFUNDS', 'CHITS', 'CHITTIES', 'CHITTY', 'CHROME', 'CINE', 'CINEMA', 'CINEMAS', 'CIRCUIT', 'CIRCUITS', 'CLEANERS', 'CLEANING', 'CLEANTECH', 'CLEARING', 'CLINIC', 'CLINICAL', 'CLINICS', 'CLOTHING', 'CLOTHINGS', 'CLOUD', 'CLUB', 'CLUBS', 'CO.', 'CO.LTD.', 'CO.PRIVATE', 'CO.PVT', 'CO.PVT.LTD.', 'COACH', 'COACHING', 'COASTAL', 'COAT', 'COATED', 'COATERS', 'COATING', 'COATINGS', 'COATS', 'CODE', 'COFFEE', 'COGENT', 'COIN', 'COIR', 'COKE', 'COLDSTORAGE', 'COLLECTION', 'COLLECTIONS', 'COLLECTIVE', 'COLLEGE', 'COLLIERIES', 'COLLIERY', 'COLONISERS', 'COLONIZER', 'COLONIZERS', 'COLOR', 'COLORS', 'COLOUR', 'COLOURS', 'COMFORTS', 'COMM', 'COMMERCE', 'COMMERCIAL', 'COMMERCIALS', 'COMMODEAL', 'COMMODITIES', 'COMMODITY', 'COMMOSALE', 'COMMOSALES', 'COMMOTRADE', 'COMMUNICATION', 'COMMUNICATIONPRIVATE', 'COMMUNICATIONS', 'COMMUNICATIONSPRIVATE', 'COMMUNITY', 'COMNET', 'COMP', 'COMPACT', 'COMPANY', 'COMPANYLIMITED', 'COMPANYPRIVATE', 'COMPANYPVT', 'COMPANYPVT.LTD.', 'COMPASS', 'COMPETENT', 'COMPLETE', 'COMPLEX', 'COMPLIANCE', 'COMPNAY', 'COMPONENT', 'COMPONENTS', 'COMPONENTSPRIVATE', 'COMPOSITE', 'COMPOSITES', 'COMPREHENSIVE', 'COMPRESSORS', 'COMPTECH', 'COMPU', 'COMPUSOFT', 'COMPUTECH', 'COMPUTER', 'COMPUTERS', 'COMPUTING', 'COMPUTRONICS', 'COMTECH', 'COMTRADE', 'CONBUILD', 'CONCAST', 'CONCEPT', 'CONCEPTS', 'CONCERN', 'CONCLAVE', 'CONCORD', 'CONCORDE', 'CONCRETE', 'CONCRETES', 'CONDITIONING', 'CONDUCTORS', 'CONES', 'CONFECTIONERS', 'CONFECTIONERY', 'CONFIDENT', 'CONGLOMERATE', 'CONNECT', 'CONNECTION', 'CONNECTIONS', 'CONNECTORS', 'CONSERVATION', 'CONSOLIDATED', 'CONSORTIUM', 'CONSTECH', 'CONSTRO', 'CONSTRUCTION', 'CONSTRUCTIONCOMPANY', 'CONSTRUCTIONPRIVATE', 'CONSTRUCTIONS', 'CONSTRUCTIONSPRIVATE', 'CONSTRUCTORS', 'CONSTRUWELL', 'CONSULT', 'CONSULTANCIES', 'CONSULTANCY', 'CONSULTANCYPRIVATE', 'CONSULTANCYSERVICES', 'CONSULTANT', 'CONSULTANTPRIVATE', 'CONSULTANTS', 'CONSULTANTSPRIVATE', 'CONSULTING', 'CONSULTINGPRIVATE', 'CONSULTINGSERVICES', 'CONSUMABLES', 'CONSUMER', 'CONSUMERS', 'CONTACT', 'CONTAINER', 'CONTAINERS', 'CONTECH', 'CONTEMPORARY', 'CONTINENTAL', 'CONTRACT', 'CONTRACTING', 'CONTRACTOR', 'CONTRACTORS', 'CONTRACTORSPRIVATE', 'CONTRACTS', 'CONTROLS', 'CONVENTION', 'CONVERGENCE', 'CONVERSION', 'CONVERTERS', 'CONVERTORS', 'CONVEYOR', 'CONVEYORS', 'COOLING', 'COPPER', 'COPRIVATE', 'COPVT', 'CORAL', 'CORE', 'CORN', 'CORNER', 'CORNERSTONE', 'COROMANDEL', 'CORONATION', 'CORP', 'CORPN', 'CORPN.', 'CORPORATE', 'CORPORATION', 'CORPORATIONLIMITED', 'CORPORATIONPRIVATE', 'CORRUGATED', 'CORRUGATORS', 'COSMETIC', 'COSMETICS', 'COSMIC', 'COSMO', 'COSMOPOLITAN', 'COSMOS', 'COST', 'COTEX', 'COTSPIN', 'COTSYN', 'COTTAGE', 'COTTAGES', 'COTTON', 'COTTONS', 'COUNCIL', 'COUNTRY', 'COUNTRYWIDE', 'COUNTY', 'COURIER', 'COURIERS', 'COUTURE', 'COZY', 'CRAFT', 'CRAFTS', 'CREAMS', 'CREATION', 'CREATIONS', 'CREATIVE', 'CREATIVES', 'CREATORS', 'CREDIT', 'CREDITS', 'CROP', 'CROPS', 'CROSS', 'CROWN', 'CRUSHER', 'CRUSHERS', 'CRUSHING', 'CRYSTAL', 'CRYSTALS', 'CUISINE', 'CUISINES', 'CULINARY', 'CULTIVATION', 'CULTIVATORS', 'CULTURAL', 'CULTURE', 'CURE', 'CURRENCY', 'CUSTOMER', 'CUTTING', 'CYBER', 'CYBERNETICS', 'CYBERTECH', 'CYCLE', 'CYCLES', 'CYLINDERS', 'DAILY', 'DAIRIES', 'DAIRY', 'DATA', 'DATABASE', 'DATACOM', 'DATAMATICS', 'DATASOFT', 'DEALCOM', 'DEALCOMM', 'DEALER', 'DEALERS', 'DEALMARK', 'DEALS', 'DEALTRADE', 'DEBT', 'DECO', 'DECOR', 'DECORATIVE', 'DECORATORS', 'DECORS', 'DEFENCE', 'DEGREE', 'DELIVERY', 'DELUX', 'DELUXE', 'DEPARTMENTAL', 'DEPOSIT', 'DEPOSITS', 'DEPOT', 'DERIVATIVES', 'DESIGN', 'DESIGNER', 'DESIGNERS', 'DESIGNING', 'DESIGNS', 'DETECTIVE', 'DETECTIVES', 'DETERGENT', 'DETERGENTS', 'DEVBUILD', 'DEVCON', 'DEVELOPER', 'DEVELOPERPRIVATE', 'DEVELOPERS', 'DEVELOPERSLIMITED', 'DEVELOPERSPRIVATE', 'DEVELOPMENT', 'DEVELOPMENTCOMPANY', 'DEVELOPMENTCORPORATION', 'DEVELOPMENTLIMITED', 'DEVELOPMENTPRIVATE', 'DEVELOPMENTS', 'DIAGNOSTIC', 'DIAGNOSTICS', 'DIAM', 'DIAMOND', 'DIAMONDS', 'DIARY', 'DIES', 'DIESEL', 'DIESELS', 'DIGI', 'DIGITAL', 'DIGITECH', 'DIGNITY', 'DIMENSION', 'DIMENSIONS', 'DISCOUNT', 'DISPLAY', 'DISTILLERIES', 'DISTILLERS', 'DISTILLERY', 'DISTRIBUTING', 'DISTRIBUTION', 'DISTRIBUTIONPRIVATE', 'DISTRIBUTIONS', 'DISTRIBUTOR', 'DISTRIBUTORS', 'DISTRIBUTORSPRIVATE', 'DOCTORS', 'DOMAIN', 'DOMESTIC', 'DOORS', 'DOT', 'DOTCOM', 'DOTS', 'DP', 'DRESSES', 'DRILLING', 'DRINKS', 'DRUG', 'DRUGS', 'DRUMS', 'DWELLERS', 'DWELLING', 'DWELLINGS', 'DYE', 'DYE-CHEM', 'DYECHEM', 'DYEING', 'DYERS', 'DYES', 'DYESTUFF', 'DYESTUFFS', 'E-COM', 'E-COMMERCE', 'E-LEARNING', 'E-SERVICES', 'E-SOLUTIONS', 'EARTHMOVERS', 'EATABLES', 'ECO', 'ECOM', 'ECOMMERCE', 'ECONOMIC', 'ECOTECH', 'EDIBLE', 'EDIBLES', 'EDUCARE', 'EDUCATION', 'EDUCATIONAL', 'EDUCATIONPRIVATE', 'EDUCATIONS', 'EDUCOM', 'EDUCON', 'EDUSOFT', 'EDUSOLUTIONS', 'EDUTAINMENT', 'EDUTECH', 'EDUVENTURES', 'EFFLUENT', 'ELECTRIC', 'ELECTRICAL', 'ELECTRICALS', 'ELECTRICALSPRIVATE', 'ELECTRICITY', 'ELECTRICS', 'ELECTRO', 'ELECTRODES', 'ELECTROMECH', 'ELECTRONIC', 'ELECTRONICS', 'ELECTRONICSPRIVATE', 'ELECTROTECH', 'ELEMENTS', 'ELEVATOR', 'ELEVATORS', 'EMBROIDERIES', 'EMBROIDERY', 'EMERGENCY', 'EMERGING', 'EMPLOYEES', 'EMPLOYMENT', 'EMPORIUM', 'EMPOWER', 'ENCLAVE', 'ENCLAVES', 'ENERGIES', 'ENERGY', 'ENERGYPRIVATE', 'ENERTECH', 'ENFIELD', 'ENGG', 'ENGG.', 'ENGICON', 'ENGICONS', 'ENGINE', 'ENGINEER', 'ENGINEERING', 'ENGINEERINGCOMPANY', 'ENGINEERINGPRIVATE', 'ENGINEERINGS', 'ENGINEERINGSERVICES', 'ENGINEERS', 'ENGINEERSPRIVATE', 'ENGINES', 'ENGITECH', 'ENTERPRISE', 'ENTERPRISES', 'ENTERPRISESPRIVATE', 'ENTERTAINERS', 'ENTERTAINMENT', 'ENTERTAINMENTPRIVATE', 'ENTERTAINMENTS', 'ENTREPRENEUR', 'ENTREPRENEURS', 'ENVIROTECH', 'ENVISION', 'EQUIPMENT', 'EQUIPMENTS', 'EQUIPMENTSPRIVATE', 'EQUITIES', 'EQUITY', 'ESCALATORS', 'ESCORTS', 'ESERVICES', 'ESOLUTIONS', 'ESQUIRE', 'ESSENTIALS', 'ESTATE', 'ESTATEPRIVATE', 'ESTATES', 'ESTATESPRIVATE', 'EX-SERVICEMEN', 'EXECUTIVE', 'EXHIBITION', 'EXHIBITIONS', 'EXHIBITORS', 'EXPEDITIONS', 'EXPERIENCE', 'EXPERT', 'EXPERTS', 'EXPLORATION', 'EXPLOSIVES', 'EXPO', 'EXPORT', 'EXPORTERS', 'EXPORTS', 'EXPORTSPRIVATE', 'EXPOSITIONS', 'EXPOTRADE', 'EXTRACTION', 'EXTRACTIONS', 'EXTRACTS', 'EXTRUSION', 'EXTRUSIONS', 'FABRIC', 'FABRICATION', 'FABRICATIONS', 'FABRICATORS', 'FABRICS', 'FABS', 'FABTECH', 'FABTEX', 'FACADE', 'FACILITATORS', 'FACILITIES', 'FACILITY', 'FACILITYMANAGEMENT', 'FACTOR', 'FACTORS', 'FACTORY', 'FACTORYPRIVATE', 'FARMER', 'FARMERS', 'FARMERSPRODUCER', 'FARMING', 'FARMS', 'FARMSPRIVATE', 'FASHION', 'FASHIONS', 'FASTENERS', 'FASTNERS', 'FASTSPEED', 'FATS', 'FEED', 'FEEDS', 'FERROUS', 'FERTILISERS', 'FERTILITY', 'FERTILIZER', 'FERTILIZERS', 'FERTILIZERSPRIVATE', 'FIBER', 'FIBERS', 'FIBRE', 'FIBRES', 'FIELD', 'FIELDS', 'FILAMENT', 'FILAMENTS', 'FILM', 'FILMS', 'FILTER', 'FILTERS', 'FILTRATION', 'FIN-LEASE', 'FINANCE', 'FINANCECOMPANY', 'FINANCELIMITED', 'FINANCEPRIVATE', 'FINANCERS', 'FINANCES', 'FINANCIAL', 'FINANCIALCONSULTANTS', 'FINANCIALS', 'FINANCIALSERVICES', 'FINANCIERS', 'FINANCING', 'FINCAP', 'FINCO', 'FINCOM', 'FINCON', 'FINCORP', 'FINE', 'FINECHEM', 'FINESSE', 'FININVEST', 'FINISHERS', 'FINISHING', 'FINLEASE', 'FINMART', 'FINSEC', 'FINSERV', 'FINSERVE', 'FINSTOCK', 'FINTECH', 'FINTRADE', 'FINVEST', 'FISH', 'FISHERIES', 'FISHING', 'FIT', 'FITNESS', 'FITTINGS', 'FIXTURES', 'FLAT', 'FLATS', 'FLAVOURS', 'FLEET', 'FLEX', 'FLEXI', 'FLEXIBLE', 'FLEXIPACK', 'FLIGHT', 'FLOOR', 'FLOORING', 'FLOORINGS', 'FLOORS', 'FLORA', 'FLORAL', 'FLORENCE', 'FLORICULTURE', 'FLORITECH', 'FLOUR', 'FLOW', 'FLOWER', 'FLOWERS', 'FLUID', 'FLYING', 'FOAM', 'FOAMS', 'FOILS', 'FOOD', 'FOODS', 'FOODSPRIVATE', 'FOODTECH', 'FOOT', 'FOOTBALL', 'FOOTCARE', 'FOOTWEAR', 'FOOTWEARS', 'FORESTRY', 'FOREX', 'FORGE', 'FORGING', 'FORGINGS', 'FORMING', 'FORMS', 'FORMULATION', 'FORMULATIONS', 'FORTUNE', 'FORUM', 'FORWARDERS', 'FOSTER', 'FOTO', 'FOUNDATION', 'FOUNDATIONS', 'FOUNDERS', 'FOUNDRIES', 'FOUNDRY', 'FRAGRANCE', 'FRAGRANCES', 'FRAME', 'FRAMES', 'FRANCHISE', 'FREIGHT', 'FROZEN', 'FRUIT', 'FRUITS', 'FUEL', 'FUELS', 'FUND', 'FUNDLIMITED', 'FUNDPRIVATE', 'FUNDS', 'FUNDSPRIVATE', 'FUNIDEA', 'FURNACE', 'FURNACES', 'FURNISHERS', 'FURNISHING', 'FURNISHINGS', 'FURNITURE', 'FURNITURES', 'FUTURES', 'GADGETS', 'GAIN', 'GALLERY', 'GAME', 'GAMES', 'GAMING', 'GARDENS', 'GARMENT', 'GARMENTS', 'GAS', 'GASES', 'GASKETS', 'GATEWAY', 'GAUGES', 'GEAR', 'GEARS', 'GEMS', 'GENERATORS', 'GENETIC', 'GENETICS', 'GENUINE', 'GEOTECH', 'GIFTS', 'GINNING', 'GLASS', 'GLASSES', 'GLAZE', 'GLAZING', 'GLOVES', 'GOODS', 'GRAINS', 'GRANITE', 'GRANITES', 'GRAPES', 'GRAPHICS', 'GRAPHITE', 'GREENTECH', 'GREY', 'GROUP', 'HABITATS', 'HANDICRAFT', 'HANDICRAFTS', 'HANDLING', 'HANDLOOM', 'HANDLOOMS', 'HARBOUR', 'HARDWARE', 'HARDWARES', 'HEALTHCARE', 'HEALTHTECH', 'HELPLINE', 'HERBALS', 'HERBS', 'HERITAGE', 'HI-TECH', 'HITECH', 'HOLDING', 'HOLDINGS', 'HOLDINGSPRIVATE', 'HOLIDAYS', 'HOSPITAL', 'HOSPITALITIES', 'HOSPITALITY', 'HOSPITALITYPRIVATE', 'HOSPITALPRIVATE', 'HOSPITALS', 'HOSPITALSPRIVATE', 'HOTELS', 'HOTELSPRIVATE', 'HOUSING', 'HYBRID', 'HYDEL', 'HYDRAULIC', 'HYDRAULICS', 'HYDRO', 'HYDROCARBONS', 'HYGIENE', 'HYGIENIC', 'I-TECH', 'I.T.', 'IMAGINE', 'IMAGING', 'IMPORT', 'IMPORTERS', 'IMPORTS', 'INDUSTRIAL', 'INDUSTRIALS', 'INDUSTRIES', 'INDUSTRIESLIMITED', 'INDUSTRIESPRIVATE', 'INDUSTRIESPVT', 'INDUSTRY', 'INFIN', 'INFINITE', 'INFINITI', 'INFINITY', 'INFO', 'INFO-TECH', 'INFOCARE', 'INFOCOM', 'INFOCOMM', 'INFOMATICS', 'INFOMEDIA', 'INFORMATIC', 'INFORMATICS', 'INFORMATION', 'INFORMATIONTECHNOLOGY', 'INFOSERVE', 'INFOSERVICES', 'INFOSOFT', 'INFOSOL', 'INFOSOLUTION', 'INFOSOLUTIONS', 'INFOSYS', 'INFOSYSTEM', 'INFOSYSTEMS', 'INFOTAINMENT', 'INFOTEC', 'INFOTECH', 'INFOTEK', 'INFOTEL', 'INFOVISION', 'INFOWARE', 'INFOWAY', 'INFOWAYS', 'INFOWEB', 'INFRA', 'INFRA-TECH', 'INFRAA', 'INFRABUILD', 'INFRABUILDCON', 'INFRABUILDERS', 'INFRACARE', 'INFRACITY', 'INFRACOM', 'INFRACON', 'INFRACONS', 'INFRACONSTRUCTION', 'INFRADEV', 'INFRADEVELOPER', 'INFRADEVELOPERS', 'INFRAENGINEERS', 'INFRAESTATE', 'INFRAESTATES', 'INFRAHEIGHT', 'INFRAPROJECT', 'INFRAPROJECTS', 'INFRAREALTY', 'INFRASERVICES', 'INFRASOLUTION', 'INFRASOLUTIONS', 'INFRASPACE', 'INFRASTRUCTURAL', 'INFRASTRUCTURE', 'INFRASTRUCTUREDEVELOPMENT', 'INFRASTRUCTURELIMITED', 'INFRASTRUCTUREPRIVATE', 'INFRASTRUCTURES', 'INFRASTRUCTURESPRIVATE', 'INFRATEC', 'INFRATECH', 'INFRATEL', 'INFRATRADE', 'INFRAVENTURE', 'INFRAVENTURES', 'INFRAVISION', 'INFRAZONE', 'INGENIOUS', 'INGOTS', 'INGREDIENTS', 'INITIATIVE', 'INITIATIVES', 'INK', 'INKS', 'INLAND', 'INN', 'INNOTECH', 'INNOVA', 'INNOVATION', 'INNOVATIONS', 'INNOVATIVE', 'INNOVATORS', 'INNOVISION', 'INORGANICS', 'INSECTICIDES', 'INSPECTION', 'INSTITUTE', 'INSTITUTEPRIVATE', 'INSTITUTES', 'INSTITUTION', 'INSTITUTIONS', 'INSTRUMENT', 'INSTRUMENTATION', 'INSTRUMENTS', 'INSULATION', 'INSULATIONS', 'INSULATORS', 'INSURANCE', 'INTECH', 'INTEGRA', 'INTEGRAL', 'INTEGRATED', 'INTEGRATION', 'INTEGRATORS', 'INTEGRITY', 'INTEL', 'INTELLECT', 'INTELLECTUAL', 'INTELLIGENCE', 'INTELLIGENT', 'INTER', 'INTERACTIVE', 'INTERCONTINENTAL', 'INTERFACE', 'INTERIORS', 'INTERLINK', 'INTERMEDIATES', 'INTERNATIONAL', 'INTERNATIONALPRIVATE', 'INTERNET', 'INTERTRADE', 'INTIMATE', 'INTIME', 'INTRA', 'INTRADE', 'INVENTIVE', 'INVEST', 'INVESTIGATION', 'INVESTMENT', 'INVESTMENTCOMPANY', 'INVESTMENTLIMITED', 'INVESTMENTPRIVATE', 'INVESTMENTS', 'INVESTMENTSLIMITED', 'INVESTMENTSPRIVATE', 'INVESTOR', 'INVESTORS', 'INVESTRADE', 'IRRIGATION', 'JEWEL', 'JEWELLERS', 'JEWELLERSPRIVATE', 'JEWELLERY', 'JEWELRY', 'JEWELS', 'KITCHEN', 'KITCHENS', 'KNIT', 'KNITS', 'KNITTERS', 'KNITTING', 'KNITWEAR', 'KNITWEARS', 'LAB', 'LABEL', 'LABELS', 'LABORATORIES', 'LABORATORIESPRIVATE', 'LABORATORY', 'LABOUR', 'LABS', 'LAMINATES', 'LAMINATIONS', 'LAMINATORS', 'LAMP', 'LAMPS', 'LANDDEVELOPERS', 'LATEX', 'LAUNDRY', 'LAW', 'LEAD', 'LEADING', 'LEARNING', 'LEASE', 'LEASING', 'LEASINGLIMITED', 'LEASINGPRIVATE', 'LEASINGS', 'LEATHER', 'LEATHERS', 'LEISURES', 'LIFESTYLE', 'LIFESTYLES', 'LIFT', 'LIFTS', 'LIGHT', 'LIGHTING', 'LIGHTINGS', 'LIGHTS', 'LIMITED', 'LIMITED(OPC)', 'LIMITED,', 'LIMITED.', 'LIMTED', 'LIMTIED', 'LINERS', 'LINES', 'LINK', 'LINKERS', 'LINKS', 'LIQUOR', 'LIQUORS', 'LOAN', 'LOANS', 'LOCKERS', 'LOCKS', 'LODGE', 'LOGIC', 'LOGICAL', 'LOGICS', 'LOGISTIC', 'LOGISTICS', 'LOGISTICSPRIVATE', 'LTD', 'LTD,', 'LTD.', 'LTD.,', 'LUBE', 'LUBES', 'LUBRICANTS', 'LUGGAGE', 'LUMBERS', 'MACHINE', 'MACHINERIES', 'MACHINERY', 'MACHINES', 'MACHINING', 'MACRO', 'MAGNET', 'MAGNETIC', 'MAGNETICS', 'MAINTENANCE', 'MAKERS', 'MALLS', 'MANAGED', 'MANAGEMENT', 'MANAGEMENTCONSULTANTS', 'MANAGEMENTPRIVATE', 'MANAGEMENTS', 'MANAGEMENTSERVICES', 'MANAGERS', 'MANAGMENT', 'MANUFACTURERS', 'MANUFACTURERSPRIVATE', 'MANUFACTURES', 'MANUFACTURING', 'MANUFACTURINGCOMPANY', 'MANUFACTURINGPRIVATE', 'MARBLE', 'MARBLES', 'MARKET', 'MARKETERS', 'MARKETING', 'MARKETINGPRIVATE', 'MARKETINGS', 'MARKETINGSERVICES', 'MARKETPLACE', 'MARKETS', 'MARKETTING', 'MATCH', 'MATCHES', 'MATE', 'MATERIAL', 'MATERIALS', 'MATRIMONIAL', 'MECH', 'MECHANICAL', 'MECHANICALS', 'MECHATRONICS', 'MED', 'MEDIA', 'MEDIATECH', 'MEDICAL', 'MEDICALS', 'MEDICAMENTS', 'MEDICINE', 'MEDICINES', 'MEDICO', 'MEDICOS', 'MEDITECH', 'MEDTECH', 'MEENA', 'MEENAKSHI', 'MEERA', 'MEERUT', 'MEET', 'MEGA', 'MEGACITY', 'MEGAMART', 'MEGH', 'MEGHA', 'MEGHALAYA', 'MEGHANA', 'MEGHDOOT', 'MEGHNA', 'MEHAK', 'MEHAR', 'MEHER', 'MEHRA', 'MEHTA', 'MEHUL', 'MELANGE', 'MELODY', 'MEMORIAL', 'MEMORY', 'MEN', 'MENON', 'MENTOR', 'MENTORS', 'MEP', 'MERA', 'MERCANTILE', 'MERCANTILES', 'MERCHANDISE', 'MERCHANDISERS', 'MERCHANDISING', 'MERCHANT', 'MERCHANTILE', 'MERCHANTS', 'MERCURY', 'MERIDIAN', 'MERIT', 'MERLIN', 'MERRY', 'MERU', 'MESH', 'MET', 'META', 'METACAST', 'METAL', 'METALIKS', 'METALLIC', 'METALLICS', 'METALLOYS', 'METALLURGICAL', 'METALLURGICALS', 'METALS', 'METERS', 'METRO', 'METROPOLIS', 'METROPOLITAN', 'MEWAR', 'MFG', 'MFG.', 'MG', 'MICA', 'MICRO', 'MICROFINANCE', 'MICRON', 'MICRONS', 'MICROSYSTEMS', 'MICROTECH', 'MIDAS', 'MIDDLE', 'MIDLAND', 'MIDWAY', 'MIGRATION', 'MILE', 'MILES', 'MILESTONE', 'MILK', 'MILL', 'MILLENIUM', 'MILLENNIUM', 'MILLERS', 'MILLING', 'MILLS', 'MILLSLIMITED', 'MILLSPRIVATE', 'MINAR', 'MINCHEM', 'MIND', 'MINDA', 'MINDS', 'MINE', 'MINECHEM', 'MINERAL', 'MINERALS', 'MINERALSPRIVATE', 'MINERVA', 'MINES', 'MINI', 'MINING', 'MINT', 'MIR', 'MIRA', 'MIRACLE', 'MIRAGE', 'MIRAJ', 'MIRROR', 'MISSION', 'MIST', 'MOBILE', 'MOBILES', 'MOBILITY', 'MOBITECH', 'MOD', 'MODE', 'MODEL', 'MODULAR', 'MOMENTUM', 'MONARCH', 'MONEY', 'MONITORING', 'MONSOON', 'MOON', 'MOONLIGHT', 'MOONLINK', 'MOONSHINE', 'MOONSTAR', 'MOSAIC', 'MOTEL', 'MOTELS', 'MOTION', 'MOTOR', 'MOTORS', 'MOULD', 'MOULDERS', 'MOULDING', 'MOULDINGS', 'MOULDS', 'MOVERS', 'MOVIE', 'MOVIES', 'MOVING', 'MULTI', 'MULTI-TRADE', 'MULTICON', 'MULTILEVEL', 'MULTILINK', 'MULTIMEDIA', 'MULTIPLE', 'MULTIPLEX', 'MULTIPROJECTS', 'MULTIPURPOSE', 'MULTISERVICES', 'MULTISOLUTIONS', 'MULTISPECIALITY', 'MULTITECH', 'MULTITRADE', 'MULTITRADING', 'MULTIVENTURES', 'NANOTECH', 'NATURALS', 'NEOTECH', 'NETCOM', 'NETLINK', 'NETMART', 'NETS', 'NETWORK', 'NETWORKING', 'NETWORKPRIVATE', 'NETWORKS', 'NETWORTH', 'NEURO', 'NURSING', 'NUTRACEUTICALS', 'NUTRI', 'NUTRIENTS', 'NUTRITION', 'NUTRITIONS', 'OIL', 'OILFIELD', 'OILS', 'OLIVE', 'ONCOLOGY', 'OPERATORS', 'OPTION', 'OPTIONS', 'ORCHID', 'ORCHIDS', 'ORGANISATION', 'ORGANISERS', 'ORGANIZATION', 'ORIENTAL', 'ORNATE', 'ORTHO', 'OUTSOURCE', 'OUTSOURCING', 'OVERSEAS', 'OWNERS', 'OXYGEN', 'OYSTER', 'OZONE', 'P.LTD.', 'PACK', 'PACKAGERS', 'PACKAGES', 'PACKAGING', 'PACKAGINGS', 'PACKERS', 'PACKING', 'PACKS', 'PAGES', 'PAINT', 'PAINTS', 'PALACE', 'PALMS', 'PANEL', 'PANELS', 'PAPER', 'PAPERS', 'PARADISE', 'PARAM', 'PARKPRIVATE', 'PARTNERS', 'PAYMENT', 'PAYMENTS', 'PEARL', 'PEARLS', 'PELICAN', 'PENS', 'PEOPLE', 'PEOPLES', 'PERFUMES', 'PERIPHERALS', 'PERSONNEL', 'PEST', 'PESTICIDES', 'PET', 'PETALS', 'PETRO', 'PETROCHEM', 'PETROCHEMICALS', 'PETROLEUM', 'PETROLEUMS', 'PHARMA', 'PHARMACEUTICAL', 'PHARMACEUTICALS', 'PHARMACEUTICALSPRIVATE', 'PHARMACHEM', 'PHARMACY', 'PHARMATECH', 'PHOENIX', 'PHOSPHATES', 'PHOTO', 'PHOTOGRAPHY', 'PHOTON', 'PICTURE', 'PICTURES', 'PIGMENTS', 'PILOT', 'PIPE', 'PIPES', 'PIPING', 'PLACEMENT', 'PLACEMENTS', 'PLANNER', 'PLANNERS', 'PLANNING', 'PLANTATION', 'PLANTATIONS', 'PLANTERS', 'PLANTS', 'PLASMA', 'PLASTECH', 'PLASTIC', 'PLASTICS', 'PLASTO', 'PLASTOCHEM', 'PLATFORM', 'PLATINUM', 'PLAY', 'PLAZA', 'PLAZZA', 'PLEASANT', 'PLEASURE', 'PLEXUS', 'PLYWOOD', 'PLYWOODS', 'PNEUMATICS', 'POINT', 'POLAR', 'POLARIS', 'POLE', 'POLES', 'POLESTAR', 'POLLUTION', 'POLY', 'POLYCHEM', 'POLYCLINIC', 'POLYCOATS', 'POLYESTER', 'POLYESTERS', 'POLYFAB', 'POLYFILMS', 'POLYGON', 'POLYMER', 'POLYMERS', 'POLYPACK', 'POLYPACKS', 'POLYPLAST', 'POLYSACKS', 'POLYTECH', 'POLYTEX', 'PONDICHERRY', 'PORT', 'PORTFOLIOS', 'PORTS', 'PORWAL', 'POST', 'POTENTIAL', 'POULTRIES', 'POULTRY', 'POWDER', 'POWDERS', 'POWERGEN', 'POWERS', 'POWERTECH', 'POWERTRONICS', 'PPRIVATE', 'PRECISIONS', 'PREMIUM', 'PRESS', 'PRESSINGS', 'PRESSURE', 'PRESTIGE', 'PRIAVTE', 'PRIMARY', 'PRIME', 'PRINTING', 'PRINTPACK', 'PRINTS', 'PRISM', 'PRIVATE', 'PRIVATELIMITED', 'PRIVATELIMITED.', 'PRIVATELTD', 'PROACTIVE', 'PROBUILD', 'PROCESS', 'PROCESSED', 'PROCESSES', 'PROCESSING', 'PROCESSINGPRIVATE', 'PROCESSOR', 'PROCESSORS', 'PROCON', 'PRODIGY', 'PRODUCE', 'PRODUCER', 'PRODUCERCOMPANY', 'PRODUCERS', 'PRODUCT', 'PRODUCTION', 'PRODUCTIONS', 'PRODUCTS', 'PRODUCTSLIMITED', 'PRODUCTSPRIVATE', 'PROFESSIONAL', 'PROFESSIONALS', 'PROFICIENT', 'PROFILE', 'PROFILES', 'PROFIT', 'PROGRESS', 'PROGRESSIVE', 'PROJECT', 'PROJECTS', 'PROJECTSLIMITED', 'PROJECTSPRIVATE', 'PROLIFIC', 'PROMINENT', 'PROMOTER', 'PROMOTERS', 'PROMOTERSPRIVATE', 'PROMOTION', 'PROMOTIONS', 'PROMOTORS', 'PROMPT', 'PROPBUILD', 'PROPERTIES', 'PROPERTIESPRIVATE', 'PROPERTY', 'PROPMART', 'PROPTECH', 'PROSPERITY', 'PROTECH', 'PROTECT', 'PROTECTIVE', 'PROTEINS', 'PROVIDENT', 'PROVIDER', 'PROVIDERS', 'PUBLICATION', 'PUBLICATIONS', 'PUBLICATIONSPRIVATE', 'PUBLISHERS', 'PUBLISHING', 'PULSES', 'PUMP', 'PUMPS', 'PVT.', 'PVT.LTD', 'PVT.LTD.', 'PVTLTD', 'PVTLTD.', 'QUALITY', 'QUARRIES', 'QUARRY', 'QUARTZ', 'QUEEN', 'RACING', 'RADIATORS', 'RAYON', 'RAYONS', 'REAL', 'REALATORS', 'REALBUILD', 'REALCON', 'REALESTATE', 'REALESTATES', 'REALHOME', 'REALINFRA', 'REALITIES', 'REALITY', 'REALMART', 'REALTECH', 'REALTERS', 'REALTIES', 'REALTIME', 'REALTOR', 'REALTORS', 'REALTORSPRIVATE', 'REALTY', 'RECKON', 'RECON', 'RECONSTRUCTION', 'RECORDING', 'RECORDS', 'RECREATION', 'RECREATIONS', 'RECRUITERS', 'RECRUITMENT', 'RECYCLERS', 'RECYCLING', 'REFINERIES', 'REFINERY', 'REFRACTORIES', 'REFRACTORY', 'REFRIGERATION', 'REMEDIES', 'RENAISSANCE', 'RENEWABLE', 'RENEWABLES', 'REROLLING', 'RESEARCH', 'RESEARCHCENTRE', 'RESEARCHFOUNDATION', 'RESEARCHINSTITUTE', 'RESEARCHPRIVATE', 'RESIDENCES', 'RESIDENCY', 'RESIDENTIAL', 'RESINS', 'RESONANCE', 'RESORT', 'RESORTS', 'RESORTSPRIVATE', 'RESOURCE', 'RESOURCES', 'RESOURCESPRIVATE', 'RESPONSE', 'RESTAURANT', 'RESTAURANTS', 'RETAIL', 'RETAILERS', 'RETAILING', 'RETAILS', 'RETREADING', 'RETREADS', 'RETREAT', 'RETREATS', 'ROADLINES', 'ROADS', 'ROADWAYS', 'ROBOTICS', 'RUBBER', 'RUBBERS', 'RUIA', 'SAPHIRE', 'SAPPHIRE', 'SAR', 'SAREE', 'SAREES', 'SATELLITE', 'SAVING', 'SAVINGS', 'SAW', 'SCAFFOLDING', 'SCAN', 'SCANS', 'SCHEME', 'SCHOOL', 'SCHOOLS', 'SCIENCE', 'SCIENCES', 'SCIENCESPRIVATE', 'SCIENTIFIC', 'SEAFOODS', 'SEAL', 'SEALS', 'SECURITIES', 'SECURITIESPRIVATE', 'SECURITY', 'SECURITYSERVICES', 'SEED', 'SEEDS', 'SENSORS', 'SERICULTURE', 'SERVICE', 'SERVICEPRIVATE', 'SERVICES', 'SERVICES(INDIA)', 'SERVICESINDIA', 'SERVICESLIMITED', 'SERVICESPRIVATE', 'SERVICESPVT', 'SHADES', 'SHARES', 'SHEETS', 'SHELTER', 'SHELTERS', 'SHIPBREAKING', 'SHIPYARD', 'SHOE', 'SHOES', 'SHOP', 'SHOPPE', 'SHOPPEE', 'SHOPPERS', 'SHOPPING', 'SIGNAGE', 'SIGNS', 'SILICA', 'SILICATE', 'SILICATES', 'SILICON', 'SILK', 'SILKS', 'SILVER', 'SILVERLINE', 'SIZING', 'SKILLS', 'SMITH', 'SNACKS', 'SOAP', 'SOAPS', 'SOFT-TECH', 'SOFTECH', 'SOFTSOLUTIONS', 'SOFTTECH', 'SOFTWARE', 'SOFTWARES', 'SOLUTION', 'SOLUTIONPRIVATE', 'SOLUTIONS', 'SOLUTIONS(INDIA)', 'SOLUTIONSPRIVATE', 'SOLVENT', 'SOLVENTS', 'SONAR', 'SOURCES', 'SOURCING', 'SPARE', 'SPARES', 'SPARKLE', 'SPICE', 'SPICES', 'SPINNERS', 'SPINNING', 'SPIRIT', 'SPIRITS', 'SPONGE', 'SPORTING', 'SPORTS', 'STAINLESS', 'STANDARD', 'STATIONERS', 'STATIONERY', 'STEEL', 'STEELS', 'STOCK', 'STOCKS', 'STONE', 'STONES', 'STORAGE', 'STORAGEPRIVATE', 'STORAGES', 'STORE', 'STORES', 'STRATEGIES', 'STRATEGY', 'STRUCTURALS', 'STRUCTURE', 'STRUCTURES', 'STUDIO', 'STUDIOS', 'STYLE', 'STYLES', 'STYLISH', 'SUGAR', 'SUGARCANE', 'SUGARS', 'SUITES', 'SUITINGS', 'SUPPLIER', 'SUPPLIERS', 'SUPPLIERSPRIVATE', 'SUPPLIES', 'SUPPLY', 'SURFACE', 'SURFACTANTS', 'SURGE', 'SURGICAL', 'SURGICALS', 'SURVEILLANCE', 'SURVEYORS', 'SURVEYS', 'SUSTAINABLE', 'SWEET', 'SWEETS', 'SYNAPSE', 'SYNERGIES', 'SYNERGY', 'SYNTHETIC', 'SYNTHETICS', 'SYSTECH', 'SYSTEM', 'SYSTEMATIC', 'SYSTEMPRIVATE', 'SYSTEMS', 'SYSTEMSPRIVATE', 'TALKIES', 'TANK', 'TANKS', 'TANNERIES', 'TANNERS', 'TANNERY', 'TANNING', 'TAPE', 'TAPES', 'TAXI', 'TAYAL', 'TEAK', 'TEC', 'TECH', 'TECHLABS', 'TECHNET', 'TECHNICAL', 'TECHNICS', 'TECHNIK', 'TECHNIQUES', 'TECHNO', 'TECHNOBUILD', 'TECHNOCAST', 'TECHNOCOM', 'TECHNOCRAFT', 'TECHNOCRAT', 'TECHNOCRATE', 'TECHNOCRATES', 'TECHNOCRATS', 'TECHNOFAB', 'TECHNOLABS', 'TECHNOLOGICAL', 'TECHNOLOGIES', 'TECHNOLOGIESPRIVATE', 'TECHNOLOGY', 'TECHNOLOGYPRIVATE', 'TECHNOPLAST', 'TECHNOSOFT', 'TECHNOSOLUTIONS', 'TECHNOSYS', 'TECHNOVISION', 'TECHSOFT', 'TECHSOLUTIONS', 'TELEMATICS', 'TELEMEDIA', 'TELENET', 'TELESERVICES', 'TELESOFT', 'TELESYSTEMS', 'TELETECH', 'TELEVENTURES', 'TELEVISION', 'TERMINALS', 'TEXFAB', 'TEXO', 'TEXTILE', 'TEXTILES', 'TEXTRADE', 'TEXTURISERS', 'TEXTURISING', 'THERAPEUTICS', 'THERAPY', 'THERMAL', 'THERMO', 'THINK', 'THREADS', 'TILE', 'TILES', 'TIMBER', 'TIMBERS', 'TISSUE', 'TISSUES', 'TOBACCO', 'TOLL', 'TOLLWAY', 'TOLLWAYS', 'TOOL', 'TOOLING', 'TOOLINGS', 'TOOLS', 'TOPAZ', 'TOPLINE', 'TOPSTAR', 'TORRENT', 'TOURISM', 'TOURIST', 'TOURS', 'TOWERS', 'TOWNSHIPS', 'TOYS', 'TRADE', 'TRADE-LINK', 'TRADECOM', 'TRADECOMM', 'TRADECON', 'TRADEFIN', 'TRADELINK', 'TRADELINKS', 'TRADEMART', 'TRADEPLACE', 'TRADER', 'TRADERS', 'TRADERSPRIVATE', 'TRADES', 'TRADEVIN', 'TRADEVISION', 'TRADEWELL', 'TRADEWING', 'TRADEWINGS', 'TRADEX', 'TRADEXIM', 'TRADING', 'TRADINGCOMPANY', 'TRADINGPRIVATE', 'TRADINGS', 'TRAFFIC', 'TRAINERS', 'TRAINING', 'TRAININGPRIVATE', 'TRANSCRIPTION', 'TRANSFORMER', 'TRANSFORMERS', 'TRANSIT', 'TRANSLOGISTICS', 'TRANSMISSION', 'TRANSMISSIONS', 'TRANSPORT', 'TRANSPORTATION', 'TRANSPORTERS', 'TRANSPORTS', 'TRANSTECH', 'TRANSWAYS', 'TRANSWORLD', 'TRAUMA', 'TRAVELLERS', 'TRAVELS', 'TRAVELSPRIVATE', 'TREADS', 'TRENDS', 'TRENDZ', 'TREXIM', 'TRIO', 'TUBE', 'TUBES', 'TYRE', 'TYRES', 'UMBRELLA', 'UNION', 'UNITECH', 'UTENSILS', 'UTILITIES', 'VACATIONS', 'VALUERS', 'VALVES', 'VAULT', 'VAULTS', 'VEHICLES', 'VENEER', 'VENEERS', 'VENTURA', 'VENTURESPRIVATE', 'VIDEO', 'WAREHOUSE', 'WAREHOUSING', 'WARES', 'WATCH', 'WATCHES', 'WATERPROOFING', 'WAVES', 'WEAR', 'WEARS', 'WEAVE', 'WEAVERS', 'WEAVES', 'WEAVING', 'WEB', 'WEBSOFT', 'WEBSOLUTIONS', 'WEBTECH', 'WEIGHING', 'WELD', 'WELDING', 'WELFARE', 'WHEAT', 'WHEEL', 'WHEELS', 'WHOLESALE', 'WINDOW', 'WINDOWS', 'WINE', 'WINERY', 'WINES', 'WING', 'WINGS', 'WIRES', 'WOOL', 'WOOLLEN', 'WOOLLENS', 'WOOLTEX', 'WORKSHOP', 'WORKSPRIVATE', 'X', 'YARD', 'YARN', 'YARNS', 'ZINC', 'ZIP', 'ZIPPERS', 'PVT']
        govt_names=['STATE','GOVERNMENT']
        if any(ext in respondent_upper for ext in govt_names):
            self.resgov=1
            
        else:
            
            self.resgov=0
        
        
    def preprocess(self):
        
        #stopwords=['&', '&DEVELOPERS', '&INFRASTRUCTURE', '&SERVICES', '(BANGALORE)PRIVATE', '(EXPORTS)', '(I)PRIVATE', '(INDIA)LIMITED', '(INDIA)PRIVATE', '(INDIA)PVT', '(OPC)', '(OPC)PRIVATE', '(P)', '(PVT)', ')PRIVATE', 'ABRASIVE', 'ABRASIVES', 'ACADEMIC', 'ACADEMICS', 'ACADEMY', 'ACADEMYPRIVATE', 'ACCENT', 'ACCESSORIES', 'ACCESSORIESPRIVATE', 'ACCOUNTING', 'ACIDS', 'ACOUSTICS', 'ACQUA', 'ADDITIVES', 'ADHESIVE', 'ADHESIVES', 'ADVERTISERS', 'ADVERTISING', 'ADVERTISINGPRIVATE', 'ADVISERS', 'ADVISOR', 'ADVISORS', 'ADVISORSPRIVATE', 'ADVISORY', 'ADVISORYPRIVATE', 'ADVISORYSERVICES', 'AEROSPACE', 'AGENCIES', 'AGENCIESPRIVATE', 'AGENCY', 'AGENCYPRIVATE', 'AGENTS', 'AGGREGATES', 'AGRI-TECH', 'AGRICARE', 'AGRICHEM', 'AGRICO', 'AGRICULTURAL', 'AGRICULTURE', 'AGRIM', 'AGRITECH', 'AGRITRADE', 'AGRO', 'AGRO-FARMS', 'AGRO-TECH', 'AGROCHEM', 'AGROFARM', 'AGROFARMS', 'AGROFOOD', 'AGROFOODS', 'AGROHA', 'AGROPRODUCTS', 'AGROS', 'AGROTECH', 'AGROVET', 'AHEAD', 'AIDS', 'AIRCON', 'AIRCONDITIONING', 'AIRLINES', 'AIRPORT', 'AIRTECH', 'AIRWAYS', 'ALCHEMY', 'ALCO', 'ALKA', 'ALKALIES', 'ALLIANCE', 'ALLIED', 'ALLIEDINDUSTRIES', 'ALLIEDSERVICES', 'ALLOY', 'ALLOYS', 'ALLUMINIUM', 'ALPHA', 'ALPS', 'ALUMINIUM', 'AMALGAMATED', 'AMBAY', 'AMUSEMENTS', 'ANALYSIS', 'ANALYTICAL', 'ANALYTICS', 'ANCHOR', 'ANCILLARIES', 'ANCILLARY', 'ANDCONSTRUCTION', 'ANDCONSTRUCTIONS', 'ANDCONSULTANCY', 'ANDCONSULTANTS', 'ANDDEVELOPERS', 'ANDDEVELOPMENT', 'ANDENGINEERING', 'ANDFINANCE', 'ANDINFRASTRUCTURE', 'ANDINVESTMENT', 'ANDINVESTMENTS', 'ANDMANAGEMENT', 'ANDMARKETING', 'ANDRESEARCH', 'ANDSERVICES', 'ANDSOLUTIONS', 'ANDTECHNOLOGY', 'ANIMATION', 'ANIMATIONS', 'ANNEX', 'APEX', 'APPAREL', 'APPARELS', 'APPLIANCE', 'APPLIANCES', 'APPLIANCESPRIVATE', 'APPLICATION', 'APPLICATIONS', 'APPLIED', 'APPS', 'APT', 'AQUACULTURE', 'AQUATECH', 'AQUATIC', 'ARCADE', 'ARCHITECT', 'ARCHITECTS', 'ARCHITECTURAL', 'ARCHITECTURE', 'ARENA', 'ARMOUR', 'ARMS', 'AROMA', 'AROMATIC', 'AROMATICS', 'ARTS', 'ASBESTOS', 'ASSESSMENT', 'ASSESSORS', 'ASSET', 'ASSETS', 'ASSIST', 'ASSISTANCE', 'ASSOCIATE', 'ASSOCIATED', 'ASSOCIATES', 'ASSOCIATESPRIVATE', 'ASSOCIATION', 'ASSOCIATIONLIMITED', 'AUDIO', 'AUTOCOMP', 'AUTOMATIC', 'AUTOMATION', 'AUTOMATIONS', 'AUTOMOBILE', 'AUTOMOBILES', 'AUTOMOTIVE', 'AUTOMOTIVES', 'AUTOPARTS', 'AUTOS', 'AUTOTECH', 'AUTOWHEELS', 'AVENUE', 'AVENUES', 'AVIATION', 'AYURVED', 'AYURVEDA', 'AYURVEDIC', 'AYURVEDICS', 'AYUSH', 'AYUSHI', 'BAGS', 'BAKERS', 'BAKERY', 'BANK', 'BANQUET', 'BANQUETS', 'BARS', 'BATH', 'BATTERIES', 'BATTERY', 'BAZAAR', 'BAZAR', 'BEANS', 'BEARINGS', 'BEAUTY', 'BELL', 'BELLS', 'BELTING', 'BETA', 'BEVERAGE', 'BEVERAGES', 'BEVERAGESPRIVATE', 'BIKES', 'BINARY', 'BIO', 'BIO-TECH', 'BIOCARE', 'BIOCHEM', 'BIOENERGY', 'BIOFUELS', 'BIOLOGICALS', 'BIOMED', 'BIOMEDICAL', 'BIOPHARMA', 'BIOSCIENCE', 'BIOSCIENCES', 'BIOTEC', 'BIOTECH', 'BIOTECHNOLOGIES', 'BIOTECHNOLOGY', 'BIOTEK', 'BIRDS', 'BISCUIT', 'BISCUITS', 'BITS', 'BIZ', 'BLEACHING', 'BLOCK', 'BLOCKS', 'BLOOM', 'BLOOMING', 'BLOSSOM', 'BLOSSOMS', 'BLOW', 'BOARD', 'BOARDS', 'BODY', 'BOILERS', 'BONANZA', 'BOND', 'BONE', 'BOOK', 'BOOKS', 'BOTTLERS', 'BOTTLING', 'BOUTIQUE', 'BOX', 'BOXES', 'BRAIN', 'BRAINS', 'BRAKES', 'BRAND', 'BRANDING', 'BRANDS', 'BRASS', 'BRAVO', 'BREEDERS', 'BREEDING', 'BREEZE', 'BREWERIES', 'BRICK', 'BRICKS', 'BROAD', 'BROADBAND', 'BROADCAST', 'BROADCASTING', 'BROKER', 'BROKERAGE', 'BROKERS', 'BROKERSPRIVATE', 'BROS', 'BROS.', 'BROTHER', 'BROTHERS', 'BRUSH', 'BUILD-TECH', 'BUILDCOM', 'BUILDCON', 'BUILDCONS', 'BUILDER', 'BUILDERS', 'BUILDERSPRIVATE', 'BUILDESTATE', 'BUILDESTATES', 'BUILDHOME', 'BUILDHOMES', 'BUILDINFRA', 'BUILDING', 'BUILDINGS', 'BUILDMART', 'BUILDMAT', 'BUILDPRO', 'BUILDPROP', 'BUILDSQUARE', 'BUILDTECH', 'BUILDWEL', 'BUILDWELL', 'BUILTECH', 'BULK', 'BULLION', 'BULLIONS', 'BUREAU', 'BUSINESS', 'BUYING', 'BYTE', 'BYTES', 'CABLE', 'CABLES', 'CABS', 'CAFE', 'CALIBRATION', 'CAM', 'CAMELLIA', 'CAMP', 'CANCER', 'CANNING', 'CANS', 'CANVAS', 'CAP', 'CAPACITORS', 'CAPFIN', 'CAPITAL', 'CAPITALS', 'CAPRI', 'CAPRICORN', 'CAPS', 'CAPSTONE', 'CAPSULES', 'CAR', 'CARAVAN', 'CARBIDE', 'CARBIDES', 'CARBON', 'CARBONS', 'CARDIAC', 'CARDS', 'CARE', 'CAREER', 'CAREERS', 'CAREPRIVATE', 'CAREWELL', 'CARGO', 'CARNATION', 'CARNIVAL', 'CARPET', 'CARPETS', 'CARRIER', 'CARRIERS', 'CARS', 'CART', 'CARTONS', 'CASA', 'CASCADE', 'CASH', 'CASHEW', 'CASSETTES', 'CAST', 'CASTING', 'CASTINGS', 'CASTLE', 'CASTOR', 'CATALYST', 'CATERERS', 'CATERING', 'CELL', 'CELLULAR', 'CELLULOSE', 'CEMENT', 'CEMENTS', 'CENTER', 'CENTERPRIVATE', 'CENTERS', 'CENTRAL', 'CENTRE', 'CENTREPRIVATE', 'CENTRES', 'CENTURION', 'CENTURY', 'CERA', 'CERAMIC', 'CERAMICS', 'CEREALS', 'CERTIFICATION', 'CERTIFICATIONS', 'CHAIN', 'CHAINS', 'CHAMBER', 'CHAMBERS', 'CHANGER', 'CHANGERS', 'CHANNEL', 'CHANNELS', 'CHARAN', 'CHARTERED', 'CHARTERING', 'CHEF', 'CHEM', 'CHEMI', 'CHEMICAL', 'CHEMICALS', 'CHEMICALSPRIVATE', 'CHEMIE', 'CHEMTECH', 'CHIP', 'CHIPS', 'CHIT', 'CHITFUND', 'CHITFUNDS', 'CHITS', 'CHITTIES', 'CHITTY', 'CHROME', 'CINE', 'CINEMA', 'CINEMAS', 'CIRCUIT', 'CIRCUITS', 'CLEANERS', 'CLEANING', 'CLEANTECH', 'CLEARING', 'CLINIC', 'CLINICAL', 'CLINICS', 'CLOTHING', 'CLOTHINGS', 'CLOUD', 'CLUB', 'CLUBS', 'CO.', 'CO.LTD.', 'CO.PRIVATE', 'CO.PVT', 'CO.PVT.LTD.', 'COACH', 'COACHING', 'COASTAL', 'COAT', 'COATED', 'COATERS', 'COATING', 'COATINGS', 'COATS', 'CODE', 'COFFEE', 'COGENT', 'COIN', 'COIR', 'COKE', 'COLDSTORAGE', 'COLLECTION', 'COLLECTIONS', 'COLLECTIVE', 'COLLEGE', 'COLLIERIES', 'COLLIERY', 'COLONISERS', 'COLONIZER', 'COLONIZERS', 'COLOR', 'COLORS', 'COLOUR', 'COLOURS', 'COMFORTS', 'COMM', 'COMMERCE', 'COMMERCIAL', 'COMMERCIALS', 'COMMODEAL', 'COMMODITIES', 'COMMODITY', 'COMMOSALE', 'COMMOSALES', 'COMMOTRADE', 'COMMUNICATION', 'COMMUNICATIONPRIVATE', 'COMMUNICATIONS', 'COMMUNICATIONSPRIVATE', 'COMMUNITY', 'COMNET', 'COMP', 'COMPACT', 'COMPANY', 'COMPANYLIMITED', 'COMPANYPRIVATE', 'COMPANYPVT', 'COMPANYPVT.LTD.', 'COMPASS', 'COMPETENT', 'COMPLETE', 'COMPLEX', 'COMPLIANCE', 'COMPNAY', 'COMPONENT', 'COMPONENTS', 'COMPONENTSPRIVATE', 'COMPOSITE', 'COMPOSITES', 'COMPREHENSIVE', 'COMPRESSORS', 'COMPTECH', 'COMPU', 'COMPUSOFT', 'COMPUTECH', 'COMPUTER', 'COMPUTERS', 'COMPUTING', 'COMPUTRONICS', 'COMTECH', 'COMTRADE', 'CONBUILD', 'CONCAST', 'CONCEPT', 'CONCEPTS', 'CONCERN', 'CONCLAVE', 'CONCORD', 'CONCORDE', 'CONCRETE', 'CONCRETES', 'CONDITIONING', 'CONDUCTORS', 'CONES', 'CONFECTIONERS', 'CONFECTIONERY', 'CONFIDENT', 'CONGLOMERATE', 'CONNECT', 'CONNECTION', 'CONNECTIONS', 'CONNECTORS', 'CONSERVATION', 'CONSOLIDATED', 'CONSORTIUM', 'CONSTECH', 'CONSTRO', 'CONSTRUCTION', 'CONSTRUCTIONCOMPANY', 'CONSTRUCTIONPRIVATE', 'CONSTRUCTIONS', 'CONSTRUCTIONSPRIVATE', 'CONSTRUCTORS', 'CONSTRUWELL', 'CONSULT', 'CONSULTANCIES', 'CONSULTANCY', 'CONSULTANCYPRIVATE', 'CONSULTANCYSERVICES', 'CONSULTANT', 'CONSULTANTPRIVATE', 'CONSULTANTS', 'CONSULTANTSPRIVATE', 'CONSULTING', 'CONSULTINGPRIVATE', 'CONSULTINGSERVICES', 'CONSUMABLES', 'CONSUMER', 'CONSUMERS', 'CONTACT', 'CONTAINER', 'CONTAINERS', 'CONTECH', 'CONTEMPORARY', 'CONTINENTAL', 'CONTRACT', 'CONTRACTING', 'CONTRACTOR', 'CONTRACTORS', 'CONTRACTORSPRIVATE', 'CONTRACTS', 'CONTROLS', 'CONVENTION', 'CONVERGENCE', 'CONVERSION', 'CONVERTERS', 'CONVERTORS', 'CONVEYOR', 'CONVEYORS', 'COOLING', 'COPPER', 'COPRIVATE', 'COPVT', 'CORAL', 'CORE', 'CORN', 'CORNER', 'CORNERSTONE', 'COROMANDEL', 'CORONATION', 'CORP', 'CORPN', 'CORPN.', 'CORPORATE', 'CORPORATION', 'CORPORATIONLIMITED', 'CORPORATIONPRIVATE', 'CORRUGATED', 'CORRUGATORS', 'COSMETIC', 'COSMETICS', 'COSMIC', 'COSMO', 'COSMOPOLITAN', 'COSMOS', 'COST', 'COTEX', 'COTSPIN', 'COTSYN', 'COTTAGE', 'COTTAGES', 'COTTON', 'COTTONS', 'COUNCIL', 'COUNTRY', 'COUNTRYWIDE', 'COUNTY', 'COURIER', 'COURIERS', 'COUTURE', 'COZY', 'CRAFT', 'CRAFTS', 'CREAMS', 'CREATION', 'CREATIONS', 'CREATIVE', 'CREATIVES', 'CREATORS', 'CREDIT', 'CREDITS', 'CROP', 'CROPS', 'CROSS', 'CROWN', 'CRUSHER', 'CRUSHERS', 'CRUSHING', 'CRYSTAL', 'CRYSTALS', 'CUISINE', 'CUISINES', 'CULINARY', 'CULTIVATION', 'CULTIVATORS', 'CULTURAL', 'CULTURE', 'CURE', 'CURRENCY', 'CUSTOMER', 'CUTTING', 'CYBER', 'CYBERNETICS', 'CYBERTECH', 'CYCLE', 'CYCLES', 'CYLINDERS', 'DAILY', 'DAIRIES', 'DAIRY', 'DATA', 'DATABASE', 'DATACOM', 'DATAMATICS', 'DATASOFT', 'DEALCOM', 'DEALCOMM', 'DEALER', 'DEALERS', 'DEALMARK', 'DEALS', 'DEALTRADE', 'DEBT', 'DECO', 'DECOR', 'DECORATIVE', 'DECORATORS', 'DECORS', 'DEFENCE', 'DEGREE', 'DELIVERY', 'DELUX', 'DELUXE', 'DEPARTMENTAL', 'DEPOSIT', 'DEPOSITS', 'DEPOT', 'DERIVATIVES', 'DESIGN', 'DESIGNER', 'DESIGNERS', 'DESIGNING', 'DESIGNS', 'DETECTIVE', 'DETECTIVES', 'DETERGENT', 'DETERGENTS', 'DEVBUILD', 'DEVCON', 'DEVELOPER', 'DEVELOPERPRIVATE', 'DEVELOPERS', 'DEVELOPERSLIMITED', 'DEVELOPERSPRIVATE', 'DEVELOPMENT', 'DEVELOPMENTCOMPANY', 'DEVELOPMENTCORPORATION', 'DEVELOPMENTLIMITED', 'DEVELOPMENTPRIVATE', 'DEVELOPMENTS', 'DIAGNOSTIC', 'DIAGNOSTICS', 'DIAM', 'DIAMOND', 'DIAMONDS', 'DIARY', 'DIES', 'DIESEL', 'DIESELS', 'DIGI', 'DIGITAL', 'DIGITECH', 'DIGNITY', 'DIMENSION', 'DIMENSIONS', 'DISCOUNT', 'DISPLAY', 'DISTILLERIES', 'DISTILLERS', 'DISTILLERY', 'DISTRIBUTING', 'DISTRIBUTION', 'DISTRIBUTIONPRIVATE', 'DISTRIBUTIONS', 'DISTRIBUTOR', 'DISTRIBUTORS', 'DISTRIBUTORSPRIVATE', 'DOCTORS', 'DOMAIN', 'DOMESTIC', 'DOORS', 'DOT', 'DOTCOM', 'DOTS', 'DP', 'DRESSES', 'DRILLING', 'DRINKS', 'DRUG', 'DRUGS', 'DRUMS', 'DWELLERS', 'DWELLING', 'DWELLINGS', 'DYE', 'DYE-CHEM', 'DYECHEM', 'DYEING', 'DYERS', 'DYES', 'DYESTUFF', 'DYESTUFFS', 'E-COM', 'E-COMMERCE', 'E-LEARNING', 'E-SERVICES', 'E-SOLUTIONS', 'EARTHMOVERS', 'EATABLES', 'ECO', 'ECOM', 'ECOMMERCE', 'ECONOMIC', 'ECOTECH', 'EDIBLE', 'EDIBLES', 'EDUCARE', 'EDUCATION', 'EDUCATIONAL', 'EDUCATIONPRIVATE', 'EDUCATIONS', 'EDUCOM', 'EDUCON', 'EDUSOFT', 'EDUSOLUTIONS', 'EDUTAINMENT', 'EDUTECH', 'EDUVENTURES', 'EFFLUENT', 'ELECTRIC', 'ELECTRICAL', 'ELECTRICALS', 'ELECTRICALSPRIVATE', 'ELECTRICITY', 'ELECTRICS', 'ELECTRO', 'ELECTRODES', 'ELECTROMECH', 'ELECTRONIC', 'ELECTRONICS', 'ELECTRONICSPRIVATE', 'ELECTROTECH', 'ELEMENTS', 'ELEVATOR', 'ELEVATORS', 'EMBROIDERIES', 'EMBROIDERY', 'EMERGENCY', 'EMERGING', 'EMPLOYEES', 'EMPLOYMENT', 'EMPORIUM', 'EMPOWER', 'ENCLAVE', 'ENCLAVES', 'ENERGIES', 'ENERGY', 'ENERGYPRIVATE', 'ENERTECH', 'ENFIELD', 'ENGG', 'ENGG.', 'ENGICON', 'ENGICONS', 'ENGINE', 'ENGINEER', 'ENGINEERING', 'ENGINEERINGCOMPANY', 'ENGINEERINGPRIVATE', 'ENGINEERINGS', 'ENGINEERINGSERVICES', 'ENGINEERS', 'ENGINEERSPRIVATE', 'ENGINES', 'ENGITECH', 'ENTERPRISE', 'ENTERPRISES', 'ENTERPRISESPRIVATE', 'ENTERTAINERS', 'ENTERTAINMENT', 'ENTERTAINMENTPRIVATE', 'ENTERTAINMENTS', 'ENTREPRENEUR', 'ENTREPRENEURS', 'ENVIROTECH', 'ENVISION', 'EQUIPMENT', 'EQUIPMENTS', 'EQUIPMENTSPRIVATE', 'EQUITIES', 'EQUITY', 'ESCALATORS', 'ESCORTS', 'ESERVICES', 'ESOLUTIONS', 'ESQUIRE', 'ESSENTIALS', 'ESTATE', 'ESTATEPRIVATE', 'ESTATES', 'ESTATESPRIVATE', 'EX-SERVICEMEN', 'EXECUTIVE', 'EXHIBITION', 'EXHIBITIONS', 'EXHIBITORS', 'EXPEDITIONS', 'EXPERIENCE', 'EXPERT', 'EXPERTS', 'EXPLORATION', 'EXPLOSIVES', 'EXPO', 'EXPORT', 'EXPORTERS', 'EXPORTS', 'EXPORTSPRIVATE', 'EXPOSITIONS', 'EXPOTRADE', 'EXTRACTION', 'EXTRACTIONS', 'EXTRACTS', 'EXTRUSION', 'EXTRUSIONS', 'FABRIC', 'FABRICATION', 'FABRICATIONS', 'FABRICATORS', 'FABRICS', 'FABS', 'FABTECH', 'FABTEX', 'FACADE', 'FACILITATORS', 'FACILITIES', 'FACILITY', 'FACILITYMANAGEMENT', 'FACTOR', 'FACTORS', 'FACTORY', 'FACTORYPRIVATE', 'FARMER', 'FARMERS', 'FARMERSPRODUCER', 'FARMING', 'FARMS', 'FARMSPRIVATE', 'FASHION', 'FASHIONS', 'FASTENERS', 'FASTNERS', 'FASTSPEED', 'FATS', 'FEED', 'FEEDS', 'FERROUS', 'FERTILISERS', 'FERTILITY', 'FERTILIZER', 'FERTILIZERS', 'FERTILIZERSPRIVATE', 'FIBER', 'FIBERS', 'FIBRE', 'FIBRES', 'FIELD', 'FIELDS', 'FILAMENT', 'FILAMENTS', 'FILM', 'FILMS', 'FILTER', 'FILTERS', 'FILTRATION', 'FIN-LEASE', 'FINANCE', 'FINANCECOMPANY', 'FINANCELIMITED', 'FINANCEPRIVATE', 'FINANCERS', 'FINANCES', 'FINANCIAL', 'FINANCIALCONSULTANTS', 'FINANCIALS', 'FINANCIALSERVICES', 'FINANCIERS', 'FINANCING', 'FINCAP', 'FINCO', 'FINCOM', 'FINCON', 'FINCORP', 'FINE', 'FINECHEM', 'FINESSE', 'FININVEST', 'FINISHERS', 'FINISHING', 'FINLEASE', 'FINMART', 'FINSEC', 'FINSERV', 'FINSERVE', 'FINSTOCK', 'FINTECH', 'FINTRADE', 'FINVEST', 'FISH', 'FISHERIES', 'FISHING', 'FIT', 'FITNESS', 'FITTINGS', 'FIXTURES', 'FLAT', 'FLATS', 'FLAVOURS', 'FLEET', 'FLEX', 'FLEXI', 'FLEXIBLE', 'FLEXIPACK', 'FLIGHT', 'FLOOR', 'FLOORING', 'FLOORINGS', 'FLOORS', 'FLORA', 'FLORAL', 'FLORENCE', 'FLORICULTURE', 'FLORITECH', 'FLOUR', 'FLOW', 'FLOWER', 'FLOWERS', 'FLUID', 'FLYING', 'FOAM', 'FOAMS', 'FOILS', 'FOOD', 'FOODS', 'FOODSPRIVATE', 'FOODTECH', 'FOOT', 'FOOTBALL', 'FOOTCARE', 'FOOTWEAR', 'FOOTWEARS', 'FORESTRY', 'FOREX', 'FORGE', 'FORGING', 'FORGINGS', 'FORMING', 'FORMS', 'FORMULATION', 'FORMULATIONS', 'FORTUNE', 'FORUM', 'FORWARDERS', 'FOSTER', 'FOTO', 'FOUNDATION', 'FOUNDATIONS', 'FOUNDERS', 'FOUNDRIES', 'FOUNDRY', 'FRAGRANCE', 'FRAGRANCES', 'FRAME', 'FRAMES', 'FRANCHISE', 'FREIGHT', 'FROZEN', 'FRUIT', 'FRUITS', 'FUEL', 'FUELS', 'FUND', 'FUNDLIMITED', 'FUNDPRIVATE', 'FUNDS', 'FUNDSPRIVATE', 'FUNIDEA', 'FURNACE', 'FURNACES', 'FURNISHERS', 'FURNISHING', 'FURNISHINGS', 'FURNITURE', 'FURNITURES', 'FUTURES', 'GADGETS', 'GAIN', 'GALLERY', 'GAME', 'GAMES', 'GAMING', 'GARDENS', 'GARMENT', 'GARMENTS', 'GAS', 'GASES', 'GASKETS', 'GATEWAY', 'GAUGES', 'GEAR', 'GEARS', 'GEMS', 'GENERATORS', 'GENETIC', 'GENETICS', 'GENUINE', 'GEOTECH', 'GIFTS', 'GINNING', 'GLASS', 'GLASSES', 'GLAZE', 'GLAZING', 'GLOVES', 'GOODS', 'GRAINS', 'GRANITE', 'GRANITES', 'GRAPES', 'GRAPHICS', 'GRAPHITE', 'GREENTECH', 'GREY', 'GROUP', 'HABITATS', 'HANDICRAFT', 'HANDICRAFTS', 'HANDLING', 'HANDLOOM', 'HANDLOOMS', 'HARBOUR', 'HARDWARE', 'HARDWARES', 'HEALTHCARE', 'HEALTHTECH', 'HELPLINE', 'HERBALS', 'HERBS', 'HERITAGE', 'HI-TECH', 'HITECH', 'HOLDING', 'HOLDINGS', 'HOLDINGSPRIVATE', 'HOLIDAYS', 'HOSPITAL', 'HOSPITALITIES', 'HOSPITALITY', 'HOSPITALITYPRIVATE', 'HOSPITALPRIVATE', 'HOSPITALS', 'HOSPITALSPRIVATE', 'HOTELS', 'HOTELSPRIVATE', 'HOUSING', 'HYBRID', 'HYDEL', 'HYDRAULIC', 'HYDRAULICS', 'HYDRO', 'HYDROCARBONS', 'HYGIENE', 'HYGIENIC', 'I-TECH', 'I.T.', 'IMAGINE', 'IMAGING', 'IMPORT', 'IMPORTERS', 'IMPORTS', 'INDUSTRIAL', 'INDUSTRIALS', 'INDUSTRIES', 'INDUSTRIESLIMITED', 'INDUSTRIESPRIVATE', 'INDUSTRIESPVT', 'INDUSTRY', 'INFIN', 'INFINITE', 'INFINITI', 'INFINITY', 'INFO', 'INFO-TECH', 'INFOCARE', 'INFOCOM', 'INFOCOMM', 'INFOMATICS', 'INFOMEDIA', 'INFORMATIC', 'INFORMATICS', 'INFORMATION', 'INFORMATIONTECHNOLOGY', 'INFOSERVE', 'INFOSERVICES', 'INFOSOFT', 'INFOSOL', 'INFOSOLUTION', 'INFOSOLUTIONS', 'INFOSYS', 'INFOSYSTEM', 'INFOSYSTEMS', 'INFOTAINMENT', 'INFOTEC', 'INFOTECH', 'INFOTEK', 'INFOTEL', 'INFOVISION', 'INFOWARE', 'INFOWAY', 'INFOWAYS', 'INFOWEB', 'INFRA', 'INFRA-TECH', 'INFRAA', 'INFRABUILD', 'INFRABUILDCON', 'INFRABUILDERS', 'INFRACARE', 'INFRACITY', 'INFRACOM', 'INFRACON', 'INFRACONS', 'INFRACONSTRUCTION', 'INFRADEV', 'INFRADEVELOPER', 'INFRADEVELOPERS', 'INFRAENGINEERS', 'INFRAESTATE', 'INFRAESTATES', 'INFRAHEIGHT', 'INFRAPROJECT', 'INFRAPROJECTS', 'INFRAREALTY', 'INFRASERVICES', 'INFRASOLUTION', 'INFRASOLUTIONS', 'INFRASPACE', 'INFRASTRUCTURAL', 'INFRASTRUCTURE', 'INFRASTRUCTUREDEVELOPMENT', 'INFRASTRUCTURELIMITED', 'INFRASTRUCTUREPRIVATE', 'INFRASTRUCTURES', 'INFRASTRUCTURESPRIVATE', 'INFRATEC', 'INFRATECH', 'INFRATEL', 'INFRATRADE', 'INFRAVENTURE', 'INFRAVENTURES', 'INFRAVISION', 'INFRAZONE', 'INGENIOUS', 'INGOTS', 'INGREDIENTS', 'INITIATIVE', 'INITIATIVES', 'INK', 'INKS', 'INLAND', 'INN', 'INNOTECH', 'INNOVA', 'INNOVATION', 'INNOVATIONS', 'INNOVATIVE', 'INNOVATORS', 'INNOVISION', 'INORGANICS', 'INSECTICIDES', 'INSPECTION', 'INSTITUTE', 'INSTITUTEPRIVATE', 'INSTITUTES', 'INSTITUTION', 'INSTITUTIONS', 'INSTRUMENT', 'INSTRUMENTATION', 'INSTRUMENTS', 'INSULATION', 'INSULATIONS', 'INSULATORS', 'INSURANCE', 'INTECH', 'INTEGRA', 'INTEGRAL', 'INTEGRATED', 'INTEGRATION', 'INTEGRATORS', 'INTEGRITY', 'INTEL', 'INTELLECT', 'INTELLECTUAL', 'INTELLIGENCE', 'INTELLIGENT', 'INTER', 'INTERACTIVE', 'INTERCONTINENTAL', 'INTERFACE', 'INTERIORS', 'INTERLINK', 'INTERMEDIATES', 'INTERNATIONAL', 'INTERNATIONALPRIVATE', 'INTERNET', 'INTERTRADE', 'INTIMATE', 'INTIME', 'INTRA', 'INTRADE', 'INVENTIVE', 'INVEST', 'INVESTIGATION', 'INVESTMENT', 'INVESTMENTCOMPANY', 'INVESTMENTLIMITED', 'INVESTMENTPRIVATE', 'INVESTMENTS', 'INVESTMENTSLIMITED', 'INVESTMENTSPRIVATE', 'INVESTOR', 'INVESTORS', 'INVESTRADE', 'IRRIGATION', 'JEWEL', 'JEWELLERS', 'JEWELLERSPRIVATE', 'JEWELLERY', 'JEWELRY', 'JEWELS', 'KITCHEN', 'KITCHENS', 'KNIT', 'KNITS', 'KNITTERS', 'KNITTING', 'KNITWEAR', 'KNITWEARS', 'LAB', 'LABEL', 'LABELS', 'LABORATORIES', 'LABORATORIESPRIVATE', 'LABORATORY', 'LABOUR', 'LABS', 'LAMINATES', 'LAMINATIONS', 'LAMINATORS', 'LAMP', 'LAMPS', 'LANDDEVELOPERS', 'LATEX', 'LAUNDRY', 'LAW', 'LEAD', 'LEADING', 'LEARNING', 'LEASE', 'LEASING', 'LEASINGLIMITED', 'LEASINGPRIVATE', 'LEASINGS', 'LEATHER', 'LEATHERS', 'LEISURES', 'LIFESTYLE', 'LIFESTYLES', 'LIFT', 'LIFTS', 'LIGHT', 'LIGHTING', 'LIGHTINGS', 'LIGHTS', 'LIMITED', 'LIMITED(OPC)', 'LIMITED,', 'LIMITED.', 'LIMTED', 'LIMTIED', 'LINERS', 'LINES', 'LINK', 'LINKERS', 'LINKS', 'LIQUOR', 'LIQUORS', 'LOAN', 'LOANS', 'LOCKERS', 'LOCKS', 'LODGE', 'LOGIC', 'LOGICAL', 'LOGICS', 'LOGISTIC', 'LOGISTICS', 'LOGISTICSPRIVATE', 'LTD', 'LTD,', 'LTD.', 'LTD.,', 'LUBE', 'LUBES', 'LUBRICANTS', 'LUGGAGE', 'LUMBERS', 'MACHINE', 'MACHINERIES', 'MACHINERY', 'MACHINES', 'MACHINING', 'MACRO', 'MAGNET', 'MAGNETIC', 'MAGNETICS', 'MAINTENANCE', 'MAKERS', 'MALLS', 'MANAGED', 'MANAGEMENT', 'MANAGEMENTCONSULTANTS', 'MANAGEMENTPRIVATE', 'MANAGEMENTS', 'MANAGEMENTSERVICES', 'MANAGERS', 'MANAGMENT', 'MANUFACTURERS', 'MANUFACTURERSPRIVATE', 'MANUFACTURES', 'MANUFACTURING', 'MANUFACTURINGCOMPANY', 'MANUFACTURINGPRIVATE', 'MARBLE', 'MARBLES', 'MARKET', 'MARKETERS', 'MARKETING', 'MARKETINGPRIVATE', 'MARKETINGS', 'MARKETINGSERVICES', 'MARKETPLACE', 'MARKETS', 'MARKETTING', 'MATCH', 'MATCHES', 'MATE', 'MATERIAL', 'MATERIALS', 'MATRIMONIAL', 'MECH', 'MECHANICAL', 'MECHANICALS', 'MECHATRONICS', 'MED', 'MEDIA', 'MEDIATECH', 'MEDICAL', 'MEDICALS', 'MEDICAMENTS', 'MEDICINE', 'MEDICINES', 'MEDICO', 'MEDICOS', 'MEDITECH', 'MEDTECH', 'MEENA', 'MEENAKSHI', 'MEERA', 'MEERUT', 'MEET', 'MEGA', 'MEGACITY', 'MEGAMART', 'MEGH', 'MEGHA', 'MEGHALAYA', 'MEGHANA', 'MEGHDOOT', 'MEGHNA', 'MEHAK', 'MEHAR', 'MEHER', 'MEHRA', 'MEHTA', 'MEHUL', 'MELANGE', 'MELODY', 'MEMORIAL', 'MEMORY', 'MEN', 'MENON', 'MENTOR', 'MENTORS', 'MEP', 'MERA', 'MERCANTILE', 'MERCANTILES', 'MERCHANDISE', 'MERCHANDISERS', 'MERCHANDISING', 'MERCHANT', 'MERCHANTILE', 'MERCHANTS', 'MERCURY', 'MERIDIAN', 'MERIT', 'MERLIN', 'MERRY', 'MERU', 'MESH', 'MET', 'META', 'METACAST', 'METAL', 'METALIKS', 'METALLIC', 'METALLICS', 'METALLOYS', 'METALLURGICAL', 'METALLURGICALS', 'METALS', 'METERS', 'METRO', 'METROPOLIS', 'METROPOLITAN', 'MEWAR', 'MFG', 'MFG.', 'MG', 'MICA', 'MICRO', 'MICROFINANCE', 'MICRON', 'MICRONS', 'MICROSYSTEMS', 'MICROTECH', 'MIDAS', 'MIDDLE', 'MIDLAND', 'MIDWAY', 'MIGRATION', 'MILE', 'MILES', 'MILESTONE', 'MILK', 'MILL', 'MILLENIUM', 'MILLENNIUM', 'MILLERS', 'MILLING', 'MILLS', 'MILLSLIMITED', 'MILLSPRIVATE', 'MINAR', 'MINCHEM', 'MIND', 'MINDA', 'MINDS', 'MINE', 'MINECHEM', 'MINERAL', 'MINERALS', 'MINERALSPRIVATE', 'MINERVA', 'MINES', 'MINI', 'MINING', 'MINT', 'MIR', 'MIRA', 'MIRACLE', 'MIRAGE', 'MIRAJ', 'MIRROR', 'MISSION', 'MIST', 'MOBILE', 'MOBILES', 'MOBILITY', 'MOBITECH', 'MOD', 'MODE', 'MODEL', 'MODULAR', 'MOMENTUM', 'MONARCH', 'MONEY', 'MONITORING', 'MONSOON', 'MOON', 'MOONLIGHT', 'MOONLINK', 'MOONSHINE', 'MOONSTAR', 'MOSAIC', 'MOTEL', 'MOTELS', 'MOTION', 'MOTOR', 'MOTORS', 'MOULD', 'MOULDERS', 'MOULDING', 'MOULDINGS', 'MOULDS', 'MOVERS', 'MOVIE', 'MOVIES', 'MOVING', 'MULTI', 'MULTI-TRADE', 'MULTICON', 'MULTILEVEL', 'MULTILINK', 'MULTIMEDIA', 'MULTIPLE', 'MULTIPLEX', 'MULTIPROJECTS', 'MULTIPURPOSE', 'MULTISERVICES', 'MULTISOLUTIONS', 'MULTISPECIALITY', 'MULTITECH', 'MULTITRADE', 'MULTITRADING', 'MULTIVENTURES', 'NANOTECH', 'NATURALS', 'NEOTECH', 'NETCOM', 'NETLINK', 'NETMART', 'NETS', 'NETWORK', 'NETWORKING', 'NETWORKPRIVATE', 'NETWORKS', 'NETWORTH', 'NEURO', 'NURSING', 'NUTRACEUTICALS', 'NUTRI', 'NUTRIENTS', 'NUTRITION', 'NUTRITIONS', 'OIL', 'OILFIELD', 'OILS', 'OLIVE', 'ONCOLOGY', 'OPERATORS', 'OPTION', 'OPTIONS', 'ORCHID', 'ORCHIDS', 'ORGANISATION', 'ORGANISERS', 'ORGANIZATION', 'ORIENTAL', 'ORNATE', 'ORTHO', 'OUTSOURCE', 'OUTSOURCING', 'OVERSEAS', 'OWNERS', 'OXYGEN', 'OYSTER', 'OZONE', 'P.LTD.', 'PACK', 'PACKAGERS', 'PACKAGES', 'PACKAGING', 'PACKAGINGS', 'PACKERS', 'PACKING', 'PACKS', 'PAGES', 'PAINT', 'PAINTS', 'PALACE', 'PALMS', 'PANEL', 'PANELS', 'PAPER', 'PAPERS', 'PARADISE', 'PARAM', 'PARKPRIVATE', 'PARTNERS', 'PAYMENT', 'PAYMENTS', 'PEARL', 'PEARLS', 'PELICAN', 'PENS', 'PEOPLE', 'PEOPLES', 'PERFUMES', 'PERIPHERALS', 'PERSONNEL', 'PEST', 'PESTICIDES', 'PET', 'PETALS', 'PETRO', 'PETROCHEM', 'PETROCHEMICALS', 'PETROLEUM', 'PETROLEUMS', 'PHARMA', 'PHARMACEUTICAL', 'PHARMACEUTICALS', 'PHARMACEUTICALSPRIVATE', 'PHARMACHEM', 'PHARMACY', 'PHARMATECH', 'PHOENIX', 'PHOSPHATES', 'PHOTO', 'PHOTOGRAPHY', 'PHOTON', 'PICTURE', 'PICTURES', 'PIGMENTS', 'PILOT', 'PIPE', 'PIPES', 'PIPING', 'PLACEMENT', 'PLACEMENTS', 'PLANNER', 'PLANNERS', 'PLANNING', 'PLANTATION', 'PLANTATIONS', 'PLANTERS', 'PLANTS', 'PLASMA', 'PLASTECH', 'PLASTIC', 'PLASTICS', 'PLASTO', 'PLASTOCHEM', 'PLATFORM', 'PLATINUM', 'PLAY', 'PLAZA', 'PLAZZA', 'PLEASANT', 'PLEASURE', 'PLEXUS', 'PLYWOOD', 'PLYWOODS', 'PNEUMATICS', 'POINT', 'POLAR', 'POLARIS', 'POLE', 'POLES', 'POLESTAR', 'POLLUTION', 'POLY', 'POLYCHEM', 'POLYCLINIC', 'POLYCOATS', 'POLYESTER', 'POLYESTERS', 'POLYFAB', 'POLYFILMS', 'POLYGON', 'POLYMER', 'POLYMERS', 'POLYPACK', 'POLYPACKS', 'POLYPLAST', 'POLYSACKS', 'POLYTECH', 'POLYTEX', 'PONDICHERRY', 'PORT', 'PORTFOLIOS', 'PORTS', 'PORWAL', 'POST', 'POTENTIAL', 'POULTRIES', 'POULTRY', 'POWDER', 'POWDERS', 'POWERGEN', 'POWERS', 'POWERTECH', 'POWERTRONICS', 'PPRIVATE', 'PRECISIONS', 'PREMIUM', 'PRESS', 'PRESSINGS', 'PRESSURE', 'PRESTIGE', 'PRIAVTE', 'PRIMARY', 'PRIME', 'PRINTING', 'PRINTPACK', 'PRINTS', 'PRISM', 'PRIVATE', 'PRIVATELIMITED', 'PRIVATELIMITED.', 'PRIVATELTD', 'PROACTIVE', 'PROBUILD', 'PROCESS', 'PROCESSED', 'PROCESSES', 'PROCESSING', 'PROCESSINGPRIVATE', 'PROCESSOR', 'PROCESSORS', 'PROCON', 'PRODIGY', 'PRODUCE', 'PRODUCER', 'PRODUCERCOMPANY', 'PRODUCERS', 'PRODUCT', 'PRODUCTION', 'PRODUCTIONS', 'PRODUCTS', 'PRODUCTSLIMITED', 'PRODUCTSPRIVATE', 'PROFESSIONAL', 'PROFESSIONALS', 'PROFICIENT', 'PROFILE', 'PROFILES', 'PROFIT', 'PROGRESS', 'PROGRESSIVE', 'PROJECT', 'PROJECTS', 'PROJECTSLIMITED', 'PROJECTSPRIVATE', 'PROLIFIC', 'PROMINENT', 'PROMOTER', 'PROMOTERS', 'PROMOTERSPRIVATE', 'PROMOTION', 'PROMOTIONS', 'PROMOTORS', 'PROMPT', 'PROPBUILD', 'PROPERTIES', 'PROPERTIESPRIVATE', 'PROPERTY', 'PROPMART', 'PROPTECH', 'PROSPERITY', 'PROTECH', 'PROTECT', 'PROTECTIVE', 'PROTEINS', 'PROVIDENT', 'PROVIDER', 'PROVIDERS', 'PUBLICATION', 'PUBLICATIONS', 'PUBLICATIONSPRIVATE', 'PUBLISHERS', 'PUBLISHING', 'PULSES', 'PUMP', 'PUMPS', 'PVT.', 'PVT.LTD', 'PVT.LTD.', 'PVTLTD', 'PVTLTD.', 'QUALITY', 'QUARRIES', 'QUARRY', 'QUARTZ', 'QUEEN', 'RACING', 'RADIATORS', 'RAYON', 'RAYONS', 'REAL', 'REALATORS', 'REALBUILD', 'REALCON', 'REALESTATE', 'REALESTATES', 'REALHOME', 'REALINFRA', 'REALITIES', 'REALITY', 'REALMART', 'REALTECH', 'REALTERS', 'REALTIES', 'REALTIME', 'REALTOR', 'REALTORS', 'REALTORSPRIVATE', 'REALTY', 'RECKON', 'RECON', 'RECONSTRUCTION', 'RECORDING', 'RECORDS', 'RECREATION', 'RECREATIONS', 'RECRUITERS', 'RECRUITMENT', 'RECYCLERS', 'RECYCLING', 'REFINERIES', 'REFINERY', 'REFRACTORIES', 'REFRACTORY', 'REFRIGERATION', 'REMEDIES', 'RENAISSANCE', 'RENEWABLE', 'RENEWABLES', 'REROLLING', 'RESEARCH', 'RESEARCHCENTRE', 'RESEARCHFOUNDATION', 'RESEARCHINSTITUTE', 'RESEARCHPRIVATE', 'RESIDENCES', 'RESIDENCY', 'RESIDENTIAL', 'RESINS', 'RESONANCE', 'RESORT', 'RESORTS', 'RESORTSPRIVATE', 'RESOURCE', 'RESOURCES', 'RESOURCESPRIVATE', 'RESPONSE', 'RESTAURANT', 'RESTAURANTS', 'RETAIL', 'RETAILERS', 'RETAILING', 'RETAILS', 'RETREADING', 'RETREADS', 'RETREAT', 'RETREATS', 'ROADLINES', 'ROADS', 'ROADWAYS', 'ROBOTICS', 'RUBBER', 'RUBBERS', 'RUIA', 'SAPHIRE', 'SAPPHIRE', 'SAR', 'SAREE', 'SAREES', 'SATELLITE', 'SAVING', 'SAVINGS', 'SAW', 'SCAFFOLDING', 'SCAN', 'SCANS', 'SCHEME', 'SCHOOL', 'SCHOOLS', 'SCIENCE', 'SCIENCES', 'SCIENCESPRIVATE', 'SCIENTIFIC', 'SEAFOODS', 'SEAL', 'SEALS', 'SECURITIES', 'SECURITIESPRIVATE', 'SECURITY', 'SECURITYSERVICES', 'SEED', 'SEEDS', 'SENSORS', 'SERICULTURE', 'SERVICE', 'SERVICEPRIVATE', 'SERVICES', 'SERVICES(INDIA)', 'SERVICESINDIA', 'SERVICESLIMITED', 'SERVICESPRIVATE', 'SERVICESPVT', 'SHADES', 'SHARES', 'SHEETS', 'SHELTER', 'SHELTERS', 'SHIPBREAKING', 'SHIPYARD', 'SHOE', 'SHOES', 'SHOP', 'SHOPPE', 'SHOPPEE', 'SHOPPERS', 'SHOPPING', 'SIGNAGE', 'SIGNS', 'SILICA', 'SILICATE', 'SILICATES', 'SILICON', 'SILK', 'SILKS', 'SILVER', 'SILVERLINE', 'SIZING', 'SKILLS', 'SMITH', 'SNACKS', 'SOAP', 'SOAPS', 'SOFT-TECH', 'SOFTECH', 'SOFTSOLUTIONS', 'SOFTTECH', 'SOFTWARE', 'SOFTWARES', 'SOLUTION', 'SOLUTIONPRIVATE', 'SOLUTIONS', 'SOLUTIONS(INDIA)', 'SOLUTIONSPRIVATE', 'SOLVENT', 'SOLVENTS', 'SONAR', 'SOURCES', 'SOURCING', 'SPARE', 'SPARES', 'SPARKLE', 'SPICE', 'SPICES', 'SPINNERS', 'SPINNING', 'SPIRIT', 'SPIRITS', 'SPONGE', 'SPORTING', 'SPORTS', 'STAINLESS', 'STANDARD', 'STATIONERS', 'STATIONERY', 'STEEL', 'STEELS', 'STOCK', 'STOCKS', 'STONE', 'STONES', 'STORAGE', 'STORAGEPRIVATE', 'STORAGES', 'STORE', 'STORES', 'STRATEGIES', 'STRATEGY', 'STRUCTURALS', 'STRUCTURE', 'STRUCTURES', 'STUDIO', 'STUDIOS', 'STYLE', 'STYLES', 'STYLISH', 'SUGAR', 'SUGARCANE', 'SUGARS', 'SUITES', 'SUITINGS', 'SUPPLIER', 'SUPPLIERS', 'SUPPLIERSPRIVATE', 'SUPPLIES', 'SUPPLY', 'SURFACE', 'SURFACTANTS', 'SURGE', 'SURGICAL', 'SURGICALS', 'SURVEILLANCE', 'SURVEYORS', 'SURVEYS', 'SUSTAINABLE', 'SWEET', 'SWEETS', 'SYNAPSE', 'SYNERGIES', 'SYNERGY', 'SYNTHETIC', 'SYNTHETICS', 'SYSTECH', 'SYSTEM', 'SYSTEMATIC', 'SYSTEMPRIVATE', 'SYSTEMS', 'SYSTEMSPRIVATE', 'TALKIES', 'TANK', 'TANKS', 'TANNERIES', 'TANNERS', 'TANNERY', 'TANNING', 'TAPE', 'TAPES', 'TAXI', 'TAYAL', 'TEAK', 'TEC', 'TECH', 'TECHLABS', 'TECHNET', 'TECHNICAL', 'TECHNICS', 'TECHNIK', 'TECHNIQUES', 'TECHNO', 'TECHNOBUILD', 'TECHNOCAST', 'TECHNOCOM', 'TECHNOCRAFT', 'TECHNOCRAT', 'TECHNOCRATE', 'TECHNOCRATES', 'TECHNOCRATS', 'TECHNOFAB', 'TECHNOLABS', 'TECHNOLOGICAL', 'TECHNOLOGIES', 'TECHNOLOGIESPRIVATE', 'TECHNOLOGY', 'TECHNOLOGYPRIVATE', 'TECHNOPLAST', 'TECHNOSOFT', 'TECHNOSOLUTIONS', 'TECHNOSYS', 'TECHNOVISION', 'TECHSOFT', 'TECHSOLUTIONS', 'TELEMATICS', 'TELEMEDIA', 'TELENET', 'TELESERVICES', 'TELESOFT', 'TELESYSTEMS', 'TELETECH', 'TELEVENTURES', 'TELEVISION', 'TERMINALS', 'TEXFAB', 'TEXO', 'TEXTILE', 'TEXTILES', 'TEXTRADE', 'TEXTURISERS', 'TEXTURISING', 'THERAPEUTICS', 'THERAPY', 'THERMAL', 'THERMO', 'THINK', 'THREADS', 'TILE', 'TILES', 'TIMBER', 'TIMBERS', 'TISSUE', 'TISSUES', 'TOBACCO', 'TOLL', 'TOLLWAY', 'TOLLWAYS', 'TOOL', 'TOOLING', 'TOOLINGS', 'TOOLS', 'TOPAZ', 'TOPLINE', 'TOPSTAR', 'TORRENT', 'TOURISM', 'TOURIST', 'TOURS', 'TOWERS', 'TOWNSHIPS', 'TOYS', 'TRADE', 'TRADE-LINK', 'TRADECOM', 'TRADECOMM', 'TRADECON', 'TRADEFIN', 'TRADELINK', 'TRADELINKS', 'TRADEMART', 'TRADEPLACE', 'TRADER', 'TRADERS', 'TRADERSPRIVATE', 'TRADES', 'TRADEVIN', 'TRADEVISION', 'TRADEWELL', 'TRADEWING', 'TRADEWINGS', 'TRADEX', 'TRADEXIM', 'TRADING', 'TRADINGCOMPANY', 'TRADINGPRIVATE', 'TRADINGS', 'TRAFFIC', 'TRAINERS', 'TRAINING', 'TRAININGPRIVATE', 'TRANSCRIPTION', 'TRANSFORMER', 'TRANSFORMERS', 'TRANSIT', 'TRANSLOGISTICS', 'TRANSMISSION', 'TRANSMISSIONS', 'TRANSPORT', 'TRANSPORTATION', 'TRANSPORTERS', 'TRANSPORTS', 'TRANSTECH', 'TRANSWAYS', 'TRANSWORLD', 'TRAUMA', 'TRAVELLERS', 'TRAVELS', 'TRAVELSPRIVATE', 'TREADS', 'TRENDS', 'TRENDZ', 'TREXIM', 'TRIO', 'TUBE', 'TUBES', 'TYRE', 'TYRES', 'UMBRELLA', 'UNION', 'UNITECH', 'UTENSILS', 'UTILITIES', 'VACATIONS', 'VALUERS', 'VALVES', 'VAULT', 'VAULTS', 'VEHICLES', 'VENEER', 'VENEERS', 'VENTURA', 'VENTURESPRIVATE', 'VIDEO', 'WAREHOUSE', 'WAREHOUSING', 'WARES', 'WATCH', 'WATCHES', 'WATERPROOFING', 'WAVES', 'WEAR', 'WEARS', 'WEAVE', 'WEAVERS', 'WEAVES', 'WEAVING', 'WEB', 'WEBSOFT', 'WEBSOLUTIONS', 'WEBTECH', 'WEIGHING', 'WELD', 'WELDING', 'WELFARE', 'WHEAT', 'WHEEL', 'WHEELS', 'WHOLESALE', 'WINDOW', 'WINDOWS', 'WINE', 'WINERY', 'WINES', 'WING', 'WINGS', 'WIRES', 'WOOL', 'WOOLLEN', 'WOOLLENS', 'WOOLTEX', 'WORKSHOP', 'WORKSPRIVATE', 'X', 'YARD', 'YARN', 'YARNS', 'ZINC', 'ZIP', 'ZIPPERS', 'PVT']
        stopwords=["PRIVATE", "PRIVATELIMITED", "PRIVATELIMITED.", "PRIVATELTD", "LIMITED", "LIMITED(OPC)", "LIMITED,", "LIMITED.", "LIMTED", "LIMTIED","LTD", "LTD,", "LTD.", "LTD.,","PVT.", "PVT.LTD", "PVT.LTD.", "PVTLTD", "PVTLTD."]
        
        
        if self.petfirm:
            
            query=(self.petitioner.upper()).split()
            resultwords  = [word for word in query if word not in stopwords]
            result = ' '.join(resultwords)
            self.petname=result
            
            
        
        
        else:
            self.petname=None
            
        if self.resfirm:
            
            query=(self.respondent.upper()).split()
            resultwords  = [word for word in query if word not in stopwords]
            result = ' '.join(resultwords)
            self.resname=result
            
        else:
            self.resname=None
        
    
    
    def get_all_info(self):
        info = {}

        self.get_citations()
        self.get_jud_order()
        self.get_eq_citations()
        self.get_court()
        self.get_bench()
        self.get_author()
        self.get_title()
        #self.get_pre()
        self.check_petitioner_firm()
        self.check_respondent_firm()
        #self.check_petitioner_gov()
        #self.check_respondent_gov()
        self.preprocess()
        

        info['court'] = self.court
        info['author'] = self.author
        info['title'] = self.title
        info['petitioner'] = self.petitioner
        info['respondent'] = self.respondent
        info['doc_date'] = self.date
        #info['pre'] = self.pre
        info['citations'] = self.citations
        info['eq_citations'] = self.eq_citations
        info['judgment_order'] = self.jud_order
        info['check_petitioner_firm']=self.petfirm
        info['check_respondent_firm']=self.resfirm
        info['pet_name']=self.petname
        info['res_name']=self.resname
        #info['check_petitioner_gov']=self.petgov
        #info['check_respondent_gov']=self.resgov
        return info

            
            


if __name__ == "__main__":
    dir= "directory_name"
    c=1
    List=[]
    List1=[]
    for filename in os.listdir(dir):
        
        
        if c==20000:
            break
                
        filepath=os.path.join(dir, filename)
        myfile=Path(filepath)  
        #print(filepath)
        case = extractMetadata(filepath)  
        info = case.get_all_info()

        #print(info)
        if info['check_petitioner_firm'] :
            List1.append(info['petitioner'])
            List.append(info['pet_name'])
        
        
        if info['check_respondent_firm']:
            List1.append(info['respondent'])
            List.append(info['res_name'])
        c=c+1   
    #print(List)
   
    query=List[20]
    print(query)
    results=process.extract(query,choices,limit=3)
    print(results)

SUMITRA SHARMA & ANR.
[('SHARMA KALYPSO PRIVATE LIMITED   ', 86), ('SATISH SHARMA ROAD BUILDER PRIVATELIMITED  ', 86), ('R K SHARMA CONSTRUCTION PRIVATE LIMITED   ', 86)]


In [2]:
print(c)

7579


In [ ]:

import pandas as pd
data = pd.read_excel('/home/ojas_d/WorldBank/Registered_companies/raw_data/Delhi_2016.xlsx')
choices=data['COMPANY_NAME'].tolist()
company_names = data['COMPANY_NAME'].tolist()
#Removing repeated words from company names which might hinder clustering
stopwords1=["PVT","M","S","ORS","ANR","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "the" ,"whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "t", "can", "will", "just", "don", "should", "now","M/S","ANR","PRIVATE", "PRIVATELIMITED", "PRIVATELIMITED.", "PRIVATELTD", "LIMITED", "LIMITED(OPC)", "LIMITED,", "LIMITED.", "LIMTED", "LIMTIED","LTD", "LTD,", "LTD.", "LTD.,","PVT.", "PVT.LTD", "PVT.LTD.", "PVTLTD", "PVTLTD."]
clean_company_names=[]
i=0;
for company in company_names:
    
    if i==20000:
        break
     
    query=(company.upper()).split()
    resultwords  = [word for word in query if word not in stopwords]
    result = ' '.join(resultwords)
    clean_company_names.append(result)
    i=i+1
    
for i in List:
    clean_company_names.append(i)
print(clean_company_names)
    




In [22]:
len(clean_company_names)

28516

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(clean_company_names)

In [ ]:
pip install sparse_dot_topn

In [24]:
print(tf_idf_matrix[0])

  (0, 4513)	0.6743613833261672
  (0, 14571)	0.7384014657883732


In [25]:

import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.29068851470947266


In [27]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})






matches_df = get_matches_df(matches, clean_company_names, top=1000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

,left_side,right_side,similairity
141,SEARCH BUILD TECH,SEARCH TECH INDIA,0.814534
231,RAMA SANDESH DEVELOPERS PVT,RAMA SANDESH BUILDWELL,0.812467
827,KULLU VALLEY LEISURE RESORTS PVT,KULLU VALLEY LEISURE RESORTS ...,0.957491
183,SEVEN HEAVEN CONSTRUCTIONS,SEVEN HEAVEN IMPEX,0.841584
310,ABB POWER,ABB INDIA,0.827938
592,ORANGE ASHOK NORTH WIND POWER,ORANGE ASHOK WIND POWER,0.879980
652,K.B. POLYCHEM (INDIA),P D POLYCHEM,0.937767
620,AMBIENCE IMPEX,AMBIENCE INDIA,0.832516
126,RAJ DEVCON,R.S. DEVCON,0.814227
21,VYOM INFRASTRUCTURES,VYOM INFRASTRUCTURES & PROJECTS,0.915835
